<a href="https://colab.research.google.com/github/sebatlab/Antaki2021/blob/main/Antaki2021_R_analysis.2022_06_13_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents
-------------------
* [Variance Explained by Genetic Risk Factors](#variance_explained_by_genetic_risk_factors)
* [Negative Correlations of Rare and Common](#negative_correlations_rare_common)
* [Gene-Phenotype Correlations](#gene_phenotype_correlations)

In [ ]:
#@title
# Install packages
install.packages("gplots")
install.packages("basicTrendline")

# Attaching packages
library(gplots)
library(basicTrendline)


# Figure 2A packages
install.packages("fmsb")
install.packages("lmtest")
install.packages("boot")
install.packages("stringr")

library(fmsb)
library(lmtest)
library(boot)
library(stringr)

install.packages("collections")
library(collections)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Input filenames and output filenames. I should add which sections take in
# which inputs and outputs.

filename_table = "SummaryGeneticData_REACH_SSC_SPARK.SBayesR.csv"
filename_phen  = "master_phen_4.SBayesR.csv"

r2_out = "R2_estimates.20210715.csv"
responses_out = "SummaryGeneticData_REACH_SSC_SPARK.responses.SBayesR.20210715.csv"
asd_genetic_correlations_by_sex_out = "asd_genetic_correlations_by_sex_final.SBayesR.20210715.tsv"
clin_results_out = "clin_results_eur.SBayesR.20210715.csv"

<a name="variance_explained_by_genetic_risk_factors"></a>
## Variance Explained by Genetic Risk Factors (Fig. 2A)

In [ ]:
# Load table
df <- read.table(filename_table, header = TRUE, sep = ",")
df <- df[df$EUR.Ancestry == 1,] # Only use samples with European ancestry
df <- df[df$Family.Type == "trio",] # Only use trios for this analysis
# Drop SV columns (not used in this analysis)
drops <- c("dnSV",
           "SV.LoF.Paternal.Transmitted","SV.LoF.Paternal.Untransmitted",
          "SV.LoF.Maternal.Transmitted","SV.LoF.Maternal.Untransmitted",
          "CRE.SV.Paternal.Transmitted","CRE.SV.Paternal.Untransmitted",
          "CRE.SV.Maternal.Transmitted","CRE.SV.Maternal.Untransmitted",
          "RVRS","PRS","GRS")
df <- df[ , !(names(df) %in% drops)]
df <- na.omit(df) # Remove rows with NA values

# Make cohort variables
df$REACH_cohort = as.numeric(df$Cohort == "REACH")
df$SPARK_cohort = as.numeric(df$Cohort == "SPARK")
df$SSC_cohort = as.numeric(df$Cohort == "SSC")

dfm <- df[df$Sex == 0,] # Dataframe containing only male samples
dff <- df[df$Sex == 1,] # Dataframe containing only female samples

# Grab PCs
# df[,grep("^PC([1-9]|10)$", names(df), value=TRUE)]
# Grab Sex
# df[,grep("^Sex$", names(df), value=TRUE)]
# Grab Phenotype
# df[,grep("^Phenotype$", names(df), value=TRUE)]

# Sample rows (without replacement by default?)
# df[sample(nrow(df), 1000), ]

In [ ]:
# Written by James Guevara

# The bootstrap is used to estimate a confidence interval for the R2 estimates.
# However, the bootstrap procedure seemingly requires a lot of CPUs and takes a lot of time
# given the amount of data in the table.
# So I ran the procedure on a compute cluster (Comet at UCSD) using the Slurm Workload Manager.
# But I've adapted the code here so that the reader can understand how we ran the
# bootstrap. 
getNagelkerkeR2_bootstrap <- function(formula, data, indices) {
  d <- data[indices,]
  fit_alt  <- glm(formula = formula, data = d, family = binomial)
  fit_null <- glm(formula = d$Phenotype~d$Sex+d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort, data=d, family=binomial)
  return(NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2)
}

getNagelkerkeR2 <- function(dataframe, alt_vars_in, null_vars_in, text) {
  d <- dataframe
  alt_variables <- alt_vars_in
  alt_formula <- reformulate(alt_variables, response = "Phenotype")
  null_variables <- null_vars_in
  null_formula <- reformulate(null_variables, response = "Phenotype")
  fit_alt  <- glm(formula = alt_formula , data = d, family = binomial)
  fit_null <- glm(formula = null_formula , data = d, family = binomial)
  nagelkerke_R2_val = NagelkerkeR2(fit_alt)$R2 - NagelkerkeR2(fit_null)$R2
  lr <- lrtest(fit_alt,fit_null) # Likelihood ratio test
  lr_p_val = lr["Pr(>Chisq)"][2,1] # p-value

  # cat(c(text,"\n"))
  # cat(c("R2:\t", nagelkerke_R2_val, "\t", "Likelihood_ratio_test_p-value:\t", lr_p_val, "\n"))

  # Put into dataframe? (R2, p-value)
  df_r2 = data.frame(text, nagelkerke_R2_val, lr_p_val)

  # Get the alt_vars_in c vector into a format appropriate for the bootstrap function 
  alt_vars_formula = paste(unlist(strsplit(alt_vars_in, split="d$", fixed=TRUE)), collapse=" ")
  alt_vars_formula = paste("Phenotype ~", alt_vars_formula, collapse= " ")
  # print(alt_vars_formula)

  # Using the bootstrap to get the confidence interval estimates (and standard error of R2)
  # r2_stats <- boot(data=d, statistic=getNagelkerkeR2_bootstrap, R=10000, formula=alt_vars_formula, parallel="multicore", ncpus=2)
  # ste = sd(r2_stats$t)
  # ci = boot.ci(r2_stats, type="bca") # Using the BCa interval (more information here: https://blogs.sas.com/content/iml/2017/07/12/bootstrap-bca-interval.html)
  # ci_lower = as.double(ci$bca[1,4])
  # ci_upper = as.double(ci$bca[1,5])
  # df_r2_tmp = data.frame(text, nagelkerke_R2_val, lr_p_val, ste, ci_lower, ci_upper)


  return(df_r2)
} 


cohort_vars <- c("d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort")
pc_vars <- c("d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10")
sex_var <- c("d$Sex") # Only use with combined models (it doesn't matter)
null_vars <- paste(cohort_vars, pc_vars, sex_var, sep = "+") 


# Full model
var <- c("d$dnMIS+d$dnLOF+d$inhLoF+d$pTDT.ASD.Dev+d$pTDT.EA.Dev+d$pTDT.SCZ.Dev")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_full_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "combined_both")
# Males
df_full_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "combined_male")
# Females
df_full_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "combined_female")


# PRS model
var <- c("d$pTDT.ASD.Dev+d$pTDT.EA.Dev+d$pTDT.SCZ.Dev")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_prs_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "prs_both")
# Males
df_prs_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "prs_male")
# Females
df_prs_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "prs_female")

# Rare model
var <- c("d$dnMIS+d$dnLOF+d$inhLoF")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_rare_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "rare_both")
# Males
df_rare_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "rare_male")
# Females
df_rare_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "rare_female")

# pTDT.ASD.Dev
var <- c("d$pTDT.ASD.Dev")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_ptdt_asd_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "ptdt.asd.dev_both")
# Males
df_ptdt_asd_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "ptdt.asd.dev_male")
# Females
df_ptdt_asd_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "ptdt.asd.dev_female")

# pTDT.EA.Dev
var <- c("d$pTDT.EA.Dev")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_ptdt_ea_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "ptdt.ea.dev_both")
# Males
df_ptdt_ea_r2_male =getNagelkerkeR2(dfm, alt_vars, null_vars, "ptdt.ea.dev_male")
# Females
df_ptdt_ea_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "ptdt.ea.dev_female")

# pTDT.SCZ.Dev
var <- c("d$pTDT.SCZ.Dev")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_ptdt_scz_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "ptdt.scz.dev_both")
# Males
df_ptdt_scz_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "ptdt.scz.dev_male")
# Females
df_ptdt_scz_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "ptdt.scz.dev_female")

# dnMIS
var <- c("d$dnMIS")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_dnmis_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "dnMIS_both")
# Males
df_dnmis_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "dnMIS_male")
# Females
df_dnmis_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "dnMIS_female")

# dnLOF
var <- c("d$dnLOF")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_dnlof_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "dnLOF_both")
# Males
df_dnlof_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "dnLOF_male")
# Females
df_dnlof_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "dnLOF_female")

# inhLoF
var <- c("d$inhLoF")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_inhlof_r2 = getNagelkerkeR2(df, alt_vars, null_vars, "inhLOF_both")
# Males
df_inhlof_r2_male = getNagelkerkeR2(dfm, alt_vars, null_vars, "inhLOF_male")
# Females
df_inhlof_r2_female = getNagelkerkeR2(dff, alt_vars, null_vars, "inhLOF_female")


# Combine all the dataframes
df_combined = rbind(df_full_r2, df_full_r2_male, df_full_r2_female,
                    df_prs_r2, df_prs_r2_male, df_prs_r2_female,
                    df_rare_r2, df_rare_r2_male, df_rare_r2_female,
                    df_ptdt_asd_r2, df_ptdt_asd_r2_male, df_ptdt_asd_r2_female,
                    df_ptdt_ea_r2, df_ptdt_ea_r2_male, df_ptdt_ea_r2_female,
                    df_ptdt_scz_r2, df_ptdt_scz_r2_male, df_ptdt_scz_r2_female,
                    df_dnmis_r2, df_dnmis_r2_male, df_dnmis_r2_female,
                    df_dnlof_r2, df_dnlof_r2_male, df_dnlof_r2_female,
                    df_inhlof_r2, df_inhlof_r2_male, df_inhlof_r2_female)

# Add a header to the combined dataframe
colnames(df_combined) <- c("R2_type", "R2", "Likelihood_Ratio_p_value")

# Write to file
write.csv(df_combined, file=r2_out, row.name=TRUE)

In [ ]:
# R2 by cohort
pc_vars <- c("d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10")
sex_var <- c("d$Sex") # Only use with combined models (it doesn't matter)
null_vars <- paste(pc_vars, sex_var, sep = "+") 

df_reach <- df[df$REACH_cohort == 1,]
df_spark <- df[df$SPARK_cohort == 1,]
df_ssc   <- df[df$SSC_cohort == 1,]

# pTDT.ASD.Dev
var <- c("d$pTDT.ASD.Dev")
alt_vars  <- paste(var, pc_vars, sex_var, sep = "+")
df_ptdt_asd_r2_reach = getNagelkerkeR2(df_reach, alt_vars, null_vars, "ptdt.asd.dev_REACH")
df_ptdt_asd_r2_spark = getNagelkerkeR2(df_spark, alt_vars, null_vars, "ptdt.asd.dev_SPARK")
df_ptdt_asd_r2_ssc   = getNagelkerkeR2(df_ssc, alt_vars, null_vars, "ptdt.asd.dev_SSC")

# Combined cohort (I already did this..)
cohort_vars <- c("d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort")
null_vars <- paste(cohort_vars, pc_vars, sex_var, sep = "+")
alt_vars  <- paste(var, cohort_vars, pc_vars, sex_var, sep = "+")
df_ptdt_asd_r2_combined   = getNagelkerkeR2(df_ssc, alt_vars, null_vars, "ptdt.asd.dev_Combined")

df_cohorts = rbind(df_ptdt_asd_r2_reach, df_ptdt_asd_r2_ssc, df_ptdt_asd_r2_spark, df_ptdt_asd_r2_combined)

# Add a header to the combined dataframe
colnames(df_cohorts) <- c("Model", "R2", "P-Value")

# Write to file
write.csv(df_cohorts, file = "SupplementaryTable5.PS_ASD_R2_by_cohort.csv", row.name = TRUE)

In [ ]:
# Responses from residuals
library(fmsb)
options(stringsAsFactors = FALSE);

d = df

# PRS scores residualized:
ptdt_prs_asd_resid <- resid(glm(d$pTDT.ASD.Dev~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d, na.action="na.exclude"))
ptdt_prs_ea_resid  <- resid(glm(d$pTDT.EA.Dev  ~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d, na.action="na.exclude"))
ptdt_prs_scz_resid <- resid(glm(d$pTDT.SCZ.Dev~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d, na.action="na.exclude"))


# Fit combined prs model
fit_alt_prs <- glm(d$Phenotype~ptdt_prs_asd_resid+ptdt_prs_ea_resid+ptdt_prs_scz_resid, data=df, family=binomial)
response_prs = predict(fit_alt_prs, type = "response")

# Rare scores residualized
snv_dn_lof_resid <- resid(glm(d$dnLOF ~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d,na.action="na.exclude"))
snv_dn_mis_resid <- resid(glm(d$dnMIS ~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d,na.action="na.exclude") )
snv_lof_t_resid  <- resid(glm(d$inhLoF~d$PC1+d$PC2+d$PC3+d$PC4+d$PC5+d$PC6+d$PC7+d$PC8+d$PC9+d$PC10+d$REACH_cohort+d$SPARK_cohort+d$SSC_cohort+d$Sex, data=d,na.action="na.exclude") )


# Fit combined rare model
fit_alt_rare <- glm(d$Phenotype~snv_dn_lof_resid+snv_dn_mis_resid+snv_lof_t_resid, data=d, family=binomial)
response_rare = predict(fit_alt_rare, type = "response")

# Fit rare + prs combined model
fit_alt_combined <- glm(d$Phenotype~ptdt_prs_asd_resid+ptdt_prs_ea_resid+ptdt_prs_scz_resid+snv_dn_lof_resid+snv_dn_mis_resid+snv_lof_t_resid, data=d, family=binomial)
response_combined = predict(fit_alt_combined, type = "response")

# Fit individual features
# ptdt_prs_asd
fit_alt_ptdt_prs_asd <- glm(d$Phenotype~ptdt_prs_asd_resid, data=d, family=binomial)
response_ptdt_prs_asd = predict(fit_alt_ptdt_prs_asd, type = "response")
# ptdt_prs_ea
fit_alt_ptdt_prs_ea <- glm(d$Phenotype~ptdt_prs_ea_resid, data=d, family=binomial)
response_ptdt_prs_ea = predict(fit_alt_ptdt_prs_ea, type = "response")
# ptdt_prs_scz
fit_alt_ptdt_prs_scz <- glm(d$Phenotype~ptdt_prs_scz_resid, data=d, family=binomial)
response_ptdt_prs_scz = predict(fit_alt_ptdt_prs_scz, type = "response")

# snv_dn_lof
fit_alt_snv_dn_lof <- glm(d$Phenotype~snv_dn_lof_resid, data=d, family=binomial)
response_snv_dn_lof = predict(fit_alt_snv_dn_lof, type = "response")
# snv_dn_mis
fit_alt_snv_dn_mis <- glm(d$Phenotype~snv_dn_mis_resid, data=d, family=binomial)
response_snv_dn_mis = predict(fit_alt_snv_dn_mis, type = "response")
# snv_lof_t
fit_alt_snv_lof_t <- glm(d$Phenotype~snv_lof_t_resid, data=d, family=binomial)
response_snv_lof_t = predict(fit_alt_snv_lof_t, type = "response")

# Responses (not z-scored yet)
# d$all_combined_score = response_combined
# d$prs_combined_score = response_prs
# d$rare_combined_score = response_rare
# d$response_ptdt_prs_asd = response_ptdt_prs_asd
# d$response_ptdt_prs_asd = response_ptdt_prs_ea
# d$response_ptdt_prs_scz = response_ptdt_prs_scz
# d$response_snv_dn_lof = response_snv_dn_lof
# d$response_snv_dn_mis = response_snv_dn_mis
# d$response_snv_lof_t = response_snv_lof_t

# Get GRS, PRS, RVRS columns (normalize all_combined_score, prs_combined_score, and rare_combined_score)
d$GRS = (response_combined - mean(response_combined))/sd(response_combined)
d$PRS = (response_prs - mean(response_prs))/sd(response_prs)
d$RVRS = (response_rare - mean(response_rare))/sd(response_rare)

write.csv(d, file = responses_out, row.name=TRUE)

<a name="negative_correlations_rare_common"></a>
## Negative correlations of rare and common (Fig. 3)
-------------------------------

In [ ]:
# Written by Danny Antaki, edited by James Guevara
options(stringsAsFactors = FALSE);
# df = read.csv("SummaryGeneticData_REACH_SSC_SPARK.csv",sep=",")
df = read.csv(responses_out,sep=",")
df$dnAll = df$dnLOF+df$dnMIS
df$cohort_covar <- factor(df$Cohort)
df <- df[df$EUR.Ancestry==1,]
vars<-c(
    "dnMIS" ,"dnLOF","dnAll",
    "inhLoF", "RVRS" , 
    "pTDT.ASD.Dev" , "pTDT.SCZ.Dev" ,"pTDT.EA.Dev",
    "PRS"
);
cols <- c("cohort_covar","Sex","Phenotype","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10",vars)
df<-na.omit(df[cols])
print(nrow(df))
###########################################
fit_model<- function(df,x,y,add_sex=FALSE){
    Y<- df[[x]]
    X<- df[[y]]
    PC1<-df$PC1
    PC2<-df$PC2
    PC3<-df$PC3
    PC4<-df$PC4
    PC5<-df$PC5
    PC6<-df$PC6
    PC7<-df$PC7
    PC8<-df$PC8
    PC9<-df$PC9
    PC10<-df$PC10
    phen<-df$Phenotype
    cohort_covar<-df$cohort_covar
    sex<-df$Sex
    sex_pv<-NA    
    if (add_sex==TRUE){
        mod<-lm(Y~X+sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        nul<-lm(Y~sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        #sex interaction
        interaction_mod<-lm(Y~X*sex+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        sex_pv<-signif(summary(interaction_mod)$coefficients[17,4],digits=2)
    } else {
        mod<-lm(Y~X+phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
        nul<-lm(Y~phen+cohort_covar+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
    }
    # slope of fit of X to Y
    est<-signif(summary(mod)$coefficients[2,1],digits=2)
    xpv<-signif(summary(mod)$coefficients[2,4],digits=2)
    #r2 is the difference of the mod with X and without X
    r2<-signif(summary(mod)$r.squared-summary(nul)$r.squared,digits=2)
    pearsons_cor<-signif(cor(Y,X, use="pairwise.complete.obs"),digits=2)    
    if (add_sex==TRUE){
        result = paste(est,xpv,r2,pearsons_cor,sex_pv,sep="\t")
    } else {
        result = paste(est,xpv,r2,pearsons_cor,sep="\t")
    }
    #sprintf("%f\t%f\t%f\t%f\t%f",est,xpv,r2,pearson_cor,sex_pv)
    return(result)
}
###########################################
header = paste("y","x",
               "male_est","male_pval","male_r2","male_cor",
               "female_est","female_pval","female_r2","female_cor",
               "combined_est","combined_pval","combined_r2","combined_cor",
               "sex_interaction_pval",
               sep="\t"
              );
output=vector(length = 1+(length(vars)^2)-(length(vars)))
output[1]<-header
i=2
for (x in vars)
{
  for(y in vars)
  {
    if (x==y){ next; } 
    com_results <- fit_model(df,x,y,add_sex=TRUE)
    mal_results <- fit_model(df[df$Sex==0,],x,y,add_sex=FALSE)
    fem_results <- fit_model(df[df$Sex==1,],x,y,add_sex=FALSE)
    # Y - X - mal
    output[i] <- paste(x,y,mal_results,fem_results,com_results,sep="\t")
    i=i+1;
  }
}
out = asd_genetic_correlations_by_sex_out
ofh<-file(out)
writeLines(output,ofh)
close(ofh)
print(">>>DONE<<<")

[1] 8301
[1] ">>>DONE<<<"


<a name="gene_phenotype_correlations"></a>
## Gene–phenotype correlations (Fig. 5)

In [ ]:
#Author: Jonathan Sebat http://sebatlab.org March 19, 2021

#The following phenotypes were available for only one cohort in either cases or controls: srs.case,vabs.case,srs.cont,scq.cont,vabs.cont,bapq.parents,srs.parents)
#for these, age, sex and PCs were included as covariates, but not cohort (a 1st set of functions is written for this test)

#SCQ, RBS and DCDQ were available on cases in the multiple cohorts (SSC and SPARK). Parental age and educational attainment were also available on multiple cohorts
#for these, "cohort" was included as a covariate (a 2nd set of functions are written for this test)



#importing the input data table that contains rare variant counts (dnLoF, dnMis, inhLoF), polygenic scores(PSasd, PSsz, PSea) and clinical phenotypes (srs, scq, rbs, vabs, dcdq, parental age, parental educational attainment, parental bapq)

# master.gen.phen.table<-read.csv("C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\master_phen_4.csv", header = TRUE)
# master.gen.phen.table<-read.csv(filename_phen, header = TRUE)
master.gen.phen.table<-read.csv("/content/master_phen_4.SBayesR.20210915.csv", header = TRUE)
# Turn cohort into a factor variable
master.gen.phen.table$cohort = factor(master.gen.phen.table$cohort)

################################################################################
##################################################################################
####### First we create the main functions to run linear regression   ###########
#######        on y = phenoype, x = genetic factor                    #############
#####################################################################################
#####################################################################################

###############    1st set of scripts ###########################################################3
#For phenotypes available on only a single cohort, "cohort" is NOT included as a covariate
#linear regression controls for age, sex and PCs
#and tests for a gene-by-sex interactions
#the functions for these are as follows
#gen.phen.1 a function that runs a linear regression and tests for gene-by-sex interaction. These are the primary results described in Fig. 5
#gen.phen.2 To characterize gene-by sex effects that are significant, gene.phen.2 runs the regression on males and females separately without sex as a covariate
#gene.phen.3 runs linear regression and gene-by-sex interaction on PARENTS, then to investigate gene by sex interactions, runs regression on mothers and fathers individually


#gen.phen.1 a function that runs a linear regression on both sexes and outputs the result

gen.phen.1<-function(master.temp,phen.col,age.col)
{
#define the sexes
male<-master.temp$sex=="Male"
female<-master.temp$sex=="Female"
  
#Test combined sample
sex<-master.temp$sex
cohort<-master.temp$cohort
PC1<-master.temp$PC1
PC2<-master.temp$PC2
PC3<-master.temp$PC3
PC4<-master.temp$PC4
PC5<-master.temp$PC5
PC6<-master.temp$PC6
PC7<-master.temp$PC7
PC8<-master.temp$PC8
PC9<-master.temp$PC9
PC10<-master.temp$PC10

#retreive phen data
y<-master.temp[,phen.col]
age<-master.temp[,age.col]


#retreive gen data
x.dnlof<-master.temp[,"dnLoF"]
x.dn<-master.temp[,"dnMIS"]
x.inh<-master.temp[,"inhLoF"]
x.ps.asd<-master.temp[,"PS.ASD.Z"]
x.ps.sz<-master.temp[,"PS.SZ.Z"]
x.ps.ea<-master.temp[,"PS.EA.Z"]

result.dnlof<-lr.1(y,x.dnlof,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.1(y,x.dn,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.1(y,x.inh,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.1(y,x.ps.asd,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.1(y,x.ps.sz,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.1(y,x.ps.ea,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
group<-rep("combined",length(variant.type))
comb.result<-cbind(group,variant.type,comb.result)

#################Test males

PC1<-master.temp$PC1[male]
PC2<-master.temp$PC2[male]
PC3<-master.temp$PC3[male]
PC4<-master.temp$PC4[male]
PC5<-master.temp$PC5[male]
PC6<-master.temp$PC6[male]
PC7<-master.temp$PC7[male]
PC8<-master.temp$PC8[male]
PC9<-master.temp$PC9[male]
PC10<-master.temp$PC10[male]

#retreive phen data
y<-master.temp[male,phen.col]
age<-master.temp[male,age.col]


#retreive gen data
x.dnlof<-master.temp[male,"dnLoF"]
x.dn<-master.temp[male,"dnMIS"]
x.inh<-master.temp[male,"inhLoF"]
x.ps.asd<-master.temp[male,"PS.ASD.Z"]
x.ps.sz<-master.temp[male,"PS.SZ.Z"]
x.ps.ea<-master.temp[male,"PS.EA.Z"]


result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

male.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
group<-rep("male",length(variant.type))
male.result<-cbind(group,variant.type,male.result)

#####Test females ######################

PC1<-master.temp$PC1[female]
PC2<-master.temp$PC2[female]
PC3<-master.temp$PC3[female]
PC4<-master.temp$PC4[female]
PC5<-master.temp$PC5[female]
PC6<-master.temp$PC6[female]
PC7<-master.temp$PC7[female]
PC8<-master.temp$PC8[female]
PC9<-master.temp$PC9[female]
PC10<-master.temp$PC10[female]

#retreive phen data
y<-master.temp[female,phen.col]
age<-master.temp[female,age.col]


#retreive gen data
x.dnlof<-master.temp[female,"dnLoF"]
x.dn<-master.temp[female,"dnMIS"]
x.inh<-master.temp[female,"inhLoF"]
x.ps.asd<-master.temp[female,"PS.ASD.Z"]
x.ps.sz<-master.temp[female,"PS.SZ.Z"]
x.ps.ea<-master.temp[female,"PS.EA.Z"]


result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)

female.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
group<-rep("female",length(variant.type))
female.result<-cbind(group,variant.type,female.result)

final.result<-rbind(female.result,male.result,comb.result)
final.result
}


#linear regression script 1 that also tests sex interaction
lr.1<-function(y,x,sex,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate

  
  #test sex interaction
  int.model<-lm(y~x*sex+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[15,4],digits=2)
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

###############################################################################33
#Then we have a function that doesn't test the sex interaction
##############################################################################

gen.phen.2<-function(master.temp,phen.col,age.col)
{
  #retrieve covariates
  PC1<-master.temp$PC1
  PC2<-master.temp$PC2
  PC3<-master.temp$PC3
  PC4<-master.temp$PC4
  PC5<-master.temp$PC5
  PC6<-master.temp$PC6
  PC7<-master.temp$PC7
  PC8<-master.temp$PC8
  PC9<-master.temp$PC9
  PC10<-master.temp$PC10
  
  #retreive phen data
  y<-master.temp[,phen.col]
  age<-master.temp[,age.col]
  
  #retreive gen data
  x.dnlof<-master.temp[,"dnLoF"]
  x.dn<-master.temp[,"dnMIS"]
  x.inh<-master.temp[,"inhLoF"]
  x.ps.asd<-master.temp[,"PS.ASD.Z"]
  x.ps.sz<-master.temp[,"PS.SZ.Z"]
  x.ps.ea<-master.temp[,"PS.EA.Z"]

  
  result.dnlof<-lr.2(y,x.dnlof,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.2(y,x.dn,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.2(y,x.inh,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.2(y,x.ps.asd,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.2(y,x.ps.sz,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.2(y,x.ps.ea,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  final.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  final.result<-cbind(variant.type,final.result)
  final.result
}



#linear regression script 2 for testing each sex individually
lr.2<-function(y,x,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~age+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate
  
  #test sex interaction
  sex.pval<-NA
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}
##################################################################
# Then we have a regression for testing the effects of parents
#################################################################

#Ftests phenotypes in parents and tests sex interaction
gen.phen.3<-function(master.temp,phen.col.f,phen.col.m)
{
  #retrieve father data
  ###########################
  sexfather<-rep(1,nrow(master.temp))
  PC1f<-master.temp$PC1_PAT
  PC2f<-master.temp$PC2_PAT
  PC3f<-master.temp$PC3_PAT
  PC4f<-master.temp$PC4_PAT
  PC5f<-master.temp$PC5_PAT
  PC6f<-master.temp$PC6_PAT
  PC7f<-master.temp$PC7_PAT
  PC8f<-master.temp$PC8_PAT
  PC9f<-master.temp$PC9_PAT
  PC10f<-master.temp$PC10_PAT
  
  #retreive phen data
  yf<-master.temp[,phen.col.f]
  
  
  #retreive gen data
  f.dnlof<-master.temp[,"dnLoF"]
  f.dn<-master.temp[,"dnMIS"]
  f.inh<-master.temp$InhLoF.Paternal.Transmitted + master.temp$InhLoF.Paternal.Untransmitted
  f.ps.asd<-master.temp$Paternal.PS.ASD.Z
  f.ps.sz<-master.temp$Paternal.PS.SZ.Z
  f.ps.ea<-master.temp$Paternal.PS.EA.Z
  
  #retrieve mother data
  ###########################
  
  sexmother<-rep(0,nrow(master.temp))
  PC1m<-master.temp$PC1_MAT
  PC2m<-master.temp$PC2_MAT
  PC3m<-master.temp$PC3_MAT
  PC4m<-master.temp$PC4_MAT
  PC5m<-master.temp$PC5_MAT
  PC6m<-master.temp$PC6_MAT
  PC7m<-master.temp$PC7_MAT
  PC8m<-master.temp$PC8_MAT
  PC9m<-master.temp$PC9_MAT
  PC10m<-master.temp$PC10_MAT
  
  #retreive phen data
  ym<-master.temp[,phen.col.m]
  
  #retreive gen data
  m.dnlof<-master.temp[,"dnLoF"]
  m.dn<-master.temp[,"dnMIS"]
  m.inh<-master.temp$InhLoF.Maternal.Transmitted + master.temp$InhLoF.Maternal.Untransmitted
  m.ps.asd<-master.temp$Maternal.PS.ASD.Z
  m.ps.sz<-master.temp$Maternal.PS.SZ.Z
  m.ps.ea<-master.temp$Maternal.PS.EA.Z
  
  #merge father and mother variables
  sex<-c(sexfather,sexmother)
  PC1<-c(PC1f,PC1m)
  PC2<-c(PC2f,PC2m)
  PC3<-c(PC3f,PC3m)
  PC4<-c(PC4f,PC4m)
  PC5<-c(PC5f,PC5m)
  PC6<-c(PC6f,PC6m)
  PC7<-c(PC7f,PC7m)
  PC8<-c(PC8f,PC8m)
  PC9<-c(PC9f,PC9m)
  PC10<-c(PC10f,PC10m)
  x.dnlof<-c(f.dnlof,m.dnlof)
  x.dn<-c(f.dn,m.dn)
  x.inh<-c(f.inh,m.inh)
  x.ps.asd<-c(f.ps.asd,m.ps.asd)
  x.ps.sz<-c(f.ps.sz,m.ps.sz)
  x.ps.ea<-c(f.ps.ea,m.ps.ea)
  y<-c(yf,ym)
  
  result.dnlof<-lr.3(y,x.dnlof,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.3(y,x.dn,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.3(y, x.inh, sex, PC1, PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.3(y,x.ps.asd,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.3(y,x.ps.sz,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.3(y,x.ps.ea,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  #combined result
  comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("parents",length=length(variant.type))
  comb.result<-cbind(group,variant.type,comb.result)
  
  
  #father result
  result.dnlof<-lr.4(yf,f.dnlof,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.dn<-lr.4(yf,f.dn,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.inh<-lr.4(yf, f.inh, PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.asd<-lr.4(yf,f.ps.asd,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.sz<-lr.4(yf,f.ps.sz,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.ea<-lr.4(yf,f.ps.ea,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  
  f.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("father",length=length(variant.type))
  f.result<-cbind(group,variant.type,f.result)
  
  #mother result
  result.dnlof<-lr.4(ym,m.dnlof,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.dn<-lr.4(ym,m.dn,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.inh<-lr.4(ym, m.inh, PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.asd<-lr.4(ym,m.ps.asd,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.sz<-lr.4(ym,m.ps.sz,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.ea<-lr.4(ym,m.ps.ea,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  
  m.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("mother",length=length(variant.type))
  m.result<-cbind(group,variant.type,m.result)
  
  #output final result
  final.result<-rbind(m.result,f.result,comb.result)
  final.result
}


#linear regression script 3 tests phenotypes in parents and tests sex interaction
lr.3<-function(y,x,sex,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate
  
  #test sex interaction
  int.model<-lm(y~x*sex+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[14,4],digits=2)
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

#linear regression script 4 tests phenotypes in mother or father without sex variable
lr.4<-function(y,x,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cc<-cor(x,y)

  
  #test sex interaction
  sex.pval<-NA
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

#############################     2nd set of scripts with cohort included as a covaraite    ##################
#in addition to age, sex and PCs
#and tests for a gene-by-sex interactions
#the functions for these are as follows
#gen.phen.1c a function that runs a linear regression and tests for gene-by-sex interaction. These are the primary results described in Fig. 5
#gen.phen.2c To characterize gene-by sex effects that are significant, gene.phen.2 runs the regression on males and females separately without sex as a covariate
#gene.phen.3c runs linear regression and gene-by-sex interaction on PARENTS, then to investigate gene by sex interactions, runs regression on mothers and fathers individually


#gen.phen.1c a function that runs a linear regression on both sexes and outputs the result

gen.phen.1c<-function(master.temp,phen.col,age.col)
{
  #define the sexes
  male<-master.temp$sex=="Male"
  female<-master.temp$sex=="Female"
  
  #Test combined sample
  sex<-master.temp$sex
  cohort<-master.temp$cohort
  PC1<-master.temp$PC1
  PC2<-master.temp$PC2
  PC3<-master.temp$PC3
  PC4<-master.temp$PC4
  PC5<-master.temp$PC5
  PC6<-master.temp$PC6
  PC7<-master.temp$PC7
  PC8<-master.temp$PC8
  PC9<-master.temp$PC9
  PC10<-master.temp$PC10
  
  #retreive phen data
  y<-master.temp[,phen.col]
  age<-master.temp[,age.col]
  
  
  #retreive gen data
  x.dnlof<-master.temp[,"dnLoF"]
  x.dn<-master.temp[,"dnMIS"]
  x.inh<-master.temp[,"inhLoF"]
  x.ps.asd<-master.temp[,"PS.ASD.Z"]
  x.ps.sz<-master.temp[,"PS.SZ.Z"]
  x.ps.ea<-master.temp[,"PS.EA.Z"]
  
  result.dnlof<-lr.1c(y,x.dnlof,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.1c(y,x.dn,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.1c(y,x.inh,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.1c(y,x.ps.asd,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.1c(y,x.ps.sz,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.1c(y,x.ps.ea,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("combined",length(variant.type))
  comb.result<-cbind(group,variant.type,comb.result)
  
  #################Test males
  
  cohort<-master.temp$cohort[male]
  PC1<-master.temp$PC1[male]
  PC2<-master.temp$PC2[male]
  PC3<-master.temp$PC3[male]
  PC4<-master.temp$PC4[male]
  PC5<-master.temp$PC5[male]
  PC6<-master.temp$PC6[male]
  PC7<-master.temp$PC7[male]
  PC8<-master.temp$PC8[male]
  PC9<-master.temp$PC9[male]
  PC10<-master.temp$PC10[male]
  
  #retreive phen data
  y<-master.temp[male,phen.col]
  age<-master.temp[male,age.col]
  
  
  #retreive gen data
  x.dnlof<-master.temp[male,"dnLoF"]
  x.dn<-master.temp[male,"dnMIS"]
  x.inh<-master.temp[male,"inhLoF"]
  x.ps.asd<-master.temp[male,"PS.ASD.Z"]
  x.ps.sz<-master.temp[male,"PS.SZ.Z"]
  x.ps.ea<-master.temp[male,"PS.EA.Z"]
  
  
  result.dnlof<-lr.2c(y,x.dnlof,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.2c(y,x.dn,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.2c(y,x.inh,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.2c(y,x.ps.asd,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.2c(y,x.ps.sz,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.2c(y,x.ps.ea,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  male.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("male",length(variant.type))
  male.result<-cbind(group,variant.type,male.result)
  
  #####Test females ######################
  cohort<-master.temp$cohort[female]
  PC1<-master.temp$PC1[female]
  PC2<-master.temp$PC2[female]
  PC3<-master.temp$PC3[female]
  PC4<-master.temp$PC4[female]
  PC5<-master.temp$PC5[female]
  PC6<-master.temp$PC6[female]
  PC7<-master.temp$PC7[female]
  PC8<-master.temp$PC8[female]
  PC9<-master.temp$PC9[female]
  PC10<-master.temp$PC10[female]
  
  #retreive phen data
  y<-master.temp[female,phen.col]
  age<-master.temp[female,age.col]
  
  
  #retreive gen data
  x.dnlof<-master.temp[female,"dnLoF"]
  x.dn<-master.temp[female,"dnMIS"]
  x.inh<-master.temp[female,"inhLoF"]
  x.ps.asd<-master.temp[female,"PS.ASD.Z"]
  x.ps.sz<-master.temp[female,"PS.SZ.Z"]
  x.ps.ea<-master.temp[female,"PS.EA.Z"]
  
  
  result.dnlof<-lr.2c(y,x.dnlof,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.2c(y,x.dn,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.2c(y,x.inh,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.2c(y,x.ps.asd,cohort,age,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.2c(y,x.ps.sz,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.2c(y,x.ps.ea,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  female.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("female",length(variant.type))
  female.result<-cbind(group,variant.type,female.result)
  
  final.result<-rbind(female.result,male.result,comb.result)
  final.result
}


#linear regression script 1 that also tests sex interaction
lr.1c<-function(y,x,sex,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+age+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+age+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate
  
  
  #test sex interaction
  int.model<-lm(y~x*sex+age+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[15,4],digits=2)
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

###############################################################################33
#Then we have a function that doesn't test the sex interaction
##############################################################################

gen.phen.2c<-function(master.temp,phen.col,age.col)
{
  #retrieve covariates
  PC1<-master.temp$PC1
  PC2<-master.temp$PC2
  PC3<-master.temp$PC3
  PC4<-master.temp$PC4
  PC5<-master.temp$PC5
  PC6<-master.temp$PC6
  PC7<-master.temp$PC7
  PC8<-master.temp$PC8
  PC9<-master.temp$PC9
  PC10<-master.temp$PC10
  
  #retreive phen data
  y<-master.temp[,phen.col]
  age<-master.temp[,age.col]
  
  #retreive gen data
  x.dnlof<-master.temp[,"dnLoF"]
  x.dn<-master.temp[,"dnMIS"]
  x.inh<-master.temp[,"inhLoF"]
  x.ps.asd<-master.temp[,"PS.ASD.Z"]
  x.ps.sz<-master.temp[,"PS.SZ.Z"]
  x.ps.ea<-master.temp[,"PS.EA.Z"]
  
  
  result.dnlof<-lr.2c(y,x.dnlof,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.2c(y,x.dn,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.2c(y,x.inh,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.2c(y,x.ps.asd,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.2c(y,x.ps.sz,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.2c(y,x.ps.ea,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  final.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  final.result<-cbind(variant.type,final.result)
  final.result
}



#linear regression script 2 for testing each sex individually
lr.2c<-function(y,x,age,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+age+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~age+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate
  
  #test sex interaction
  sex.pval<-NA
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}
##################################################################
# Then we have a regression for testing the effects of parents
#################################################################

#Ftests phenotypes in parents and tests sex interaction
gen.phen.3c<-function(master.temp,phen.col.f,phen.col.m)
{
  #retrieve father data
  ###########################
  sexfather<-rep(1,nrow(master.temp))
  cohort<-master.temp$cohort
  PC1f<-master.temp$PC1_PAT
  PC2f<-master.temp$PC2_PAT
  PC3f<-master.temp$PC3_PAT
  PC4f<-master.temp$PC4_PAT
  PC5f<-master.temp$PC5_PAT
  PC6f<-master.temp$PC6_PAT
  PC7f<-master.temp$PC7_PAT
  PC8f<-master.temp$PC8_PAT
  PC9f<-master.temp$PC9_PAT
  PC10f<-master.temp$PC10_PAT
  
  #retreive phen data
  yf<-master.temp[,phen.col.f]
  
  
  #retreive gen data
  f.dnlof<-master.temp[,"dnLoF"]
  f.dn<-master.temp[,"dnMIS"]
  f.inh<-master.temp$InhLoF.Paternal.Transmitted + master.temp$InhLoF.Paternal.Untransmitted
  f.ps.asd<-master.temp$Paternal.PS.ASD.Z
  f.ps.sz<-master.temp$Paternal.PS.SZ.Z
  f.ps.ea<-master.temp$Paternal.PS.EA.Z
  
  #retrieve mother data
  ###########################
  
  sexmother<-rep(0,nrow(master.temp))
  PC1m<-master.temp$PC1_MAT
  PC2m<-master.temp$PC2_MAT
  PC3m<-master.temp$PC3_MAT
  PC4m<-master.temp$PC4_MAT
  PC5m<-master.temp$PC5_MAT
  PC6m<-master.temp$PC6_MAT
  PC7m<-master.temp$PC7_MAT
  PC8m<-master.temp$PC8_MAT
  PC9m<-master.temp$PC9_MAT
  PC10m<-master.temp$PC10_MAT
  
  #retreive phen data
  ym<-master.temp[,phen.col.m]
  
  #retreive gen data
  m.dnlof<-master.temp[,"dnLoF"]
  m.dn<-master.temp[,"dnMIS"]
  m.inh<-master.temp$InhLoF.Maternal.Transmitted + master.temp$InhLoF.Maternal.Untransmitted
  m.ps.asd<-master.temp$Maternal.PS.ASD.Z
  m.ps.sz<-master.temp$Maternal.PS.SZ.Z
  m.ps.ea<-master.temp$Maternal.PS.EA.Z
  
  #merge father and mother variables
  sex<-c(sexfather,sexmother)
  comb.cohort<-c(cohort,cohort)
  PC1<-c(PC1f,PC1m)
  PC2<-c(PC2f,PC2m)
  PC3<-c(PC3f,PC3m)
  PC4<-c(PC4f,PC4m)
  PC5<-c(PC5f,PC5m)
  PC6<-c(PC6f,PC6m)
  PC7<-c(PC7f,PC7m)
  PC8<-c(PC8f,PC8m)
  PC9<-c(PC9f,PC9m)
  PC10<-c(PC10f,PC10m)
  x.dnlof<-c(f.dnlof,m.dnlof)
  x.dn<-c(f.dn,m.dn)
  x.inh<-c(f.inh,m.inh)
  x.ps.asd<-c(f.ps.asd,m.ps.asd)
  x.ps.sz<-c(f.ps.sz,m.ps.sz)
  x.ps.ea<-c(f.ps.ea,m.ps.ea)
  y<-c(yf,ym)
  
  result.dnlof<-lr.3c(y,x.dnlof,sex,comb.cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.dn<-lr.3c(y,x.dn,sex,comb.cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.inh<-lr.3c(y, x.inh, sex,comb.cohort,PC1, PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.asd<-lr.3c(y,x.ps.asd,sex,comb.cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.sz<-lr.3c(y,x.ps.sz,sex,comb.cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  result.ps.ea<-lr.3c(y,x.ps.ea,sex,comb.cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
  
  #combined result
  comb.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("parents",length=length(variant.type))
  comb.result<-cbind(group,variant.type,comb.result)
  
  
  #father result
  result.dnlof<-lr.4c(yf,f.dnlof,cohort,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.dn<-lr.4c(yf,f.dn,cohort,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.inh<-lr.4c(yf, f.inh,cohort, PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.asd<-lr.4c(yf,f.ps.asd,cohort,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.sz<-lr.4c(yf,f.ps.sz,cohort,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  result.ps.ea<-lr.4c(yf,f.ps.ea,cohort,PC1f,PC2f,PC3f,PC4f,PC5f,PC6f,PC7f,PC8f,PC9f,PC10f)
  
  f.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("father",length=length(variant.type))
  f.result<-cbind(group,variant.type,f.result)
  
  #mother result
  result.dnlof<-lr.4c(ym,m.dnlof,cohort,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.dn<-lr.4c(ym,m.dn,cohort,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.inh<-lr.4c(ym, m.inh,cohort, PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.asd<-lr.4c(ym,m.ps.asd,cohort,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.sz<-lr.4c(ym,m.ps.sz,cohort,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  result.ps.ea<-lr.4c(ym,m.ps.ea,cohort,PC1m,PC2m,PC3m,PC4m,PC5m,PC6m,PC7m,PC8m,PC9m,PC10m)
  
  m.result<-rbind(result.dnlof,result.dn,result.inh,result.ps.asd,result.ps.sz,result.ps.ea)
  variant.type<-c("dnlof","dnmis","inhLoF", "ps.asd","ps.sz","ps.ea")
  group<-rep("mother",length=length(variant.type))
  m.result<-cbind(group,variant.type,m.result)
  
  #output final result
  final.result<-rbind(m.result,f.result,comb.result)
  final.result
}


#linear regression script 3 tests phenotypes in parents and tests sex interaction
lr.3c<-function(y,x,sex,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+sex+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~sex+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cor<-cor.test(x,y)
  pearson.cc<-pearson.cor$estimate
  
  #test sex interaction
  int.model<-lm(y~x*sex+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  sex.pval<-signif(summary(int.model)$coefficients[15,4],digits=2)
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

#linear regression script 4 tests phenotypes in mother or father without sex variable
lr.4c<-function(y,x,cohort,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10)
{
  model<-lm(y~x+cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  n.model<-lm(y~cohort+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10)
  est<-signif(summary(model)$coefficients[2,1],digits=2)
  est.stderr<-signif(summary(model)$coefficients[2,2],digits=2)
  x.pval<-signif(summary(model)$coefficients[2,4],digits=2)
  r.sq<-signif(summary(model)$r.squared-summary(n.model)$r.squared,digits=2)
  pearson.cc<-cor(x,y)
  
  
  #test sex interaction
  sex.pval<-NA
  
  result<-cbind(r.sq,est,est.stderr,pearson.cc,x.pval,sex.pval)
  result
}

################################################################################################################
#################   Then the analysis of each gene-phenotype correlation is run as follows    ##################
###############################################################################################################


#identify the subset of case, controls, and european ancesty
cont<-master.gen.phen.table$phen=="CON"
case<-master.gen.phen.table$phen=="ASD"
eur<-master.gen.phen.table$is_eur==1


#The analysis will be restricted to subjects of confirmed european ancestry
# master.temp.case.eur<-master.gen.phen.table[case & eur,]
# master.temp.cont.eur<-master.gen.phen.table[cont & eur,]
master.temp.case.eur<-master.gen.phen.table[case ,]
master.temp.cont.eur<-master.gen.phen.table[cont ,]


####################################################
# Test each phenotype in offspring
#####################################################


#VABS

phen.col<-"vabs.child.z"
age.col<-"vabs.child.age"

vine.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
vine.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)
# vine.case.eur<-gen.phen.1c(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
# vine.cont.eur<-gen.phen.1c(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)


phenotype<-rep("VABS",nrow(vine.case.eur))
vine.case.eur<-cbind(phenotype,vine.case.eur)

phenotype<-rep("VABS",nrow(vine.cont.eur))
vine.cont.eur<-cbind(phenotype,vine.cont.eur)

#SRS
phen.col<-"srs.child.z"
age.col<-"srs.child.age"

srs.case.eur<-gen.phen.1(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
srs.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)
# srs.case.eur<-gen.phen.1c(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
# srs.cont.eur<-gen.phen.1c(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)


phenotype<-rep("SRS",nrow(srs.case.eur))
srs.case.eur<-cbind(phenotype,srs.case.eur)

phenotype<-rep("SRS",nrow(srs.cont.eur))
srs.cont.eur<-cbind(phenotype,srs.cont.eur)


#SCQ
phen.col<-"scq.child.z"
age.col<-"scq.child.age"

scq.case.eur<-gen.phen.1c(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)
scq.cont.eur<-gen.phen.1(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)
# scq.cont.eur<-gen.phen.1c(master.temp = master.temp.cont.eur, phen.col=phen.col, age.col=age.col)


phenotype<-rep("SCQ",nrow(scq.case.eur))
scq.case.eur<-cbind(phenotype,scq.case.eur)
phenotype<-rep("SCQ",nrow(scq.cont.eur))
scq.cont.eur<-cbind(phenotype,scq.cont.eur)


#DCDQ
phen.col<-"dcdq.child.z"
age.col<-"dcdq.child.age"

dcdq.case.eur<-gen.phen.1c(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("DCDQ",nrow(dcdq.case.eur))
dcdq.case.eur<-cbind(phenotype,dcdq.case.eur)


#rbs
phen.col<-"rbs.child.z"
age.col<-"scq.child.age"
# age.col<-"rbs.child.age"


rbs.case.eur<-gen.phen.1c(master.temp = master.temp.case.eur, phen.col=phen.col, age.col=age.col)

phenotype<-rep("RBS",nrow(rbs.case.eur))
rbs.case.eur<-cbind(phenotype,rbs.case.eur)



############################################
# Testing Behavioral Phenotypes in Parents
############################################

cont<-master.gen.phen.table$phen=="CON"
case<-master.gen.phen.table$phen=="ASD"
eur<-master.gen.phen.table$is_eur==1


#Remove the sibling control trios (so the same parents are not tested twice)
# master.temp.case.eur<-master.gen.phen.table[case & eur,]
master.temp.case.eur<-master.gen.phen.table[case,]


#also remove any younger ASD siblings of cases so we have each set of parents represented once
#sort the trios by mothers age in ascending order so that the oldest case is on top
sort.order<-order(master.temp.case.eur$mother.age.at.birth.z,decreasing=FALSE)
master.temp.case.unique<-master.temp.case.eur[sort.order,]
#and unique the family ID
master.temp.case.unique<-master.temp.case.unique[!duplicated(master.temp.case.unique$fid),]

##############BAPQ

#define the phenotypes
phen.col.f<-"father.bapq.z"
phen.col.m<-"mother.bapq.z"

bapq.parents.eur<-gen.phen.3(master.temp.case.unique,phen.col.f,phen.col.m)

phenotype<-rep("BAPQ",nrow(bapq.parents.eur))
bapq.parents.eur<-cbind(phenotype,bapq.parents.eur)


###################SRS
#define the phenotypes
phen.col.f<-"father.srs.z"
phen.col.m<-"mother.srs.z"

srs.parents.eur<-gen.phen.3(master.temp.case.unique,phen.col.f,phen.col.m)

phenotype<-rep("SRS",nrow(srs.parents.eur))
srs.parents.eur<-cbind(phenotype,srs.parents.eur)


###################Parental Age
#define the phenotypes
phen.col.f<-"father.age.at.birth.z"
phen.col.m<-"mother.age.at.birth.z"

parental.age.eur<-gen.phen.3c(master.temp.case.unique,phen.col.f,phen.col.m)


phenotype<-rep("Parental Age",nrow(parental.age.eur))
parental.age.eur<-cbind(phenotype,parental.age.eur)

###################EA
#define the phenotypes
phen.col.f<-"father.EA.z"
phen.col.m<-"mother.EA.z"

parental.ea.eur<-gen.phen.3c(master.temp.case.unique,phen.col.f,phen.col.m)

phenotype<-rep("Parental EA",nrow(parental.ea.eur))
parental.ea.eur<-cbind(phenotype,parental.ea.eur)


################################################################
##### Combine all of the results and write them out to csv file
###############################################################

case.results.eur<-rbind(srs.case.eur,scq.case.eur,vine.case.eur,dcdq.case.eur,rbs.case.eur)
case.results.eur<-cbind(case.status=rep("case",nrow(case.results.eur)),case.results.eur)
control.results.eur<-rbind(srs.cont.eur, scq.cont.eur,vine.cont.eur)
control.results.eur<-cbind(case.status=rep("cont",nrow(control.results.eur)),control.results.eur)
parent.results.eur<-rbind(bapq.parents.eur,srs.parents.eur,parental.age.eur,parental.ea.eur)
parent.results.eur<-cbind(case.status=rep("parent",nrow(parent.results.eur)),parent.results.eur)

clin.results.eur<-rbind(case.results.eur,control.results.eur,parent.results.eur)
# write.csv(clin.results.eur,"C:\\Users\\jsebat\\sebatlab Dropbox\\Autism\\Autism_Illumina_WGS\\RV_PRS_Analysis_Jun2020\\clin_results_eur.csv")
# write.csv(clin.results.eur,clin_results_out)
write.csv(clin.results.eur, "clin_results_eur_and_noneur.SBayesR.csv")




In [ ]:
# Installing packages for Figure 5 display
install.packages("tidyverse")
install.packages("dplyr")
install.packages("RColorBrewer")
install.packages("ggplot2")
install.packages("scales")
install.packages("ggpubr", repos = "https://ftp.osuosl.org/pub/cran/", lib="/usr/lib/R/site-library")

library(tidyverse)
library(dplyr)
library(RColorBrewer)
library(ggplot2)
library(scales)
library(ggpubr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘matrixStats’, ‘RcppArmadillo’, ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘sp’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’, ‘corrplot’, ‘car’, ‘ggrepel’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1

In [ ]:
# library(svglite)
# remove.packages("svglite")
# install.packages("svglite", type = "source")
# library(svglite)
# install.packages('svglite')
# install.packages("svglite", type = "source")
# library(svglite)

# remove.packages("svglite")
str(allPackage <- installed.packages())
allPackage [, c(1,3:5)]
install.packages("svglite", type = "source")
str(allPackage <- installed.packages())
allPackage [, c(1,3:5)]


In [ ]:
asd_df <- read.csv(file = clin_results_out, header = TRUE, na.string = "NA")
asd_df <- read.csv(file = "clin_results_eur_and_noneur.SBayesR.csv", header = TRUE, na.string = "NA")

MASSIVE = 50 + 10 + 10
BIG = 30 + 5

# Rounding
asd_df[195,10] = round(asd_df[195,10], digits = 2)
asd_df[195,11] = round(asd_df[195,11], digits = 2)

# Make p-value column ("logP.2" will used for scaling circles in figure)
asd_df$x.pval_log = -log10(asd_df$x.pval)
asd_df <- transform(asd_df, logP.2 = ifelse(x.pval_log >= 5, 5, x.pval_log))

# Concatenate the phenotype and case.status variables
asd_df$pheno = paste(asd_df$phenotype, "_", asd_df$case.status, sep="")

# Create a column of significance labels for significant genotype effects
asd_df$stars <- cut(asd_df$x.pval, breaks=c(-Inf, 0.0007, 0.05, Inf), label = c("**", "*", ""))  # Create column of significance labels for main effect

# Filter data for panel A (cases & controls)
asd_cascon <- asd_df %>%
  filter(case.status %in% c("case", "cont")) %>%
  filter(group %in% c("combined")) %>%
  filter(variant.type %in% c( "dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"))

# Specify the factor levels in the order we want
asd_cascon$pheno <- factor(asd_cascon$pheno, levels = c("RBS_case", "SCQ_case", "SCQ_cont", "SRS_case", "SRS_cont", "VABS_case", "VABS_cont", "DCDQ_case"))
# Reverse the order as follow
asd_cascon$pheno <- factor(asd_cascon$pheno, levels = rev(levels(asd_cascon$pheno)))

asd_cascon$variant.type <- factor(asd_cascon$variant.type, levels = c("dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"))

### Panel A ###
pdf("panelA.test.pdf", width = 30, height = 30)
panelA =
ggballoonplot(asd_cascon, x = "variant.type", y = "pheno", size = "logP.2", size.range = c(5, 20), fill = "est", ggtheme = theme_bw()) +
scale_fill_gradientn(colors = c("blue3", "blue", "white", "indianred1", "red3"), values = rescale(c(-0.43, -0.30, 0, 0.10, 0.28)), limits = c(-0.43, 0.28), guide = guide_colorbar(barwidth = 6, barheight = 30, title.position = "top", label.theme = element_text(size = MASSIVE))) +
geom_text(aes(label = stars), color = "black", vjust = 0.8, size = BIG - 5) +
scale_size(range = c(15*1.53, 60*1.53), breaks =c(1, 2, 3, 4, 5)) +
scale_y_discrete(breaks = c("RBS_case", "SCQ_case", "SCQ_cont", "SRS_case", "SRS_cont", "VABS_case", "VABS_cont", "DCDQ_case"), labels = c("RBS case", "SCQ case", "SCQ cont", "SRS case", "SRS cont", "VABS case", "VABS cont", "DCDQ case")) +
scale_x_discrete(breaks = c("dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"), labels=c("dnMIS", "dnLoF", "inhLoF", expression(PS[ASD]), expression(PS[SCZ]), expression(PS[EA]))) +
labs(size = "-log10(P-value)") +
labs(fill = "Effect Size") +
theme(panel.grid.major = element_blank()) +
theme(panel.grid.minor = element_blank()) +
# Text
theme(axis.text.x = element_text(color = "black", size = MASSIVE)) +
theme(axis.text.y = element_text(color = "black", size = MASSIVE)) +
# Legend
theme(legend.title = element_text(size = MASSIVE)) +
theme(legend.text = element_text(color = "black", size = MASSIVE))
panelA
dev.off();


# Filter data for panel B (parents)
asd_parents <- asd_df %>%
  filter(group %in% c("parents")) %>%
  filter(variant.type %in% c( "dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"))

# Specify the factor levels in the order you want
asd_parents$pheno <- factor(asd_parents$pheno, levels = c("SRS_parent", "BAPQ_parent", "Parental Age_parent", "Parental EA_parent"))
# Reverse the order
asd_parents$pheno <- factor(asd_parents$pheno, levels = rev(levels(asd_parents$pheno)))

asd_parents$variant.type <- factor(asd_parents$variant.type, levels = c("dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"))

### Panel B ###
pdf("panelB.test.pdf", width = 30, height = 30)
panelB =
ggballoonplot(asd_parents, x = "variant.type", y = "pheno", size = "logP.2", size.range = c(5, 20), fill = "est", ggtheme = theme_bw()) +
scale_fill_gradientn(colors = c("blue3", "blue", "white", "indianred1", "red3"), values = rescale(c(-0.43, -0.30, 0, 0.10, 0.28)), limits = c(-0.43, 0.28)) +
geom_text(aes(label = stars), color = "black", vjust = 0.8, size = BIG - 5) +
scale_size(range = c(15*1.53, 60*1.53), breaks =c(1, 2, 3, 4, 5)) +
scale_y_discrete(breaks = c("SRS_parent", "BAPQ_parent", "Parental Age_parent", "Parental EA_parent"), labels = c("SRS", "BAPQ", "Parental Age", "Parental EA")) +
scale_x_discrete(breaks = c("dnmis", "dnlof", "inhLoF", "ps.asd", "ps.sz", "ps.ea"), labels = c("dnMIS", "dnLoF",  "inhLoF", expression(PS[ASD]),  expression(PS[SCZ]), expression(PS[EA]))) +
labs(size = "-log10(P-value)", labels = c("1", "2", "3", "4", "5")) +
labs(fill = "Effect Size [sd]") +
theme(panel.grid.major = element_blank()) +
theme(panel.grid.minor = element_blank()) +
# Text
theme(axis.text.x = element_text(color = "black", size = MASSIVE)) +
theme(axis.text.y = element_text(color = "black", size = MASSIVE)) +
# Legend
theme(legend.title = element_text(size = MASSIVE)) +
theme(legend.text = element_text(color = "black", size = MASSIVE))
panelB
dev.off();

### PANEL AB ###
pdf("panelAB.test.pdf", width = 40, height = 40, onefile = FALSE)
panelAB = 
ggarrange(panelA, panelB,
          labels = c("A","B"),
          nrow = 2,
          ncol = 1,
          widths = c(1, 1), 
          heights = c(1.6, 1),
          # vjust = 1.2,
          common.legend = TRUE, 
          legend = "right")
# theme(plot.margin = unit(c(0,3,0,0), "cm")) + 
panelAB
dev.off();


Scale for 'size' is already present. Adding another scale for 'size', which
will replace the existing scale.



png 
  2

Scale for 'size' is already present. Adding another scale for 'size', which
will replace the existing scale.



png 
  2

png 
  2

In [ ]:
asd_df <- read.csv(file = clin_results_out, header = TRUE, na.string = "NA")
asd_df <- read.csv(file = "clin_results_eur_and_noneur.SBayesR.csv", header = TRUE, na.string = "NA")

# Create columns of significance labels
asd_df$stars.g <- cut(asd_df$x.pval, breaks=c(-Inf, 0.0007, 0.05, Inf), label=c("**", "*", ""))  # Create column of significance labels for genotype effect
asd_df$stars.g.sex <- cut(asd_df$sex.pval, breaks=c(-Inf, 0.0007, 0.05, Inf), label=c("**", "*", ""))  # Create column of significance labels for genotype by sex effect

### PANEL C ###

# Get significant sex interactions p-values and save them for later (for displaying in barplots)
controls_scq_combined_pssz = subset(x = asd_df,
    subset = case.status == "cont" & phenotype == "SCQ" & group == "combined" & variant.type == "ps.sz",)
controls_scq_combined_pssz_sex_pval = toString(controls_scq_combined_pssz$sex.pval)

controls_srs_combined_psasd = subset(x = asd_df,
    subset = case.status == "cont" & phenotype == "SRS" & group == "combined" & variant.type == "ps.asd",)
controls_srs_combined_psasd_sex_pval = toString(controls_srs_combined_psasd$sex.pval)

# cases_scq_combined_inhlof = subset(x = asd_df,
#     subset = case.status == "case" & phenotype == "SCQ" & group == "combined" & variant.type == "inhLoF")
# cases_scq_combined_inhlof_sex_pval = toString(cases_scq_combined_inhlof$sex.pval)


# Select data for panel C (controls for all g*sex interactions PS-SCZ with SCQ and VABS)
asd_C.1 <- asd_df %>%
  filter(case.status %in% c("cont")) %>%
  filter(phenotype %in% c("SRS")) %>%
  filter(group %in% c("male", "female")) %>%
  filter(variant.type %in% c("ps.asd"))

asd_C.2 <- asd_df %>%
  filter(case.status %in% c("cont")) %>%
  filter(phenotype %in% c("SCQ")) %>%
  filter(group %in% c("male", "female")) %>%
  filter(variant.type %in% c("ps.sz"))

# asd_C.3 <- asd_df %>%
#   filter(case.status %in% c("case")) %>%
#   filter(phenotype %in% c("SCQ")) %>%
#   filter(group %in% c("male", "female")) %>%
#   filter(variant.type %in% c("inhLoF"))

asd_C12 <- rbind(asd_C.1, asd_C.2)
my_labellerC12 <- as_labeller(c(SCQ = "SCQ - PS[SCZ]", SRS = "SRS - PS[ASD]"), default = label_parsed) 

# asd_C3 <- rbind(asd_C.3)
# my_labellerC3 <- as_labeller(c(SCQ = "SCQ - inhLoF"), default = label_parsed)

# # Make panel C12 p-value dataframe
# pval_C12_df <- data.frame(
#   phenotype = c("SCQ", "SRS"),
#   start = c("Female", "Female"),
#   end = c("Male", "Male"),
#   y.position = c(0.15, 0.15),
#   pval = c(paste("P = ", controls_scq_combined_pssz_sex_pval,  sep=""), 
#            paste("P = ", controls_srs_combined_psasd_sex_pval, sep=""))
# )

# # Make panel C3 p-value dataframe
# pval_C3_df <- data.frame(
#   phenotype = c("SCQ"),
#   start = c("Female"),
#   end = c("Male"),
#   y.position = c(0.15),
#   pval = c(paste("P = ", cases_scq_combined_inhlof_sex_pval, sep=""))
# )

asd_C12$phen = factor(asd_C12$phenotype, levels=c('SRS','SCQ'))
asd_C12$stars <- cut(asd_C12$x.pval, breaks = c(-Inf, 0.001, 0.01, 0.05, Inf), label = c("***", "**", "*", ""))  # Create column of significance labels for main effect

pdf("panelC.12.test.pdf", width = 30, height = 30)
panelC12 =
ggplot(asd_C12, aes(x = group, y = est, fill = group)) +
geom_bar(stat = "identity", width = 1, position = position_nudge(x = 0.5)) +
facet_wrap(~phenotype, ncol = 2, labeller = my_labellerC12) +
# P-value asterisks
# geom_text(aes(label = stars), color = "black", vjust = -6, size = BIG - 5, hjust = -1.4) +
geom_text(aes(label = stars), color = "black", vjust = 0, size = BIG - 5, hjust = 0) +
# Standard error whiskers
geom_linerange(aes(ymin = est - est.stderr, ymax = est + est.stderr), position = position_nudge(0.5), size = 1) +
scale_fill_manual(values = c("coral", "dodgerblue")) +
labs(y = expression(paste("Effect size of PS [SD]"))) +
labs(x = expression(paste("Controls"))) +
# Text
# theme(axis.title.x = element_blank()) +
theme(axis.title.x = element_text(size = MASSIVE + 5, color = "black", vjust = 10.0)) +
theme(axis.text.x = element_blank()) +
# theme(strip.text.x = element_text(size = MASSIVE - 30, color = "black")) +
theme(strip.text.x = element_text(size = MASSIVE, color = "black")) +
# theme(axis.text.y = element_text(size = MASSIVE - 30)) +
theme(axis.text.y = element_text(size = MASSIVE, color = "black")) +
theme(axis.title.y = element_text(size = MASSIVE + 5, color = "black")) +
# Legend
theme(legend.title = element_blank()) +
theme(legend.text = element_text(size=MASSIVE)) +
theme(legend.key.size = unit(5, "line")) +
theme(panel.background = element_blank()) +
theme(strip.background = element_rect(color = "white", fill = "white", size = 1.5, linetype = "solid")) +
ylim(-0.10, 0.15)
# p-value significance lines
# geom_signif(data = pval_C12_df, aes(xmin = 1.5, xmax = 2.5, annotations = pval, y_position = y.position), manual = TRUE, inherit.aes = FALSE, textsize = BIG - 20)
panelC12
dev.off();

# pdf("panelC.3.test.pdf", width = 30, height = 30)
# panelC3 =
# ggplot(asd_C3, aes(x = group, y = est, fill = group)) +
# geom_bar(stat = "identity", width = 1, position = position_nudge(x = 0.5)) +
# facet_wrap(~phenotype, ncol = 1, labeller = my_labellerC3) +
# geom_linerange(aes(ymin = est - est.stderr, ymax = est + est.stderr), position = position_nudge(0.5), size = 1) +
# scale_fill_manual(values = c("coral", "dodgerblue")) +
# labs(y = expression(paste("Effect size of inhLoF [SD]"))) +
# labs(x = expression(paste("Cases"))) +
# # Text
# # theme(axis.title.x = element_blank()) +
# theme(axis.title.x = element_text(size = MASSIVE + 5, color = "black", vjust = 10.0)) +
# theme(axis.text.x = element_blank()) +
# theme(strip.text.x = element_text(size = MASSIVE - 30, color = "black")) +
# # theme(axis.text.y = element_text(size = MASSIVE - 30)) +
# theme(axis.text.y = element_blank()) +
# theme(axis.ticks.y = element_blank()) +
# theme(axis.title.y = element_text(size = MASSIVE + 5, color = "black")) +
# # Legend
# theme(legend.title = element_blank()) +
# theme(legend.text = element_text(size=MASSIVE)) +
# theme(legend.key.size = unit(5, "line")) +
# theme(panel.background = element_blank()) +
# theme(strip.background = element_rect(color = "white", fill = "white", size = 1.5, linetype = "solid")) +
# ylim(-0.10, 0.15) +
# # p-value significance lines
# geom_signif(data = pval_C3_df, aes(xmin = 1.5, xmax = 2.5, annotations = pval, y_position = y.position), manual = TRUE, inherit.aes = FALSE, textsize = BIG - 20)
# panelC3
# dev.off();

# # Combine panels C12 and C3 into C123
# pdf("panelC.123.test.pdf", width = 30, height = 30, onefile = FALSE)
# panelC123 =
# ggarrange(panelC12, panelC3,
#           labels = c("C12", "C3"),
#           nrow = 1,
#           ncol = 2,
#           common.legend = TRUE,
#           legend = "right",
#           widths = c(2, 1))
# panelC123
# dev.off();


### PANEL D ###
# Select data for panel D (Parental Age Effect size: dnLoF, dnMIS, inhLoF, PS_SCZ, PS_ASD, PS_EA)
asd_D <- asd_df %>%
  filter(phenotype %in% c("Parental Age")) %>%
  filter(group %in% c("mother", "father")) %>%
  # filter(group %in% c("parents")) %>%
  filter(variant.type %in% c("dnlof", "inhLoF", "dnmis", "ps.sz", "ps.asd", "ps.ea"))

my_labellerD = as_labeller(c(dnlof = "dnLoF", inhLoF = "inhLoF", dnmis = "dnMIS", ps.sz = "PS[SCZ]", ps.asd = "PS[ASD]", ps.ea = "PS[EA]"), default = label_parsed)

# Get significant sex interaction p-values
parent_pa_parents_dnlof = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "dnlof",)
parent_pa_parents_dnlof_sex_pval = toString(parent_pa_parents_dnlof$sex.pval)

parent_pa_parents_inhlof = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "inhLoF",)
parent_pa_parents_inhlof_sex_pval = toString(parent_pa_parents_inhlof$sex.pval)

parent_pa_parents_dnmis = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "dnmis",)
parent_pa_parents_dnmis_sex_pval = toString(parent_pa_parents_dnmis$sex.pval)

parent_pa_parents_pssz = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "ps.sz",)
parent_pa_parents_pssz_sex_pval = toString(parent_pa_parents_pssz$sex.pval)

parent_pa_parents_psasd = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "ps.asd",)
parent_pa_parents_psasd_sex_pval = toString(parent_pa_parents_psasd$sex.pval)

parent_pa_parents_psea = subset(x = asd_df,
    subset = case.status == "parent" & phenotype == "Parental Age" & group == "parents" & variant.type == "ps.ea",)
parent_pa_parents_psea_sex_pval = toString(parent_pa_parents_psea$sex.pval)

# Make the p-value dataframe for panel D
pval_D_df <- data.frame(
  variant.type = c("dnlof", "inhLoF", "dnmis", "ps.sz", "ps.asd", "ps.ea"),
  start = c("Father", "Father"),
  end = c("Mother", "Mother"),
  y.position = c(0.2, 0.2),
  pval = c(paste("P = ", parent_pa_parents_dnlof_sex_pval,  sep=""), 
           paste("P = ", parent_pa_parents_inhlof_sex_pval, sep=""),
           paste("P = ", parent_pa_parents_dnmis_sex_pval,  sep=""),
           paste("P = ", parent_pa_parents_pssz_sex_pval,   sep=""),
           paste("P = ", parent_pa_parents_psasd_sex_pval,  sep=""),
           paste("P = ", parent_pa_parents_psea_sex_pval,   sep=""))
)

options(repr.plot.width=15, repr.plot.height=8)

asd_D$stars <- cut(asd_D$x.pval, breaks = c(-Inf, 0.001, 0.01, 0.05, Inf), label = c("***", "**", "*", ""))  # Create column of significance labels for main effect

pdf("panelD.test.pdf", width = 30, height = 30)
panelD =
ggplot(asd_D, aes(x = group, y = est, fill = group)) +
geom_bar(stat = "identity", width = 1, position = position_nudge(x = 0.5)) +
facet_wrap(~variant.type, ncol = 6, labeller = my_labellerD) +
# P-value asterisks
# geom_text(aes(label = stars), color = "black", vjust = -3, size = BIG - 5, hjust = 0) +
geom_text(aes(label = stars), color = "black", vjust = 0, size = BIG - 5, hjust = 0) +
# Standard error whiskers
geom_linerange(aes(ymin = est - est.stderr, ymax = est + est.stderr), position = position_nudge(0.5), size = 1) +
scale_fill_manual(values = c("dodgerblue", "coral")) +
labs(y = "Effect Size [sd]") +
theme(axis.text.x = element_blank()) +
# Text
theme(axis.title.x = element_blank()) +
theme(axis.text.x = element_blank()) +
# theme(strip.text.x = element_text(size = MASSIVE - 30, color = "black")) +
theme(strip.text.x = element_text(size = MASSIVE, color = "black")) +
# theme(axis.text.y = element_text(size = MASSIVE - 30)) +
theme(axis.text.y = element_text(size = MASSIVE, color = "black")) +
theme(axis.title.y = element_text(size = MASSIVE + 5, color = "black")) +
# Legend
theme(legend.title = element_blank()) +
theme(legend.text = element_text(size = MASSIVE)) +
theme(legend.key.size = unit(5, "line")) +
theme(panel.background = element_blank()) +
theme(strip.background = element_rect(color = "white", fill = "white", size = 1.5, linetype = "solid")) +
# y-axis limits
ylim(-0.10, 0.20)
# p-value significance lines
# geom_signif(data = pval_D_df, aes(xmin = 1.5, xmax = 2.5, annotations = pval, y_position = y.position), manual = TRUE, inherit.aes = FALSE, textsize = BIG - 20)
panelD
dev.off();

### PANEL CD ###

pdf("panelCD.test.pdf", width = 40, height = 40)
panelCD =
ggarrange(panelC12, panelD, # panelC123 -> panelC12
          labels = c("C", "D"),
          nrow = 2,
          ncol = 1,
          common.legend = FALSE,
          legend = "right",
          widths = c(1, 1))
panelCD
dev.off();



### PANEL ABCD ###

# pdf("fig5.ABCD.test.pdf",width=60, height = 40)
# pdf("fig5.pdf",width=60, height = 40)
pdf("fig5.eur_and_noneur.pdf",width=60, height = 40)
# svg("fig5.svg", width=60, height=40)
# svglite("fig5.svglite.svg", width = 60, height = 40)
# ggsave("fig5.ggsave.pdf", plot = panelABCD, width = 120, height = 80, units = "cm", limitsize = FALSE)
panelABCD =
panelABCD = 
ggarrange(panelAB, panelCD,
          nrow = 1,
          ncol = 2,
          align = "v",
          widths = c(1.4, 1), heights = c(1, 1),
          common.legend = FALSE)
# panelABCD$layout$clip = "off"
panelABCD;
dev.off();

# ggsave("fig5.ggsave.svg", plot = panelABCD, width = 60, height = 40, units = "cm")



png 
  2

png 
  2

png 
  2

png 
  2

In [ ]:
# Figure 4B

install.packages("corrplot2.tar.gz", repos = NULL)
install.packages("reshape2")
library(corrplot2)
library(reshape2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plyr’


corrplot2 0.1 loaded


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [ ]:
options(stringsAsFactors = FALSE);
df = read.csv(asd_genetic_correlations_by_sex_out, sep = "\t")

pal <- colorRampPalette(c("#000080","white","#b30000"))

get_mat <- function(df, name) {
  mat <- acast(df, y~x, value.var = name, fill = NA)
  ord = c("dnMIS", "dnLOF", "inhLoF",
          "pTDT.ASD.Dev", "pTDT.SCZ.Dev", "pTDT.EA.Dev",
          "RVRS", "PRS");
  mat <- mat[ord, ord]
  
  labs = c("dnMIS", "dnLoF", "inhLoF",
           "$PS[ASD]", "$PS[SCZ]", "$PS[EA]",
           "RVRS", "CVRS");



  colnames(mat) <- labs
  rownames(mat) <- labs
  return(mat)
}



n_pal <- 40
rows <- c("dnMIS","dnLoF","inhLoF","RVRS");
cols <- c("$PS[ASD]","$PS[SCZ]","$PS[EA]","CVRS");


# Males
m_co <- get_mat(df,"male_cor")
m_pv <- get_mat(df,"male_pval")
f_co <- get_mat(df,"female_cor")
f_pv <- get_mat(df,"female_pval")

all_co <- get_mat(df,"combined_cor")
all_pv <- get_mat(df,"combined_pval")

m_co <- m_co[rows,cols]
f_co <- f_co[rows,cols]
all_co <- all_co[rows,cols]

m_pv <- m_pv[rows,cols]
f_pv <- f_pv[rows,cols]
all_pv <- all_pv[rows,cols]

# Rebrand female row names
row.names(f_co) <- c(1,2,3,4)
row.names(f_pv) <- c(1,2,3,4)
# rbind the matrices
sex_co <- rbind(m_co,f_co)
sex_pv <- rbind(m_pv,f_pv)
# Reorder rows
rows <- c("dnMIS",1,"dnLoF",2,"inhLoF",3,"RVRS",4);
sex_co<-sex_co[rows,]
sex_pv<-sex_pv[rows,]


dum <- rbind(1, 0.1, 0.049, 0.0099)
row.names(dum) <- c(1, 0.1, 0.05, 0.01)

sig_lev <- c(.01, .05)

# pdf("genetic_corr_pval_boxes.pdf", width = 16, height = 16)
pdf("fig4b.box.legend.pdf", width=16, height = 16)
# svg("fig4b.box.legend.svg", width=16, height = 16)
corrplot2(
  dum,
  # cbind(dum_cor),
  method = "square",
  outline = TRUE,
  is.corr.= FALSE,
  col = "navy", # color
  tl.col = "black",
  # tl.srt = 30, # rotation 
  tl.cex = 3,
  p.mat = cbind(dum),
  insig = "label_sig",
  cl.pos = FALSE,
  sig.level = sig_lev,
  pch.cex = 5,
  pch.col = "white",
  norm_mat = dum,
);
dev.off()


dum <- rbind(1, 0.75, 0.5, 0.25, 0.1, 0.049, 0.0249, 0.0099)
dum_cor <- matrix(min(m_co, f_co), nrow = length(dum), ncol = 1)
row.names(dum_cor) <- c(1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01)
sig_lev <- c(.01, .05)

# Combined
# pdf("combined_genetic_corr.pdf", width=16, height = 16)
pdf("fig4b.pdf", width=16, height = 16)
# svg("fig4b.svg", width=16, height = 16)
corrplot2(
  all_co,
  method = "square",
  outline = TRUE,
  is.corr = FALSE,
  col = pal(n_pal),
  tl.col = "black",
  tl.srt = 0, # 90
  tl.cex = 3, # 1.5
  tl.offset = 1,
  p.mat = all_pv,
  sig.level =  sig_lev,
  insig = "label_sig",
  pch.cex = 5, # 5
  pch.col = "white",
  norm_mat = dum,
  cl.lim = c(min(m_co, f_co), max(m_co, f_co)),
  cor_max = max(m_co, f_co),
  cor_min = min(m_co,f_co),
  cl.cex = 2,
  cl.ratio = 0.2,
);
dev.off()


Warning message in text.default(pos.xlabel[, 1], pos.xlabel[, 2], newcolnames, srt = tl.srt, :
“"is.corr." is not a graphical parameter”
Warning message in text.default(pos.ylabel[, 1], pos.ylabel[, 2], newrownames, col = tl.col, :
“"is.corr." is not a graphical parameter”
Warning message in title(title, ...):
“"is.corr." is not a graphical parameter”


png 
  2

png 
  2

In [ ]:
system('rm -rf /content/sample_data', intern=TRUE)
system('cd /home && pwd && tar -cvzf antaki_analysis.tar.gz /content', intern=TRUE)

character(0)

[1] "/home"                                                                     
 [2] "/content/"                                                                 
 [3] "/content/.config/"                                                         
 [4] "/content/.config/.last_survey_prompt.yaml"                                 
 [5] "/content/.config/config_sentinel"                                          
 [6] "/content/.config/.last_opt_in_prompt.yaml"                                 
 [7] "/content/.config/active_config"                                            
 [8] "/content/.config/logs/"                                                    
 [9] "/content/.config/logs/2021.07.16/"                                         
[10] "/content/.config/logs/2021.07.16/13.19.23.269640.log"                      
[11] "/content/.config/logs/2021.07.16/13.20.21.584309.log"                      
[12] "/content/.config/logs/2021.07.16/13.20.04.172291.log"                      
[13] "/content/.config/logs/2021.07.16/13.20.22.328313.log"                      
[14] "/content/.config/logs/2021.07.16/13.19.42.311157.log"                      
[15] "/content/.config/logs/2021.07.16/13.19.58.237112.log"                      
[16] "/content/.config/gce"                                                      
[17] "/content/.config/.last_update_check.json"                                  
[18] "/content/.config/configurations/"                                          
[19] "/content/.config/configurations/config_default"                            
[20] "/content/combined_genetic_corr.pdf"                                        
[21] "/content/master_phen_4.SBayesR.csv"                                        
[22] "/content/panelCD.test.pdf"                                                 
[23] "/content/all_t1burden_tdt_results.tsv"                                     
[24] "/content/panelD.test.pdf"                                                  
[25] "/content/SummaryGeneticData_REACH_SSC_SPARK.responses.SBayesR.20210715.csv"
[26] "/content/genetic_corr_pval_boxes.pdf"                                      
[27] "/content/clin_results_eur.SBayesR.20210715.csv"                            
[28] "/content/panelAB.test.pdf"                                                 
[29] "/content/fig5.pdf"                                                         
[30] "/content/panelC.12.test.pdf"                                               
[31] "/content/panelB.test.pdf"                                                  
[32] "/content/fig5.ABCD.test.pdf"                                               
[33] "/content/r2_table.csv"                                                     
[34] "/content/fig4b.pdf"                                                        
[35] "/content/panelA.test.pdf"                                                  
[36] "/content/corrplot2.tar.gz"                                                 
[37] "/content/fig4b.box.legend.pdf"                                             
[38] "/content/asd_genetic_correlations_by_sex_final.SBayesR.20210715.tsv"       
[39] "/content/SummaryGeneticData_REACH_SSC_SPARK.SBayesR.csv"

In [ ]:
# Author: Jonathan Sebat http://sebatlab.org March 19, 2021

# The following phenotypes were available for only one cohort in either cases or controls: srs.case, vabs.case, srs.cont, scq.cont, vabs.cont, bapq.parents, srs.parents)
# For these, age, sex and PCs were included as covariates, but not cohort (a 1st set of functions is written for this test)

# SCQ, RBS and DCDQ were available on cases in the multiple cohorts (SSC and SPARK). Parental age and educational attainment were also available on multiple cohorts
# For these, "cohort" was included as a covariate (a 2nd set of functions are written for this test)

# Importing the input data table that contains rare variant counts (dnLoF, dnMis, inhLoF), polygenic scores(PSasd, PSsz, PSea) and clinical phenotypes (srs, scq, rbs, vabs, dcdq, parental age, parental educational attainment, parental bapq)
master.gen.phen.table <- read.csv(filename_phen, header = TRUE)

# Turn cohort into a factor variable
# master.gen.phen.table$cohort = factor(master.gen.phen.table$cohort)
# Make cohort variables (1 is REACH, 2 is SSC, 3 is SPARK)
master.gen.phen.table$REACH_cohort = as.numeric(master.gen.phen.table$cohort == 1)
master.gen.phen.table$SSC_cohort   = as.numeric(master.gen.phen.table$cohort == 2)
master.gen.phen.table$SPARK_cohort = as.numeric(master.gen.phen.table$cohort == 3)


# Analysis will restricted to subjects of confirmed European descent
master.gen.phen.table = master.gen.phen.table[master.gen.phen.table$is_eur == 1,]

# colnames(master.gen.phen.table)
# Phenotypes
# 'srs.child.age'
# 'vabs.child.age'
# 'scq.child.age'
# 'dcdq.child.age'
# 'mother.EA'
# 'father.EA'
# 'srs.mother'
# 'srs.father'
# 'mother.bapq'
# 'father.bapq'
# 'srs.child.z'
# 'vabs.child.z'
# 'rbs.child.z'
# 'scq.child.z'
# 'dcdq.child.z'
# 'father.age.at.birth.z'
# 'mother.age.at.birth.z'
# 'father.EA.z'
# 'mother.EA.z'
# 'father.srs.z'
# 'mother.srs.z'
# 'father.bapq.z'
# 'mother.bapq.z'

# Genetic factors
# 'dnLoF'
# 'dnMIS'
# 'inhLoF'
# 'PS.ASD.Z'
# 'PS.EA.Z'
# 'PS.SZ.Z'
# 'Paternal.PS.ASD.Z'
# 'Paternal.PS.EA.Z'
# 'Paternal.PS.SZ.Z'
# 'Maternal.PS.ASD.Z'
# 'Maternal.PS.EA.Z'
# 'Maternal.PS.SZ.Z'

run_lr <- function(dataframe, case_status, null_variables, gene_var, response_variable) {
  df = dataframe
  # The model
  if (case_status == "ASD"){
    df = df[df$phen == "ASD",]
  }
  else if (case_status == "CON") {
    df = df[df$phen == "CON",]
  }

  null_formula <- reformulate(null_variables, response = response_variable)
  null_model <- lm(formula = null_formula, data = df, na.action = na.exclude)
  model_summary_null = summary(null_model)

  gene_var <- c(gene_var)
  alt_variables  <- paste(gene_var, null_variables, sep = "+")
  alt_formula <- reformulate(alt_variables, response = response_variable)
  alt_model  <- lm(formula = alt_formula , data = df, na.action = na.exclude)
  model_summary_alt = summary(alt_model)

  # Get number of observations in each model
  # nobs(null_model)
  # nobs(alt_model)

  corr_equation <- paste(response_variable, gene_var, sep = "+")
  corr_formula <- reformulate(corr_equation, response = NULL)
  pearson.cor = cor.test(formula = corr_formula, data = df, na.action = na.exclude)


  # Test sex interaction (all of this just to make a simple string from a character vector...)
  null_variables_vector = unlist(strsplit(null_variables, split = "[+]"))
  null_variables_vector_no_sex_term <- null_variables_vector[1:length(null_variables_vector) - 1]
  null_variables_no_sex_term <- paste(null_variables_vector_no_sex_term, collapse = "+")
  sex_interaction_variable <- paste(gene_var, "sex", sep = "*")
  sex_interaction_terms <- paste(sex_interaction_variable, null_variables_no_sex_term, sep = "+")
  
  sex_interaction_formula <- reformulate(sex_interaction_terms, response = response_variable)
  sex_interaction_model <- lm(formula = sex_interaction_formula, data = df, na.action = na.exclude)
  model_summary_sex_interaction = summary(sex_interaction_model)


  # Sex stratification
  df_combined = df
  # Sets Males to 1, and Females to 2.
  df_combined$sex <- ifelse(df_combined$sex == "Male", 1, 2)
  df_male   = df_combined[df_combined$sex == 1,]
  df_female = df_combined[df_combined$sex == 2,]

  null_model_male <- lm(formula = null_formula, data = df_male, na.action = na.exclude)
  alt_model_male  <- lm(formula = alt_formula, data = df_male, na.action = na.exclude)

  model_summary_null_male <- summary(null_model_male)
  model_summary_alt_male <-  summary(alt_model_male)

  pearson.cor.male = cor.test(formula = corr_formula, data = df_male, na.action = na.exclude)

  null_model_female <- lm(formula = null_formula, data = df_female, na.action = na.exclude)
  alt_model_female  <- lm(formula = alt_formula, data = df_female, na.action = na.exclude)

  model_summary_null_female <- summary(null_model_female)
  model_summary_alt_female <-  summary(alt_model_female)

  pearson.cor.female = cor.test(formula = corr_formula, data = df_female, na.action = na.exclude)

  output <- list(model_summary_alt, model_summary_null, 
                 pearson.cor, model_summary_sex_interaction, 
                 model_summary_alt_male, model_summary_null_male, pearson.cor.male,
                 model_summary_alt_female, model_summary_null_female, pearson.cor.female)
  return(output)
}

# Null variables common to all models
cohort_vars <- c("REACH_cohort+SSC_cohort+SPARK_cohort")
pc_vars <- c("PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")
sex_var <- c("sex") # Only use with combined models (it doesn't matter)
null_vars <- paste(cohort_vars, pc_vars, sex_var, sep = "+") 

# DCDQ models
age_var <- c("dcdq.child.age")
null_vars <- paste(age_var, cohort_vars, pc_vars, sex_var, sep = "+") 
response_var = c("dcdq.child.z")
# Cases
dcdq.dnlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
dcdq.dnmis.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
dcdq.inhlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
dcdq.ps_asd.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
dcdq.ps_ea.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
dcdq.ps_sz.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)
# DCDQ data not available for controls
# dcdq.dnlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
# dcdq.dnmis.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
# dcdq.inhlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
# dcdq.ps_asd.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
# dcdq.ps_ea.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
# dcdq.ps_sz.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)


# RBS models
age_var <- c("scq.child.age") # There isn't any rbs.child.age?
null_vars <- paste(age_var, cohort_vars, pc_vars, sex_var, sep = "+") 
response_var = c("rbs.child.z")
# Cases
rbs.dnlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
rbs.dnmis.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
rbs.inhlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
rbs.ps_asd.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
rbs.ps_ea.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
rbs.ps_sz.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)
# RBS data not available for controls
# rbs.dnlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
# rbs.dnmis.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
# rbs.inhlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
# rbs.ps_asd.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
# rbs.ps_ea.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
# rbs.ps_sz.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)

# SCQ models
age_var <- c("scq.child.age")
null_vars <- paste(age_var, cohort_vars, pc_vars, sex_var, sep = "+") 
response_var = c("scq.child.z")
# Cases
scq.dnlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
scq.dnmis.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
scq.inhlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
scq.ps_asd.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
scq.ps_ea.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
scq.ps_sz.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)
# Controls
scq.dnlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
scq.dnmis.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
scq.inhlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
scq.ps_asd.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
scq.ps_ea.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
scq.ps_sz.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)

# SRS models
age_var <- c("srs.child.age")
null_vars <- paste(age_var, cohort_vars, pc_vars, sex_var, sep = "+") 
response_var = c("srs.child.z")
# Cases
srs.dnlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
srs.dnmis.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
srs.inhlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
srs.ps_asd.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
srs.ps_ea.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
srs.ps_sz.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)
# Controls
srs.dnlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
srs.dnmis.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
srs.inhlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
srs.ps_asd.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
srs.ps_ea.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
srs.ps_sz.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)

# VABS models
age_var <- c("vabs.child.age")
null_vars <- paste(age_var, cohort_vars, pc_vars, sex_var, sep = "+") 
response_var = c("vabs.child.z")
# Cases
vabs.dnlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
vabs.dnmis.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
vabs.inhlof.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
vabs.ps_asd.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
vabs.ps_ea.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
vabs.ps_sz.case.summary = run_lr(dataframe = master.gen.phen.table, case_status = "ASD", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)
# Controls
vabs.dnlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnLoF", response_variable = response_var)
vabs.dnmis.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "dnMIS", response_variable = response_var)
vabs.inhlof.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "inhLoF", response_variable = response_var)
vabs.ps_asd.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.ASD.Z", response_variable = response_var)
vabs.ps_ea.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.EA.Z", response_variable = response_var)
vabs.ps_sz.control.summary = run_lr(dataframe = master.gen.phen.table, case_status = "CON", null_variables = null_vars, gene_var = "PS.SZ.Z", response_variable = response_var)

# Parental models
run_lr_parental <- function(dataframe, null_variables, gene_var, response_variable, combined = FALSE) {
  df = dataframe

  null_formula <- reformulate(null_variables, response = response_variable)
  null_model <- lm(formula = null_formula, data = df, na.action = na.exclude)
  model_summary_null = summary(null_model)

  gene_var <- c(gene_var)
  alt_variables  <- paste(gene_var, null_variables, sep = "+")
  alt_formula <- reformulate(alt_variables, response = response_variable)
  alt_model  <- lm(formula = alt_formula , data = df, na.action = na.exclude)
  model_summary_alt = summary(alt_model)

  # Get number of observations in each model
  # nobs(null_model)
  # nobs(alt_model)

  corr_equation <- paste(response_variable, gene_var, sep = "+")
  corr_formula <- reformulate(corr_equation, response = NULL)
  pearson.cor = cor.test(formula = corr_formula, data = df, na.action = na.exclude)

  output <- list(model_summary_alt, model_summary_null, pearson.cor)
  return(output)
}


# Remove sibling control trios (so that the same parents are not tested twice)
master.gen.phen.table.case = master.gen.phen.table[master.gen.phen.table$phen == "ASD",]
# Also, remove any younger ASD siblings of cases so that we have each set of parents represented only once
sort.order <- order(master.gen.phen.table.case$mother.age.at.birth.z, decreasing = FALSE)
# Sort the trios by mother's age in ascending order so that the oldest case is on top
master.gen.phen.table.case.unique <- master.gen.phen.table.case[sort.order, ]
# And unique family ID
master.gen.phen.table.case.unique <- master.gen.phen.table.case.unique[!duplicated(master.gen.phen.table.case.unique$fid), ]

# Make new variables for inherited variants
master.gen.phen.table.case.unique$father.inh = master.gen.phen.table.case.unique$InhLoF.Paternal.Transmitted + master.gen.phen.table.case.unique$InhLoF.Paternal.Untransmitted
master.gen.phen.table.case.unique$mother.inh = master.gen.phen.table.case.unique$InhLoF.Maternal.Transmitted + master.gen.phen.table.case.unique$InhLoF.Maternal.Untransmitted

# Null variables common to the parental models
# Null model variables
cohort_vars <- c("REACH_cohort+SSC_cohort+SPARK_cohort")
pc_vars_father <- c("PC1_PAT+PC2_PAT+PC3_PAT+PC4_PAT+PC5_PAT+PC6_PAT+PC7_PAT+PC8_PAT+PC9_PAT+PC10_PAT")
pc_vars_mother <- c("PC1_MAT+PC2_MAT+PC3_MAT+PC4_MAT+PC5_MAT+PC6_MAT+PC7_MAT+PC8_MAT+PC9_MAT+PC10_MAT")
null_vars_father <- paste(cohort_vars, pc_vars_father, sep = "+") 
null_vars_mother <- paste(cohort_vars, pc_vars_mother, sep = "+") 

# BAPQ
# Father
response_var = c("father.bapq.z")
bapq.dnlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnLoF", response_variable = response_var)
bapq.dnmis.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnMIS", response_variable = response_var)
bapq.inhlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "father.inh", response_variable = response_var)
bapq.ps_asd.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.ASD.Z", response_variable = response_var)
bapq.ps_ea.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.EA.Z", response_variable = response_var)
bapq.ps_sz.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.SZ.Z", response_variable = response_var)
# Mother
response_var = c("mother.bapq.z")
bapq.dnlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnLoF", response_variable = response_var)
bapq.dnmis.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnMIS", response_variable = response_var)
bapq.inhlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "mother.inh", response_variable = response_var)
bapq.ps_asd.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.ASD.Z", response_variable = response_var)
bapq.ps_ea.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.EA.Z", response_variable = response_var)
bapq.ps_sz.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.SZ.Z", response_variable = response_var)

# SRS
# Father
response_var = c("father.srs.z")
srs.dnlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnLoF", response_variable = response_var)
srs.dnmis.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnMIS", response_variable = response_var)
srs.inhlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "father.inh", response_variable = response_var)
srs.ps_asd.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.ASD.Z", response_variable = response_var)
srs.ps_ea.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.EA.Z", response_variable = response_var)
srs.ps_sz.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.SZ.Z", response_variable = response_var)
# Mother
response_var = c("mother.srs.z")
srs.dnlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnLoF", response_variable = response_var)
srs.dnmis.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnMIS", response_variable = response_var)
srs.inhlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "mother.inh", response_variable = response_var)
srs.ps_asd.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.ASD.Z", response_variable = response_var)
srs.ps_ea.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.EA.Z", response_variable = response_var)
srs.ps_sz.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.SZ.Z", response_variable = response_var)

# Parents' ages at birth
# Father
response_var = c("father.age.at.birth.z")
age_at_birth.dnlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnLoF", response_variable = response_var)
age_at_birth.dnmis.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnMIS", response_variable = response_var)
age_at_birth.inhlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "father.inh", response_variable = response_var)
age_at_birth.ps_asd.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.ASD.Z", response_variable = response_var)
age_at_birth.ps_ea.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.EA.Z", response_variable = response_var)
age_at_birth.ps_sz.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.SZ.Z", response_variable = response_var)
# Mother
response_var = c("mother.age.at.birth.z")
age_at_birth.dnlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnLoF", response_variable = response_var)
age_at_birth.dnmis.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnMIS", response_variable = response_var)
age_at_birth.inhlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "mother.inh", response_variable = response_var)
age_at_birth.ps_asd.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.ASD.Z", response_variable = response_var)
age_at_birth.ps_ea.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.EA.Z", response_variable = response_var)
age_at_birth.ps_sz.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.SZ.Z", response_variable = response_var)

# Parents' educational attainment
# Father
response_var = c("father.EA.z")
ea.dnlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnLoF", response_variable = response_var)
ea.dnmis.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "dnMIS", response_variable = response_var)
ea.inhlof.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "father.inh", response_variable = response_var)
ea.ps_asd.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.ASD.Z", response_variable = response_var)
ea.ps_ea.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.EA.Z", response_variable = response_var)
ea.ps_sz.father.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_father, gene_var = "Paternal.PS.SZ.Z", response_variable = response_var)
# Mother
response_var = c("mother.EA.z")
ea.dnlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnLoF", response_variable = response_var)
ea.dnmis.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "dnMIS", response_variable = response_var)
ea.inhlof.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "mother.inh", response_variable = response_var)
ea.ps_asd.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.ASD.Z", response_variable = response_var)
ea.ps_ea.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.EA.Z", response_variable = response_var)
ea.ps_sz.mother.summary = run_lr_parental(dataframe = master.gen.phen.table.case.unique, null_variables = null_vars_mother, gene_var = "Maternal.PS.SZ.Z", response_variable = response_var)


# Combined parental model
# Making father table
master.gen.phen.table.case.unique.father = master.gen.phen.table.case.unique[, 
          c("fid", "cohort", "REACH_cohort", "SSC_cohort", "SPARK_cohort", 
            "dnLoF", "dnMIS", "father.inh",
            "Paternal.PS.ASD.Z", "Paternal.PS.EA.Z", "Paternal.PS.SZ.Z",
            "PC1_PAT", "PC2_PAT", "PC3_PAT", "PC4_PAT", "PC5_PAT",
            "PC6_PAT", "PC7_PAT", "PC8_PAT", "PC9_PAT", "PC10_PAT",
            "father.age.at.birth.z", "father.EA.z", "father.srs.z", "father.bapq.z"), 
          drop = FALSE]
master.gen.phen.table.case.unique.father$Sex = 1

colnames(master.gen.phen.table.case.unique.father) <- c("FID", "Cohort",
              "REACH_cohort", "SSC_cohort", "SPARK_cohort",
              "dnLoF", "dnMis", "inh",
              "PS.ASD.Z", "PS.EA.Z", "PS.SZ.Z",
              "PC1", "PC2", "PC3", "PC4", "PC5", 
              "PC6", "PC7", "PC8", "PC9", "PC10",
              "age.at.birth.z", "EA.z", "srs.z", "bapq.z",
              "Sex")
row.names(master.gen.phen.table.case.unique.father) <- NULL

# Making mother table
master.gen.phen.table.case.unique.mother = master.gen.phen.table.case.unique[, 
          c("fid", "cohort", "REACH_cohort", "SSC_cohort", "SPARK_cohort", 
            "dnLoF", "dnMIS", "mother.inh",
            "Maternal.PS.ASD.Z", "Maternal.PS.EA.Z", "Maternal.PS.SZ.Z",
            "PC1_MAT", "PC2_MAT", "PC3_MAT", "PC4_MAT", "PC5_MAT",
            "PC6_MAT", "PC7_MAT", "PC8_MAT", "PC9_MAT", "PC10_MAT",
            "mother.age.at.birth.z", "mother.EA.z", "mother.srs.z", "mother.bapq.z"), 
          drop = FALSE]
master.gen.phen.table.case.unique.mother$Sex = 2

colnames(master.gen.phen.table.case.unique.mother) <- c("FID", "Cohort",
              "REACH_cohort", "SSC_cohort", "SPARK_cohort",
              "dnLoF", "dnMis", "inh",
              "PS.ASD.Z", "PS.EA.Z", "PS.SZ.Z",
              "PC1", "PC2", "PC3", "PC4", "PC5", 
              "PC6", "PC7", "PC8", "PC9", "PC10",
              "age.at.birth.z", "EA.z", "srs.z", "bapq.z",
              "Sex")
row.names(master.gen.phen.table.case.unique.mother) <- NULL

# Make the final combined parental table
master.gen.phen.table.case.unique.combined <- rbind(master.gen.phen.table.case.unique.father, master.gen.phen.table.case.unique.mother)

# Combined parental model function
run_lr_parental_combined <- function(dataframe, cohort_vars, pc_vars, sex_var, gene_var, response_variable) {
  df = dataframe

  null_variables <- paste(cohort_vars, pc_vars, sex_var, sep = "+") 

  null_formula <- reformulate(null_variables, response = response_variable)
  null_model <- lm(formula = null_formula, data = df, na.action = na.exclude)
  model_summary_null = summary(null_model)

  gene_var <- c(gene_var)
  alt_variables  <- paste(gene_var, null_variables, sep = "+")
  alt_formula <- reformulate(alt_variables, response = response_variable)
  alt_model  <- lm(formula = alt_formula , data = df, na.action = na.exclude)
  model_summary_alt = summary(alt_model)

  corr_equation <- paste(response_variable, gene_var, sep = "+")
  corr_formula <- reformulate(corr_equation, response = NULL)
  pearson.cor = cor.test(formula = corr_formula, data = df, na.action = na.exclude)

  # Test sex interaction
  sex_interaction_variable <- paste(gene_var, sex_var, sep = "*")
  sex_interaction_terms <- paste(sex_interaction_variable, cohort_vars, pc_vars, sep = "+")
  
  sex_interaction_formula <- reformulate(sex_interaction_terms, response = response_variable)
  sex_interaction_model <- lm(formula = sex_interaction_formula, data = df, na.action = na.exclude)
  model_summary_sex_interaction = summary(sex_interaction_model)


  output <- list(model_summary_alt, model_summary_null, pearson.cor, model_summary_sex_interaction)
  return(output)
}


# Run the combined parental models
# Null model variables
cohort_vars <- c("REACH_cohort+SSC_cohort+SPARK_cohort")
pc_vars <- c("PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")
sex_var <- c("Sex") # Only use with combined models (it doesn't matter)

# BAPQ models
response_var = c("bapq.z")
bapq.dnlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLoF", response_variable = response_var)
bapq.dnmis.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMis", response_variable = response_var)
bapq.inhlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inh", response_variable = response_var)
bapq.ps_asd.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.ASD.Z", response_variable = response_var)
bapq.ps_ea.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.EA.Z", response_variable = response_var)
bapq.ps_sz.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.SZ.Z", response_variable = response_var)

# Parental Age models
response_var = c("age.at.birth.z")
age_at_birth.dnlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLoF", response_variable = response_var)
age_at_birth.dnmis.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMis", response_variable = response_var)
age_at_birth.inhlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inh", response_variable = response_var)
age_at_birth.ps_asd.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.ASD.Z", response_variable = response_var)
age_at_birth.ps_ea.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.EA.Z", response_variable = response_var)
age_at_birth.ps_sz.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.SZ.Z", response_variable = response_var)

# Parental EA models
response_var = c("EA.z")
ea.dnlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLoF", response_variable = response_var)
ea.dnmis.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMis", response_variable = response_var)
ea.inhlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inh", response_variable = response_var)
ea.ps_asd.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.ASD.Z", response_variable = response_var)
ea.ps_ea.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.EA.Z", response_variable = response_var)
ea.ps_sz.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.SZ.Z", response_variable = response_var)

# SRS models
response_var = c("srs.z")
srs.dnlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLoF", response_variable = response_var)
srs.dnmis.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMis", response_variable = response_var)
srs.inhlof.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inh", response_variable = response_var)
srs.ps_asd.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.ASD.Z", response_variable = response_var)
srs.ps_ea.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.EA.Z", response_variable = response_var)
srs.ps_sz.combined.summary = run_lr_parental_combined(dataframe = master.gen.phen.table.case.unique.combined, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PS.SZ.Z", response_variable = response_var)



# Put model summaries in a list
dcdq_case_model_summaries <- list(dcdq.dnlof.case.summary, dcdq.dnmis.case.summary, dcdq.inhlof.case.summary, dcdq.ps_asd.case.summary, dcdq.ps_ea.case.summary, dcdq.ps_sz.case.summary)
rbs_case_model_summaries <- list(rbs.dnlof.case.summary, rbs.dnmis.case.summary, rbs.inhlof.case.summary, rbs.ps_asd.case.summary, rbs.ps_ea.case.summary, rbs.ps_sz.case.summary)
scq_case_model_summaries <- list(scq.dnlof.case.summary, scq.dnmis.case.summary, scq.inhlof.case.summary, scq.ps_asd.case.summary, scq.ps_ea.case.summary, scq.ps_sz.case.summary)
scq_control_model_summaries <- list(scq.dnlof.control.summary, scq.dnmis.control.summary, scq.inhlof.control.summary, scq.ps_asd.control.summary, scq.ps_ea.control.summary, scq.ps_sz.control.summary)
srs_case_model_summaries <- list(srs.dnlof.case.summary, srs.dnmis.case.summary, srs.inhlof.case.summary, srs.ps_asd.case.summary, srs.ps_ea.case.summary, srs.ps_sz.case.summary)
srs_control_model_summaries <- list(srs.dnlof.control.summary, srs.dnmis.control.summary, srs.inhlof.control.summary, srs.ps_asd.control.summary, srs.ps_ea.control.summary, srs.ps_sz.control.summary)
vabs_case_model_summaries <- list(vabs.dnlof.case.summary, vabs.dnmis.case.summary, vabs.inhlof.case.summary, vabs.ps_asd.case.summary, vabs.ps_ea.case.summary, vabs.ps_sz.case.summary)
vabs_control_model_summaries <- list(vabs.dnlof.control.summary, vabs.dnmis.control.summary, vabs.inhlof.control.summary, vabs.ps_asd.control.summary, vabs.ps_ea.control.summary, vabs.ps_sz.control.summary)

# Combine model outputs to get summary statistics and put in a table
model_dict <- ordered_dict(items = NULL, keys = NULL)

model_dict$set("DCDQ_case", dcdq_case_model_summaries)
model_dict$set("RBS_case", rbs_case_model_summaries)
model_dict$set("SCQ_case", scq_case_model_summaries)
model_dict$set("SCQ_cont", scq_control_model_summaries)
model_dict$set("SRS_case", srs_case_model_summaries)
model_dict$set("SRS_cont", srs_control_model_summaries)
model_dict$set("VABS_case", vabs_case_model_summaries)
model_dict$set("VABS_cont", vabs_control_model_summaries)

keys = list("DCDQ_case", "RBS_case", "SCQ_case", "SCQ_cont", "SRS_case", "SRS_cont", "VABS_case", "VABS_cont")

clinical_results_table_offspring <- data.frame()

for (key in keys) {

  phenotype = str_split(key, "_")[[1]][1]
  case.status = str_split(key, "_")[[1]][2]

  model_summary_list = model_dict$get(key)
  for (model_summaries in model_summary_list) {
    # Combined
    model_summary_alt  = model_summaries[[1]]
    model_summary_null = model_summaries[[2]]

    x = rownames(coefficients(model_summary_alt))[2]
    x_estimate = signif(coefficients(model_summary_alt)[2, 1], digits = 2)
    x_stderr = signif(coefficients(model_summary_alt)[2, 2], digits = 2)
    x_pval = signif(coefficients(model_summary_alt)[2, 4], digits = 2)
    r2 = signif(model_summary_alt$r.squared - model_summary_null$r.squared, digits = 2)

    # Correlation
    pearson.cor = model_summaries[[3]]
    pearson.cc = signif(pearson.cor$estimate, digits = 2)

    # Sex interaction
    model_summary_sex_interaction = model_summaries[[4]]
    sex_interaction_model_coeffs = coefficients(model_summary_sex_interaction)
    num_rows = dim(sex_interaction_model_coeffs)[1]
    sex_interaction_coeff = sex_interaction_model_coeffs[num_rows, ]
    sex_interaction_pval = signif(sex_interaction_coeff[[4]], digits = 2)

    # Males
    model_summary_alt_male    = model_summaries[[5]]
    model_summary_null_male   = model_summaries[[6]]
    pearson.cor.male = model_summaries[[7]]
    pearson.cc.male = signif(pearson.cor.male$estimate, digits = 2)


    x_estimate_male = signif(coefficients(model_summary_alt_male)[2, 1], digits = 2)
    x_stderr_male = signif(coefficients(model_summary_alt_male)[2, 2], digits = 2)
    x_pval_male = signif(coefficients(model_summary_alt_male)[2, 4], digits = 2)
    r2_male = signif(model_summary_alt_male$r.squared - model_summary_null_male$r.squared, digits = 2)

    # Females
    model_summary_alt_female  = model_summaries[[8]]
    model_summary_null_female = model_summaries[[9]]
    pearson.cor.female = model_summaries[[10]]
    pearson.cc.female = signif(pearson.cor.female$estimate, digits = 2)

    x_estimate_female = signif(coefficients(model_summary_alt_female)[2, 1], digits = 2)
    x_stderr_female = signif(coefficients(model_summary_alt_female)[2, 2], digits = 2)
    x_pval_female = signif(coefficients(model_summary_alt_female)[2, 4], digits = 2)
    r2_female = signif(model_summary_alt_female$r.squared - model_summary_null_female$r.squared, digits = 2)

    df_tmp_female = data.frame(case.status, phenotype, "female",
                  x, r2_female, x_estimate_female, x_stderr_female, 
                  pearson.cc.female, x_pval_female, NA)
    
    df_tmp_male = data.frame(case.status, phenotype, "male",
                  x, r2_male, x_estimate_male, x_stderr_male, 
                  pearson.cc.male, x_pval_male, NA)

    df_tmp_combined = data.frame(case.status, phenotype, "combined",
                  x, r2, x_estimate, x_stderr, 
                  pearson.cc, x_pval, sex_interaction_pval)

    colnames(df_tmp_female) = colnames(df_tmp_male) = colnames(df_tmp_combined)
    df_tmp = rbind(df_tmp_combined, df_tmp_male, df_tmp_female)

    clinical_results_table_offspring = rbind(clinical_results_table_offspring, df_tmp)
  }
}

row.names(clinical_results_table_offspring) <- NULL

# Put model summaries in a list
bapq_father_model_summaries <- list(bapq.dnlof.father.summary, bapq.dnmis.father.summary, bapq.inhlof.father.summary, bapq.ps_asd.father.summary, bapq.ps_ea.father.summary, bapq.ps_sz.father.summary)
bapq_mother_model_summaries <- list(bapq.dnlof.mother.summary, bapq.dnmis.mother.summary, bapq.inhlof.mother.summary, bapq.ps_asd.mother.summary, bapq.ps_ea.mother.summary, bapq.ps_sz.mother.summary)
srs_father_model_summaries <- list(srs.dnlof.father.summary, srs.dnmis.father.summary, srs.inhlof.father.summary, srs.ps_asd.father.summary, srs.ps_ea.father.summary, srs.ps_sz.father.summary)
srs_mother_model_summaries <- list(srs.dnlof.mother.summary, srs.dnmis.mother.summary, srs.inhlof.mother.summary, srs.ps_asd.mother.summary, srs.ps_ea.mother.summary, srs.ps_sz.mother.summary)
parental.age_father_model_summaries <- list(age_at_birth.dnlof.father.summary, age_at_birth.dnmis.father.summary, age_at_birth.inhlof.father.summary, age_at_birth.ps_asd.father.summary, age_at_birth.ps_ea.father.summary, age_at_birth.ps_sz.father.summary)
parental.age_mother_model_summaries <- list(age_at_birth.dnlof.mother.summary, age_at_birth.dnmis.mother.summary, age_at_birth.inhlof.mother.summary, age_at_birth.ps_asd.mother.summary, age_at_birth.ps_ea.mother.summary, age_at_birth.ps_sz.mother.summary)
parental.EA_father_model_summaries <- list(ea.dnlof.father.summary, ea.dnmis.father.summary, ea.inhlof.father.summary, ea.ps_asd.father.summary, ea.ps_ea.father.summary, ea.ps_sz.father.summary)
parental.EA_mother_model_summaries <- list(ea.dnlof.mother.summary, ea.dnmis.mother.summary, ea.inhlof.mother.summary, ea.ps_asd.mother.summary, ea.ps_ea.mother.summary, ea.ps_sz.mother.summary)

# Combine model outputs to get summary statistics and put in a table
model_dict_parents <- ordered_dict(items = NULL, keys = NULL)

model_dict_parents$set("BAPQ_father", bapq_father_model_summaries)
model_dict_parents$set("BAPQ_mother", bapq_mother_model_summaries)
model_dict_parents$set("SRS_father", srs_father_model_summaries)
model_dict_parents$set("SRS_mother", srs_mother_model_summaries)
model_dict_parents$set("Parental Age_father", parental.age_father_model_summaries)
model_dict_parents$set("Parental Age_mother", parental.age_mother_model_summaries)
model_dict_parents$set("Parental EA_father", parental.EA_father_model_summaries)
model_dict_parents$set("Parental EA_mother", parental.EA_mother_model_summaries)

keys_parents = list("BAPQ_father", "BAPQ_mother", "SRS_father", "SRS_mother", "Parental Age_father", "Parental Age_mother", "Parental EA_father", "Parental EA_mother")

clinical_results_table_parents <- data.frame()

for (key in keys_parents) {

  phenotype = str_split(key, "_")[[1]][1]
  group = str_split(key, "_")[[1]][2]

  case.status = "parent"

  model_summary_list = model_dict_parents$get(key)
  for (model_summaries in model_summary_list) {
    # Combined
    model_summary_alt  = model_summaries[[1]]
    model_summary_null = model_summaries[[2]]

    x = rownames(coefficients(model_summary_alt))[2]
    x_estimate = signif(coefficients(model_summary_alt)[2, 1], digits = 2)
    x_stderr = signif(coefficients(model_summary_alt)[2, 2], digits = 2)
    x_pval = signif(coefficients(model_summary_alt)[2, 4], digits = 2)
    r2 = signif(model_summary_alt$r.squared - model_summary_null$r.squared, digits = 2)

    # Correlation
    pearson.cor = model_summaries[[3]]
    pearson.cc = signif(pearson.cor$estimate, digits = 2)

    df_tmp = data.frame(case.status, phenotype, group,
                  x, r2, x_estimate, x_stderr, 
                  NA, x_pval, NA)

    clinical_results_table_parents = rbind(clinical_results_table_parents, df_tmp)
  }
}

row.names(clinical_results_table_parents) <- NULL

# Put combined parents model summaries in a list
bapq_parents_model_summaries <- list(bapq.dnlof.combined.summary, bapq.dnmis.combined.summary, bapq.inhlof.combined.summary, bapq.ps_asd.combined.summary, bapq.ps_ea.combined.summary, bapq.ps_sz.combined.summary)
parental.age_parents_model_summaries <- list(age_at_birth.dnlof.combined.summary, age_at_birth.dnmis.combined.summary, age_at_birth.inhlof.combined.summary, age_at_birth.ps_asd.combined.summary, age_at_birth.ps_ea.combined.summary, age_at_birth.ps_sz.combined.summary)
parental.EA_parents_model_summaries <- list(ea.dnlof.combined.summary, ea.dnmis.combined.summary, ea.inhlof.combined.summary, ea.ps_asd.combined.summary, ea.ps_ea.combined.summary, ea.ps_sz.combined.summary)
SRS_parents_model_summaries <- list(srs.dnlof.combined.summary, srs.dnmis.combined.summary, srs.inhlof.combined.summary, srs.ps_asd.combined.summary, srs.ps_ea.combined.summary, srs.ps_sz.combined.summary)

# Combine model outputs to get summary statistics and put in a table
model_dict_combined <- ordered_dict(items = NULL, keys = NULL)

model_dict_combined$set("BAPQ_parents", bapq_parents_model_summaries)
model_dict_combined$set("Parental Age_parents", parental.age_parents_model_summaries)
model_dict_combined$set("Parental EA_parents", parental.EA_parents_model_summaries)
model_dict_combined$set("SRS_parents", SRS_parents_model_summaries)

keys_combined = list("BAPQ_parents", "Parental Age_parents", "Parental EA_parents", "SRS_parents")

clinical_results_table_combined <- data.frame()

for (key in keys_combined) {

  phenotype = str_split(key, "_")[[1]][1]
  group = str_split(key, "_")[[1]][2]

  case.status = "parent"

  model_summary_list = model_dict_combined$get(key)
  for (model_summaries in model_summary_list) {
    # Combined
    model_summary_alt  = model_summaries[[1]]
    model_summary_null = model_summaries[[2]]

    x = rownames(coefficients(model_summary_alt))[2]
    x_estimate = signif(coefficients(model_summary_alt)[2, 1], digits = 2)
    x_stderr = signif(coefficients(model_summary_alt)[2, 2], digits = 2)
    x_pval = signif(coefficients(model_summary_alt)[2, 4], digits = 2)
    r2 = signif(model_summary_alt$r.squared - model_summary_null$r.squared, digits = 2)

    # Correlation
    pearson.cor = model_summaries[[3]]
    pearson.cc = signif(pearson.cor$estimate, digits = 2)

    # Sex interaction
    model_summary_sex_interaction = model_summaries[[4]]
    sex_interaction_model_coeffs = coefficients(model_summary_sex_interaction)
    num_rows = dim(sex_interaction_model_coeffs)[1]
    sex_interaction_coeff = sex_interaction_model_coeffs[num_rows, ]
    sex_interaction_pval = signif(sex_interaction_coeff[[4]], digits = 2)

    df_tmp = data.frame(case.status, phenotype, group,
                  x, r2, x_estimate, x_stderr, 
                  pearson.cc, x_pval, sex_interaction_pval)

    clinical_results_table_combined = rbind(clinical_results_table_combined, df_tmp)
  }
}

row.names(clinical_results_table_combined) <- NULL

colnames(clinical_results_table_offspring) = colnames(clinical_results_table_parents) = colnames(clinical_results_table_combined)
df_clinical_results = rbind(clinical_results_table_offspring, clinical_results_table_parents, clinical_results_table_combined)

df_clinical_results

case.status,phenotype,group,x,r2,x_estimate,x_stderr,pearson.cc,x_pval,sex_interaction_pval
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
case,DCDQ,combined,dnLoF,1.1e-02,-0.42000,0.065,-0.11000,1.5e-10,0.074
case,DCDQ,male,dnLoF,6.6e-03,-0.36000,0.073,-0.09400,9.8e-07,NA
case,DCDQ,female,dnLoF,3.3e-02,-0.52000,0.140,-0.18000,3.7e-04,NA
case,DCDQ,combined,dnMIS,5.3e-03,-0.35000,0.074,-0.08100,3.4e-06,0.640
case,DCDQ,male,dnMIS,3.5e-03,-0.33000,0.084,-0.07300,9.2e-05,NA
case,DCDQ,female,dnMIS,2.2e-02,-0.39000,0.160,-0.11000,1.7e-02,NA
case,DCDQ,combined,inhLoF,1.0e-03,-0.04300,0.021,-0.03400,3.9e-02,0.910
case,DCDQ,male,inhLoF,1.2e-03,-0.04500,0.022,-0.03600,4.2e-02,NA
case,DCDQ,female,inhLoF,4.1e-04,-0.02900,0.052,-0.02800,5.7e-01,NA


In [ ]:
df = read.csv(responses_out, sep = ",")
df$dnAll = df$dnLOF + df$dnMIS
df <- df[df$EUR.Ancestry == 1, ]

# Fit models
run_lr_genetic_correlations <- function(dataframe, phenotype_var, cohort_vars, pc_vars, sex_var, gene_var, response_variable) {
  df = dataframe

  null_variables <- paste(phenotype_var, cohort_vars, pc_vars, sex_var, sep = "+") 

  null_formula <- reformulate(null_variables, response = response_variable)
  null_model <- lm(formula = null_formula, data = df, na.action = na.exclude)
  model_summary_null = summary(null_model)

  gene_var <- c(gene_var)
  alt_variables  <- paste(gene_var, null_variables, sep = "+")
  alt_formula <- reformulate(alt_variables, response = response_variable)
  alt_model  <- lm(formula = alt_formula , data = df, na.action = na.exclude)
  model_summary_alt = summary(alt_model)

  corr_equation <- paste(response_variable, gene_var, sep = "+")
  corr_formula <- reformulate(corr_equation, response = NULL)
  pearson.cor = cor.test(formula = corr_formula, data = df, na.action = na.exclude)

  # Test sex interaction
  sex_interaction_variable <- paste(gene_var, sex_var, sep = "*")
  sex_interaction_terms <- paste(sex_interaction_variable, phenotype_var, cohort_vars, pc_vars, sep = "+")
  
  sex_interaction_formula <- reformulate(sex_interaction_terms, response = response_variable)
  sex_interaction_model <- lm(formula = sex_interaction_formula, data = df, na.action = na.exclude)
  model_summary_sex_interaction = summary(sex_interaction_model)

  # Sex stratification
  # Males are 0, females are 1
  df_male   = df[df$Sex == 0,]
  df_female = df[df$Sex == 1,]

  null_model_male <- lm(formula = null_formula, data = df_male, na.action = na.exclude)
  alt_model_male  <- lm(formula = alt_formula, data = df_male, na.action = na.exclude)

  model_summary_null_male <- summary(null_model_male)
  model_summary_alt_male <-  summary(alt_model_male)

  pearson.cor.male = cor.test(formula = corr_formula, data = df_male, na.action = na.exclude)

  null_model_female <- lm(formula = null_formula, data = df_female, na.action = na.exclude)
  alt_model_female  <- lm(formula = alt_formula, data = df_female, na.action = na.exclude)

  model_summary_null_female <- summary(null_model_female)
  model_summary_alt_female <-  summary(alt_model_female)

  pearson.cor.female = cor.test(formula = corr_formula, data = df_female, na.action = na.exclude)

  output <- list(model_summary_alt, model_summary_null, 
                 pearson.cor, model_summary_sex_interaction, 
                 model_summary_alt_male, model_summary_null_male, pearson.cor.male,
                 model_summary_alt_female, model_summary_null_female, pearson.cor.female)
  return(output)
}


# Null model variables
phenotype_var <- c("Phenotype")
cohort_vars <- c("REACH_cohort+SSC_cohort+SPARK_cohort")
pc_vars <- c("PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10")
sex_var <- c("Sex") # Only use with combined models (it doesn't matter)

# I can just for loop this, but I prefer this approach...

# Responses (response.input.summary)
# dnMis
response_var = c("dnMIS")
# dnmis.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
dnmis.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
dnmis.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
dnmis.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
dnmis.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
dnmis.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
dnmis.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
dnmis.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
dnmis.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# dnLoF
response_var = c("dnLOF")
dnlof.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
# dnlof.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
dnlof.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
dnlof.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
dnlof.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
dnlof.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
dnlof.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
dnlof.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
dnlof.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# dnAll
response_var = c("dnAll")
dnall.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
dnall.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
# dnall.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
dnall.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
dnall.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
dnall.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
dnall.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
dnall.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
dnall.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# inhLoF
response_var = c("inhLoF")
inhlof.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
inhlof.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
inhlof.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
# inhlof.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
inhlof.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
inhlof.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
inhlof.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
inhlof.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
inhlof.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# RVRS (Rare Variant Risk Score)
response_var = c("RVRS")
rvrs.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
rvrs.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
rvrs.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
rvrs.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
# rvrs.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
rvrs.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
rvrs.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
rvrs.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
rvrs.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# pTDT.ASD.Dev
response_var = c("pTDT.ASD.Dev")
ptdt_asd_dev.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
ptdt_asd_dev.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
ptdt_asd_dev.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
ptdt_asd_dev.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
ptdt_asd_dev.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
# ptdt_asd_dev.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
ptdt_asd_dev.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
ptdt_asd_dev.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
ptdt_asd_dev.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# pTDT.SCZ.Dev
response_var = c("pTDT.SCZ.Dev")
ptdt_scz_dev.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
ptdt_scz_dev.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
ptdt_scz_dev.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
ptdt_scz_dev.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
ptdt_scz_dev.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
ptdt_scz_dev.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
# ptdt_scz_dev.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
ptdt_scz_dev.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
ptdt_scz_dev.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# pTDT.EA.Dev
response_var = c("pTDT.EA.Dev")
ptdt_ea_dev.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
ptdt_ea_dev.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
ptdt_ea_dev.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
ptdt_ea_dev.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
ptdt_ea_dev.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
ptdt_ea_dev.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
ptdt_ea_dev.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
# ptdt_ea_dev.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
ptdt_ea_dev.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)

# PRS (a.k.a. Common Variant Risk Score, or CVRS)
response_var = c("PRS")
prs.dnmis.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnMIS", response_variable = response_var)
prs.dnlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnLOF", response_variable = response_var)
prs.dnall.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "dnAll", response_variable = response_var)
prs.inhlof.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "inhLoF", response_variable = response_var)
prs.rvrs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "RVRS", response_variable = response_var)
prs.ptdt_asd_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.ASD.Dev", response_variable = response_var)
prs.ptdt_scz_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.SCZ.Dev", response_variable = response_var)
prs.ptdt_ea_dev.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "pTDT.EA.Dev", response_variable = response_var)
# prs.prs.summary = run_lr_genetic_correlations(dataframe = df, phenotype_var = phenotype_var, cohort_vars = cohort_vars, pc_vars = pc_vars, sex_var = sex_var, gene_var = "PRS", response_variable = response_var)


# Put model summaries in a list
dnmis_model_summaries <- list(dnmis.dnlof.summary, dnmis.dnall.summary, dnmis.inhlof.summary, dnmis.rvrs.summary, dnmis.ptdt_asd_dev.summary, dnmis.ptdt_scz_dev.summary, dnmis.ptdt_ea_dev.summary, dnmis.prs.summary)
dnlof_model_summaries <- list(dnlof.dnmis.summary, dnlof.dnall.summary, dnlof.inhlof.summary, dnlof.rvrs.summary, dnlof.ptdt_asd_dev.summary, dnlof.ptdt_scz_dev.summary, dnlof.ptdt_ea_dev.summary, dnlof.prs.summary)
dnall_model_summaries <- list(dnall.dnmis.summary, dnall.dnlof.summary, dnall.inhlof.summary, dnall.rvrs.summary, dnall.ptdt_asd_dev.summary, dnall.ptdt_scz_dev.summary, dnall.ptdt_ea_dev.summary, dnall.prs.summary)
inhlof_model_summaries <- list(inhlof.dnmis.summary, inhlof.dnlof.summary, inhlof.dnall.summary, inhlof.rvrs.summary, inhlof.ptdt_asd_dev.summary, inhlof.ptdt_scz_dev.summary, inhlof.ptdt_ea_dev.summary, inhlof.prs.summary)
rvrs_model_summaries <- list(rvrs.dnmis.summary, rvrs.dnlof.summary, rvrs.dnall.summary, rvrs.inhlof.summary, rvrs.ptdt_asd_dev.summary, rvrs.ptdt_scz_dev.summary, rvrs.ptdt_ea_dev.summary, rvrs.prs.summary)
ptdt.asd.dev_model_summaries <- list(ptdt_asd_dev.dnmis.summary, ptdt_asd_dev.dnlof.summary, ptdt_asd_dev.dnall.summary, ptdt_asd_dev.inhlof.summary, ptdt_asd_dev.rvrs.summary, ptdt_asd_dev.ptdt_scz_dev.summary, ptdt_asd_dev.ptdt_ea_dev.summary, ptdt_asd_dev.prs.summary)
ptdt.scz.dev_model_summaries <- list(ptdt_scz_dev.dnmis.summary, ptdt_scz_dev.dnlof.summary, ptdt_scz_dev.dnall.summary, ptdt_scz_dev.inhlof.summary, ptdt_scz_dev.rvrs.summary, ptdt_scz_dev.ptdt_asd_dev.summary, ptdt_scz_dev.ptdt_ea_dev.summary, ptdt_scz_dev.prs.summary)
ptdt.ea.dev_model_summaries <- list(ptdt_ea_dev.dnmis.summary, ptdt_ea_dev.dnlof.summary, ptdt_ea_dev.dnall.summary, ptdt_ea_dev.inhlof.summary, ptdt_ea_dev.rvrs.summary, ptdt_ea_dev.ptdt_asd_dev.summary, ptdt_ea_dev.ptdt_scz_dev.summary, ptdt_ea_dev.prs.summary)
prs_model_summaries <- list(prs.dnmis.summary, prs.dnlof.summary, prs.dnall.summary, prs.inhlof.summary, prs.rvrs.summary, prs.ptdt_asd_dev.summary, prs.ptdt_scz_dev.summary, prs.ptdt_ea_dev.summary)

model_dict <- ordered_dict(items = NULL, keys = NULL)
model_dict$set("dnMIS", dnmis_model_summaries)
model_dict$set("dnLOF", dnlof_model_summaries)
model_dict$set("dnAll", dnall_model_summaries)
model_dict$set("inhLoF", inhlof_model_summaries)
model_dict$set("RVRS", rvrs_model_summaries)
model_dict$set("pTDT.ASD.Dev", ptdt.asd.dev_model_summaries)
model_dict$set("pTDT.SCZ.Dev", ptdt.scz.dev_model_summaries)
model_dict$set("pTDT.EA.Dev", ptdt.ea.dev_model_summaries)
model_dict$set("PRS", prs_model_summaries)

keys = list(
  "dnMIS", "dnLOF", "dnAll", "inhLoF", "RVRS",
  "pTDT.ASD.Dev", "pTDT.SCZ.Dev", "pTDT.EA.Dev", "PRS"
)

genetic_correlations_table <- data.frame()

for (key in keys) {
  y = key

  model_summary_list = model_dict$get(key)
  for (model_summaries in model_summary_list) {
    # Combined
    model_summary_alt  = model_summaries[[1]]
    model_summary_null = model_summaries[[2]]

    x = rownames(coefficients(model_summary_alt))[2]

    x_estimate = signif(coefficients(model_summary_alt)[2, 1], digits = 2)
    x_stderr = signif(coefficients(model_summary_alt)[2, 2], digits = 2)
    x_pval = signif(coefficients(model_summary_alt)[2, 4], digits = 2)
    r2 = signif(model_summary_alt$r.squared - model_summary_null$r.squared, digits = 2)

    # Correlation
    pearson.cor = model_summaries[[3]]
    pearson.cc = signif(pearson.cor$estimate, digits = 2)

    # Sex interaction
    model_summary_sex_interaction = model_summaries[[4]]
    sex_interaction_model_coeffs = coefficients(model_summary_sex_interaction)
    num_rows = dim(sex_interaction_model_coeffs)[1]
    sex_interaction_coeff = sex_interaction_model_coeffs[num_rows, ]
    sex_interaction_pval = signif(sex_interaction_coeff[[4]], digits = 2)

    # Males
    model_summary_alt_male    = model_summaries[[5]]
    model_summary_null_male   = model_summaries[[6]]
    pearson.cor.male = model_summaries[[7]]
    pearson.cc.male = signif(pearson.cor.male$estimate, digits = 2)

    x_estimate_male = signif(coefficients(model_summary_alt_male)[2, 1], digits = 2)
    x_stderr_male = signif(coefficients(model_summary_alt_male)[2, 2], digits = 2)
    x_pval_male = signif(coefficients(model_summary_alt_male)[2, 4], digits = 2)
    r2_male = signif(model_summary_alt_male$r.squared - model_summary_null_male$r.squared, digits = 2)

    # Females
    model_summary_alt_female  = model_summaries[[8]]
    model_summary_null_female = model_summaries[[9]]
    pearson.cor.female = model_summaries[[10]]
    pearson.cc.female = signif(pearson.cor.female$estimate, digits = 2)

    x_estimate_female = signif(coefficients(model_summary_alt_female)[2, 1], digits = 2)
    x_stderr_female = signif(coefficients(model_summary_alt_female)[2, 2], digits = 2)
    x_pval_female = signif(coefficients(model_summary_alt_female)[2, 4], digits = 2)
    r2_female = signif(model_summary_alt_female$r.squared - model_summary_null_female$r.squared, digits = 2)


    df_tmp = data.frame(y, x, 
                  x_estimate_male, x_pval_male, r2_male, pearson.cc.male,
                  x_estimate_female, x_pval_female, r2_female, pearson.cc.female,
                  x_estimate, x_pval, r2, pearson.cc,
                  sex_interaction_pval)

    genetic_correlations_table = rbind(genetic_correlations_table, df_tmp)

  }
}

row.names(genetic_correlations_table) <- NULL
header <- c("Y", "X", 
            "Male Estimate", "Male P-value", "Male r2", "Male Correlation", 
            "Female Estimate", "Female P-value", "Female r2", "Female Correlation", 
            "Combined Estimate", "Combined P-value", "Combined r2", "Combined Correlation", 
            "Gene by Sex Interaction P-value")
colnames(genetic_correlations_table) <- header

genetic_correlations_table

Y,X,Male Estimate,Male P-value,Male r2,Male Correlation,Female Estimate,Female P-value,Female r2,Female Correlation,Combined Estimate,Combined P-value,Combined r2,Combined Correlation,Gene by Sex Interaction P-value
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
dnMIS,dnLOF,3.4e-02,8.7e-03,1.2e-03,0.0370,0.08900,2.4e-06,8.6e-03,0.11000,0.0530,4.9e-07,3.0e-03,0.06000,0.00560
dnMIS,dnAll,4.9e-01,0.0e+00,5.1e-01,0.7100,0.47000,0.0e+00,5.1e-01,0.72000,0.4800,0.0e+00,5.1e-01,0.72000,0.13000
dnMIS,inhLoF,4.7e-03,2.1e-01,2.7e-04,0.0160,-0.00570,3.4e-01,3.6e-04,-0.01300,0.0017,5.9e-01,3.5e-05,0.00750,0.19000
dnMIS,RVRS,7.8e-02,2.0e-168,1.2e-01,0.3500,0.08400,3.5e-87,1.4e-01,0.39000,0.0800,5.2e-256,1.3e-01,0.36000,0.17000
dnMIS,pTDT.ASD.Dev,-7.8e-03,5.6e-02,6.3e-04,-0.0230,0.00096,8.8e-01,9.0e-06,0.01400,-0.0053,1.2e-01,2.8e-04,-0.01200,0.13000
dnMIS,pTDT.SCZ.Dev,-3.9e-05,9.9e-01,1.4e-08,0.0010,-0.00170,8.0e-01,2.6e-05,-0.00220,-0.0007,8.5e-01,4.5e-06,-0.00028,0.94000
dnMIS,pTDT.EA.Dev,-1.8e-03,6.8e-01,2.9e-05,-0.0029,-0.01100,1.0e-01,1.0e-03,-0.02100,-0.0045,2.3e-01,1.7e-04,-0.00880,0.36000
dnMIS,PRS,-4.7e-03,1.0e-01,4.7e-04,-0.0190,-0.00370,4.2e-01,2.6e-04,-0.00330,-0.0045,6.6e-02,4.1e-04,-0.01400,0.58000
dnLOF,dnMIS,3.5e-02,8.7e-03,1.2e-03,0.0370,0.09900,2.4e-06,8.5e-03,0.11000,0.0570,4.9e-07,3.0e-03,0.06000,0.00400


In [ ]:
# Read data
master_phen <- read.csv("master_phen_4.csv", header = T)
colnames(master_phen)
# get Fig.5 variables
cols <- c("phen","sex","cohort","duo","is_eur","dnLoF","dnMIS","inhLoF","PS.ASD.Z",
          "PS.EA.Z","PS.SZ.Z","rbs.child.z","scq.child.z","srs.child.z",
          "vabs.child.z","dcdq.child.z","PC1","PC2","PC3","PC4","PC5","PC6",
          "PC7","PC8","PC9","PC10")
master_phen2 <- master_phen[,cols]
master_phen2[,"is_eur"] <- as.factor(master_phen2[,"is_eur"])
# pick cases where is_eur = 1   and   duo = "trio"
europ <- which(master_phen2[,"is_eur"]==1)
trio <- which(master_phen2[,"duo"]=="trio")
pick <- intersect(europ,trio)
master_phen_filtered <- master_phen2[pick,]
# 00:44
# > length(which(is.na(master_phen_filtered$rbs.child.z)))
# [1] 5086
# > dim(master_phen_filtered)
# [1] 9354   26
# > dim(na.omit(master_phen_filtered))
# [1] 819  26


length(which(is.na(master_phen_filtered$rbs.child.z)))

dim(master_phen_filtered)

dim(na.omit(master_phen_filtered))

[1] "fid"                           "iid"                          
 [3] "phen"                          "sex"                          
 [5] "cohort"                        "duo"                          
 [7] "family"                        "is_eur"                       
 [9] "dnLoF"                         "dnMIS"                        
[11] "inhLoF"                        "InhLoF.Paternal.Transmitted"  
[13] "InhLoF.Paternal.Untransmitted" "InhLoF.Maternal.Transmitted"  
[15] "InhLoF.Maternal.Untransmitted" "PS.ASD.Z"                     
[17] "Paternal.PS.ASD.Z"             "Maternal.PS.ASD.Z"            
[19] "PS.EA.Z"                       "Paternal.PS.EA.Z"             
[21] "Maternal.PS.EA.Z"              "PS.SZ.Z"                      
[23] "Paternal.PS.SZ.Z"              "Maternal.PS.SZ.Z"             
[25] "PC1"                           "PC2"                          
[27] "PC3"                           "PC4"                          
[29] "PC5"                           "PC6"                          
[31] "PC7"                           "PC8"                          
[33] "PC9"                           "PC10"                         
[35] "PC1_PAT"                       "PC2_PAT"                      
[37] "PC3_PAT"                       "PC4_PAT"                      
[39] "PC5_PAT"                       "PC6_PAT"                      
[41] "PC7_PAT"                       "PC8_PAT"                      
[43] "PC9_PAT"                       "PC10_PAT"                     
[45] "PC1_MAT"                       "PC2_MAT"                      
[47] "PC3_MAT"                       "PC4_MAT"                      
[49] "PC5_MAT"                       "PC6_MAT"                      
[51] "PC7_MAT"                       "PC8_MAT"                      
[53] "PC9_MAT"                       "PC10_MAT"                     
[55] "srs.child.age"                 "vabs.child.age"               
[57] "scq.child.age"                 "dcdq.child.age"               
[59] "mother.EA"                     "father.EA"                    
[61] "srs.mother"                    "srs.father"                   
[63] "mother.bapq"                   "father.bapq"                  
[65] "srs.child.z"                   "vabs.child.z"                 
[67] "rbs.child.z"                   "scq.child.z"                  
[69] "dcdq.child.z"                  "father.age.at.birth.z"        
[71] "mother.age.at.birth.z"         "father.EA.z"                  
[73] "mother.EA.z"                   "father.srs.z"                 
[75] "mother.srs.z"                  "father.bapq.z"                
[77] "mother.bapq.z"

[1] 0

[1]  0 26

[1]  0 26

In [ ]:
# Read data
master_phen <- read.csv("master_phen_4.SBayesR.20210915.csv", header = T)
colnames(master_phen)
# get Fig.5 variables
cols <- c("phen","sex","cohort","duo","is_eur","dnLoF","dnMIS","inhLoF","PS.ASD.Z",
          "PS.EA.Z","PS.SZ.Z","rbs.child.z","scq.child.z","srs.child.z",
          "vabs.child.z","dcdq.child.z","PC1","PC2","PC3","PC4","PC5","PC6",
          "PC7","PC8","PC9","PC10")
master_phen2 <- master_phen[,cols]
master_phen2[,"is_eur"] <- as.factor(master_phen2[,"is_eur"])
# pick cases where is_eur = 1   and   duo = "trio"
europ <- which(master_phen2[,"is_eur"]==1)
trio <- which(master_phen2[,"duo"]=="trio")
pick <- intersect(europ,trio)
master_phen_filtered <- master_phen2[pick,]
# 00:44
# > length(which(is.na(master_phen_filtered$rbs.child.z)))
# [1] 5086
# > dim(master_phen_filtered)
# [1] 9354   26
# > dim(na.omit(master_phen_filtered))
# [1] 819  26


length(which(is.na(master_phen_filtered$rbs.child.z)))

dim(master_phen_filtered)

dim(na.omit(master_phen_filtered))

[1] "fid"                           "iid"                          
 [3] "phen"                          "sex"                          
 [5] "cohort"                        "duo"                          
 [7] "family"                        "is_eur"                       
 [9] "dnLoF"                         "dnMIS"                        
[11] "inhLoF"                        "InhLoF.Paternal.Transmitted"  
[13] "InhLoF.Paternal.Untransmitted" "InhLoF.Maternal.Transmitted"  
[15] "InhLoF.Maternal.Untransmitted" "PC1"                          
[17] "PC2"                           "PC3"                          
[19] "PC4"                           "PC5"                          
[21] "PC6"                           "PC7"                          
[23] "PC8"                           "PC9"                          
[25] "PC10"                          "PC1_PAT"                      
[27] "PC2_PAT"                       "PC3_PAT"                      
[29] "PC4_PAT"                       "PC5_PAT"                      
[31] "PC6_PAT"                       "PC7_PAT"                      
[33] "PC8_PAT"                       "PC9_PAT"                      
[35] "PC10_PAT"                      "PC1_MAT"                      
[37] "PC2_MAT"                       "PC3_MAT"                      
[39] "PC4_MAT"                       "PC5_MAT"                      
[41] "PC6_MAT"                       "PC7_MAT"                      
[43] "PC8_MAT"                       "PC9_MAT"                      
[45] "PC10_MAT"                      "srs.child.age"                
[47] "vabs.child.age"                "scq.child.age"                
[49] "dcdq.child.age"                "mother.EA"                    
[51] "father.EA"                     "srs.mother"                   
[53] "srs.father"                    "mother.bapq"                  
[55] "father.bapq"                   "srs.child.z"                  
[57] "vabs.child.z"                  "rbs.child.z"                  
[59] "scq.child.z"                   "dcdq.child.z"                 
[61] "father.age.at.birth.z"         "mother.age.at.birth.z"        
[63] "father.EA.z"                   "mother.EA.z"                  
[65] "father.srs.z"                  "mother.srs.z"                 
[67] "father.bapq.z"                 "mother.bapq.z"                
[69] "PS.ASD.Z"                      "PS.EA.Z"                      
[71] "PS.SZ.Z"                       "Paternal.PS.ASD.Z"            
[73] "Paternal.PS.EA.Z"              "Paternal.PS.SZ.Z"             
[75] "Maternal.PS.ASD.Z"             "Maternal.PS.EA.Z"             
[77] "Maternal.PS.SZ.Z"

[1] 5086

[1] 9354   26

[1] 819  26

In [ ]:
# Read data
master_phen <- read.csv("master_phen_4.SBayesR.csv", header = T)
colnames(master_phen)
# get Fig.5 variables
cols <- c("phen","sex","cohort","duo","is_eur","dnLoF","dnMIS","inhLoF","PS.ASD.Z",
          "PS.EA.Z","PS.SZ.Z","rbs.child.z","scq.child.z","srs.child.z",
          "vabs.child.z","dcdq.child.z","PC1","PC2","PC3","PC4","PC5","PC6",
          "PC7","PC8","PC9","PC10")
master_phen2 <- master_phen[,cols]
master_phen2[,"is_eur"] <- as.factor(master_phen2[,"is_eur"])
# pick cases where is_eur = 1   and   duo = "trio"
europ <- which(master_phen2[,"is_eur"]==1)
trio <- which(master_phen2[,"duo"]=="trio")
pick <- intersect(europ,trio)
master_phen_filtered <- master_phen2[pick,]
# 00:44
# > length(which(is.na(master_phen_filtered$rbs.child.z)))
# [1] 5086
# > dim(master_phen_filtered)
# [1] 9354   26
# > dim(na.omit(master_phen_filtered))
# [1] 819  26


length(which(is.na(master_phen_filtered$rbs.child.z)))

dim(master_phen_filtered)

dim(na.omit(master_phen_filtered))

[1] "fid"                           "iid"                          
 [3] "phen"                          "sex"                          
 [5] "cohort"                        "duo"                          
 [7] "family"                        "is_eur"                       
 [9] "dnLoF"                         "dnMIS"                        
[11] "inhLoF"                        "InhLoF.Paternal.Transmitted"  
[13] "InhLoF.Paternal.Untransmitted" "InhLoF.Maternal.Transmitted"  
[15] "InhLoF.Maternal.Untransmitted" "PC1"                          
[17] "PC2"                           "PC3"                          
[19] "PC4"                           "PC5"                          
[21] "PC6"                           "PC7"                          
[23] "PC8"                           "PC9"                          
[25] "PC10"                          "PC1_PAT"                      
[27] "PC2_PAT"                       "PC3_PAT"                      
[29] "PC4_PAT"                       "PC5_PAT"                      
[31] "PC6_PAT"                       "PC7_PAT"                      
[33] "PC8_PAT"                       "PC9_PAT"                      
[35] "PC10_PAT"                      "PC1_MAT"                      
[37] "PC2_MAT"                       "PC3_MAT"                      
[39] "PC4_MAT"                       "PC5_MAT"                      
[41] "PC6_MAT"                       "PC7_MAT"                      
[43] "PC8_MAT"                       "PC9_MAT"                      
[45] "PC10_MAT"                      "srs.child.age"                
[47] "vabs.child.age"                "scq.child.age"                
[49] "dcdq.child.age"                "mother.EA"                    
[51] "father.EA"                     "srs.mother"                   
[53] "srs.father"                    "mother.bapq"                  
[55] "father.bapq"                   "srs.child.z"                  
[57] "vabs.child.z"                  "rbs.child.z"                  
[59] "scq.child.z"                   "dcdq.child.z"                 
[61] "father.age.at.birth.z"         "mother.age.at.birth.z"        
[63] "father.EA.z"                   "mother.EA.z"                  
[65] "father.srs.z"                  "mother.srs.z"                 
[67] "father.bapq.z"                 "mother.bapq.z"                
[69] "PS.ASD.Z"                      "PS.EA.Z"                      
[71] "PS.SZ.Z"                       "Paternal.PS.ASD.Z"            
[73] "Paternal.PS.EA.Z"              "Paternal.PS.SZ.Z"             
[75] "Maternal.PS.ASD.Z"             "Maternal.PS.EA.Z"             
[77] "Maternal.PS.SZ.Z"

[1] 5086

[1] 9354   26

[1] 819  26

In [ ]:
#@title Misc

# Sys.info()
# sysname'Linux'release'5.4.104+'version'#1 SMP Sat Jun 5 09:50:34 PDT 2021'nodename'1cb9b7bc95e9'machine'x86_64'login'unknown'user'root'effective_user'root'

# sessionInfo()
# R version 4.1.1 (2021-08-10)
# Platform: x86_64-pc-linux-gnu (64-bit)
# Running under: Ubuntu 18.04.5 LTS

# Matrix products: default
# BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
# LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

# locale:
#  [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
#  [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
#  [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
#  [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
#  [9] LC_ADDRESS=C               LC_TELEPHONE=C            
# [11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

# attached base packages:
# [1] stats     graphics  grDevices utils     datasets  methods   base     

# loaded via a namespace (and not attached):
#  [1] fansi_0.5.0       digest_0.6.27     utf8_1.2.2        crayon_1.4.1     
#  [5] IRdisplay_1.0     repr_1.1.3        lifecycle_1.0.0   jsonlite_1.7.2   
#  [9] evaluate_0.14     pillar_1.6.2      rlang_0.4.11      uuid_0.1-4       
# [13] vctrs_0.3.8       ellipsis_0.3.2    IRkernel_1.2      tools_4.1.1      
# [17] compiler_4.1.1    base64enc_0.1-3   pbdZMQ_0.3-5      htmltools_0.5.1.1


# print(installed.packages())
#               Package         LibPath                         Version  
# IRdisplay     "IRdisplay"     "/usr/local/lib/R/site-library" "1.0"    
# IRkernel      "IRkernel"      "/usr/local/lib/R/site-library" "1.2"    
# pbdZMQ        "pbdZMQ"        "/usr/local/lib/R/site-library" "0.3-5"  
# repr          "repr"          "/usr/local/lib/R/site-library" "1.1.3"  
# askpass       "askpass"       "/usr/lib/R/site-library"       "1.1"    
# assertthat    "assertthat"    "/usr/lib/R/site-library"       "0.2.1"  
# backports     "backports"     "/usr/lib/R/site-library"       "1.2.1"  
# base64enc     "base64enc"     "/usr/lib/R/site-library"       "0.1-3"  
# bit           "bit"           "/usr/lib/R/site-library"       "4.0.4"  
# bit64         "bit64"         "/usr/lib/R/site-library"       "4.0.5"  
# blob          "blob"          "/usr/lib/R/site-library"       "1.2.2"  
# brew          "brew"          "/usr/lib/R/site-library"       "1.0-6"  
# brio          "brio"          "/usr/lib/R/site-library"       "1.1.2"  
# broom         "broom"         "/usr/lib/R/site-library"       "0.7.9"  
# cachem        "cachem"        "/usr/lib/R/site-library"       "1.0.6"  
# callr         "callr"         "/usr/lib/R/site-library"       "3.7.0"  
# cellranger    "cellranger"    "/usr/lib/R/site-library"       "1.1.0"  
# cli           "cli"           "/usr/lib/R/site-library"       "3.0.1"  
# clipr         "clipr"         "/usr/lib/R/site-library"       "0.7.1"  
# colorspace    "colorspace"    "/usr/lib/R/site-library"       "2.0-2"  
# commonmark    "commonmark"    "/usr/lib/R/site-library"       "1.7"    
# cpp11         "cpp11"         "/usr/lib/R/site-library"       "0.3.1"  
# crayon        "crayon"        "/usr/lib/R/site-library"       "1.4.1"  
# credentials   "credentials"   "/usr/lib/R/site-library"       "1.3.1"  
# curl          "curl"          "/usr/lib/R/site-library"       "4.3.2"  
# data.table    "data.table"    "/usr/lib/R/site-library"       "1.14.0" 
# DBI           "DBI"           "/usr/lib/R/site-library"       "1.1.1"  
# dbplyr        "dbplyr"        "/usr/lib/R/site-library"       "2.1.1"  
# desc          "desc"          "/usr/lib/R/site-library"       "1.3.0"  
# devtools      "devtools"      "/usr/lib/R/site-library"       "2.4.2"  
# diffobj       "diffobj"       "/usr/lib/R/site-library"       "0.3.4"  
# digest        "digest"        "/usr/lib/R/site-library"       "0.6.27" 
# dplyr         "dplyr"         "/usr/lib/R/site-library"       "1.0.7"  
# dtplyr        "dtplyr"        "/usr/lib/R/site-library"       "1.1.0"  
# ellipsis      "ellipsis"      "/usr/lib/R/site-library"       "0.3.2"  
# evaluate      "evaluate"      "/usr/lib/R/site-library"       "0.14"   
# fansi         "fansi"         "/usr/lib/R/site-library"       "0.5.0"  
# farver        "farver"        "/usr/lib/R/site-library"       "2.1.0"  
# fastmap       "fastmap"       "/usr/lib/R/site-library"       "1.1.0"  
# forcats       "forcats"       "/usr/lib/R/site-library"       "0.5.1"  
# fs            "fs"            "/usr/lib/R/site-library"       "1.5.0"  
# gargle        "gargle"        "/usr/lib/R/site-library"       "1.2.0"  
# generics      "generics"      "/usr/lib/R/site-library"       "0.1.0"  
# gert          "gert"          "/usr/lib/R/site-library"       "1.3.2"  
# ggplot2       "ggplot2"       "/usr/lib/R/site-library"       "3.3.5"  
# gh            "gh"            "/usr/lib/R/site-library"       "1.3.0"  
# gitcreds      "gitcreds"      "/usr/lib/R/site-library"       "0.1.1"  
# glue          "glue"          "/usr/lib/R/site-library"       "1.4.2"  
# googledrive   "googledrive"   "/usr/lib/R/site-library"       "2.0.0"  
# googlesheets4 "googlesheets4" "/usr/lib/R/site-library"       "1.0.0"  
# gtable        "gtable"        "/usr/lib/R/site-library"       "0.3.0"  
# haven         "haven"         "/usr/lib/R/site-library"       "2.4.3"  
# highr         "highr"         "/usr/lib/R/site-library"       "0.9"    
# hms           "hms"           "/usr/lib/R/site-library"       "1.1.0"  
# htmltools     "htmltools"     "/usr/lib/R/site-library"       "0.5.1.1"
# httr          "httr"          "/usr/lib/R/site-library"       "1.4.2"  
# ids           "ids"           "/usr/lib/R/site-library"       "1.0.1"  
# ini           "ini"           "/usr/lib/R/site-library"       "0.3.1"  
# isoband       "isoband"       "/usr/lib/R/site-library"       "0.2.5"  
# jsonlite      "jsonlite"      "/usr/lib/R/site-library"       "1.7.2"  
# knitr         "knitr"         "/usr/lib/R/site-library"       "1.33"   
# labeling      "labeling"      "/usr/lib/R/site-library"       "0.4.2"  
# lifecycle     "lifecycle"     "/usr/lib/R/site-library"       "1.0.0"  
# lubridate     "lubridate"     "/usr/lib/R/site-library"       "1.7.10" 
# magrittr      "magrittr"      "/usr/lib/R/site-library"       "2.0.1"  
# markdown      "markdown"      "/usr/lib/R/site-library"       "1.1"    
# memoise       "memoise"       "/usr/lib/R/site-library"       "2.0.0"  
# mime          "mime"          "/usr/lib/R/site-library"       "0.11"   
# modelr        "modelr"        "/usr/lib/R/site-library"       "0.1.8"  
# munsell       "munsell"       "/usr/lib/R/site-library"       "0.5.0"  
# openssl       "openssl"       "/usr/lib/R/site-library"       "1.4.3"  
# pillar        "pillar"        "/usr/lib/R/site-library"       "1.6.2"  
# pkgbuild      "pkgbuild"      "/usr/lib/R/site-library"       "1.2.0"  
# pkgconfig     "pkgconfig"     "/usr/lib/R/site-library"       "2.0.3"  
# pkgload       "pkgload"       "/usr/lib/R/site-library"       "1.2.1"  
# praise        "praise"        "/usr/lib/R/site-library"       "1.0.0"  
# prettyunits   "prettyunits"   "/usr/lib/R/site-library"       "1.1.1"  
# processx      "processx"      "/usr/lib/R/site-library"       "3.5.2"  
# progress      "progress"      "/usr/lib/R/site-library"       "1.2.2"  
# ps            "ps"            "/usr/lib/R/site-library"       "1.6.0"  
# purrr         "purrr"         "/usr/lib/R/site-library"       "0.3.4"  
# R6            "R6"            "/usr/lib/R/site-library"       "2.5.1"  
# rappdirs      "rappdirs"      "/usr/lib/R/site-library"       "0.3.3"  
# rcmdcheck     "rcmdcheck"     "/usr/lib/R/site-library"       "1.3.3"  
# RColorBrewer  "RColorBrewer"  "/usr/lib/R/site-library"       "1.1-2"  
# Rcpp          "Rcpp"          "/usr/lib/R/site-library"       "1.0.7"  
# readr         "readr"         "/usr/lib/R/site-library"       "2.0.1"  
# readxl        "readxl"        "/usr/lib/R/site-library"       "1.3.1"  
# rematch       "rematch"       "/usr/lib/R/site-library"       "1.0.1"  
# rematch2      "rematch2"      "/usr/lib/R/site-library"       "2.1.2"  
# remotes       "remotes"       "/usr/lib/R/site-library"       "2.4.0"  
# reprex        "reprex"        "/usr/lib/R/site-library"       "2.0.1"  
# rlang         "rlang"         "/usr/lib/R/site-library"       "0.4.11" 
# rmarkdown     "rmarkdown"     "/usr/lib/R/site-library"       "2.10"   
# roxygen2      "roxygen2"      "/usr/lib/R/site-library"       "7.1.1"  
# rprojroot     "rprojroot"     "/usr/lib/R/site-library"       "2.0.2"  
# rstudioapi    "rstudioapi"    "/usr/lib/R/site-library"       "0.13"   
# rversions     "rversions"     "/usr/lib/R/site-library"       "2.1.1"  
# rvest         "rvest"         "/usr/lib/R/site-library"       "1.0.1"  
# scales        "scales"        "/usr/lib/R/site-library"       "1.1.1"  
# selectr       "selectr"       "/usr/lib/R/site-library"       "0.4-2"  
# sessioninfo   "sessioninfo"   "/usr/lib/R/site-library"       "1.1.1"  
# stringi       "stringi"       "/usr/lib/R/site-library"       "1.7.3"  
# stringr       "stringr"       "/usr/lib/R/site-library"       "1.4.0"  
# svglite       "svglite"       "/usr/lib/R/site-library"       "2.0.0"  
# sys           "sys"           "/usr/lib/R/site-library"       "3.4"    
# systemfonts   "systemfonts"   "/usr/lib/R/site-library"       "1.0.2"  
# testthat      "testthat"      "/usr/lib/R/site-library"       "3.0.4"  
# tibble        "tibble"        "/usr/lib/R/site-library"       "3.1.3"  
# tidyr         "tidyr"         "/usr/lib/R/site-library"       "1.1.3"  
# tidyselect    "tidyselect"    "/usr/lib/R/site-library"       "1.1.1"  
# tidyverse     "tidyverse"     "/usr/lib/R/site-library"       "1.3.1"  
# tinytex       "tinytex"       "/usr/lib/R/site-library"       "0.33"   
# tzdb          "tzdb"          "/usr/lib/R/site-library"       "0.1.2"  
# usethis       "usethis"       "/usr/lib/R/site-library"       "2.0.1"  
# utf8          "utf8"          "/usr/lib/R/site-library"       "1.2.2"  
# uuid          "uuid"          "/usr/lib/R/site-library"       "0.1-4"  
# vctrs         "vctrs"         "/usr/lib/R/site-library"       "0.3.8"  
# viridisLite   "viridisLite"   "/usr/lib/R/site-library"       "0.4.0"  
# vroom         "vroom"         "/usr/lib/R/site-library"       "1.5.4"  
# waldo         "waldo"         "/usr/lib/R/site-library"       "0.3.0"  
# whisker       "whisker"       "/usr/lib/R/site-library"       "0.4"    
# withr         "withr"         "/usr/lib/R/site-library"       "2.4.2"  
# xfun          "xfun"          "/usr/lib/R/site-library"       "0.25"   
# xml2          "xml2"          "/usr/lib/R/site-library"       "1.3.2"  
# xopen         "xopen"         "/usr/lib/R/site-library"       "1.0.0"  
# yaml          "yaml"          "/usr/lib/R/site-library"       "2.2.1"  
# zip           "zip"           "/usr/lib/R/site-library"       "2.2.0"  
# base          "base"          "/usr/lib/R/library"            "4.1.1"  
# boot          "boot"          "/usr/lib/R/library"            "1.3-28" 
# class         "class"         "/usr/lib/R/library"            "7.3-19" 
# cluster       "cluster"       "/usr/lib/R/library"            "2.1.2"  
# codetools     "codetools"     "/usr/lib/R/library"            "0.2-18" 
# compiler      "compiler"      "/usr/lib/R/library"            "4.1.1"  
# datasets      "datasets"      "/usr/lib/R/library"            "4.1.1"  
# foreign       "foreign"       "/usr/lib/R/library"            "0.8-81" 
# graphics      "graphics"      "/usr/lib/R/library"            "4.1.1"  
# grDevices     "grDevices"     "/usr/lib/R/library"            "4.1.1"  
# grid          "grid"          "/usr/lib/R/library"            "4.1.1"  
# KernSmooth    "KernSmooth"    "/usr/lib/R/library"            "2.23-20"
# lattice       "lattice"       "/usr/lib/R/library"            "0.20-44"
# MASS          "MASS"          "/usr/lib/R/library"            "7.3-54" 
# Matrix        "Matrix"        "/usr/lib/R/library"            "1.3-4"  
# methods       "methods"       "/usr/lib/R/library"            "4.1.1"  
# mgcv          "mgcv"          "/usr/lib/R/library"            "1.8-36" 
# nlme          "nlme"          "/usr/lib/R/library"            "3.1-152"
# nnet          "nnet"          "/usr/lib/R/library"            "7.3-16" 
# parallel      "parallel"      "/usr/lib/R/library"            "4.1.1"  
# rpart         "rpart"         "/usr/lib/R/library"            "4.1-15" 
# spatial       "spatial"       "/usr/lib/R/library"            "7.3-14" 
# splines       "splines"       "/usr/lib/R/library"            "4.1.1"  
# stats         "stats"         "/usr/lib/R/library"            "4.1.1"  
# stats4        "stats4"        "/usr/lib/R/library"            "4.1.1"  
# survival      "survival"      "/usr/lib/R/library"            "3.2-13" 
# tcltk         "tcltk"         "/usr/lib/R/library"            "4.1.1"  
# tools         "tools"         "/usr/lib/R/library"            "4.1.1"  
# utils         "utils"         "/usr/lib/R/library"            "4.1.1"  
#               Priority     
# IRdisplay     NA           
# IRkernel      NA           
# pbdZMQ        NA           
# repr          NA           
# askpass       NA           
# assertthat    NA           
# backports     NA           
# base64enc     NA           
# bit           NA           
# bit64         NA           
# blob          NA           
# brew          NA           
# brio          NA           
# broom         NA           
# cachem        NA           
# callr         NA           
# cellranger    NA           
# cli           NA           
# clipr         NA           
# colorspace    NA           
# commonmark    NA           
# cpp11         NA           
# crayon        NA           
# credentials   NA           
# curl          NA           
# data.table    NA           
# DBI           NA           
# dbplyr        NA           
# desc          NA           
# devtools      NA           
# diffobj       NA           
# digest        NA           
# dplyr         NA           
# dtplyr        NA           
# ellipsis      NA           
# evaluate      NA           
# fansi         NA           
# farver        NA           
# fastmap       NA           
# forcats       NA           
# fs            NA           
# gargle        NA           
# generics      NA           
# gert          NA           
# ggplot2       NA           
# gh            NA           
# gitcreds      NA           
# glue          NA           
# googledrive   NA           
# googlesheets4 NA           
# gtable        NA           
# haven         NA           
# highr         NA           
# hms           NA           
# htmltools     NA           
# httr          NA           
# ids           NA           
# ini           NA           
# isoband       NA           
# jsonlite      NA           
# knitr         NA           
# labeling      NA           
# lifecycle     NA           
# lubridate     NA           
# magrittr      NA           
# markdown      NA           
# memoise       NA           
# mime          NA           
# modelr        NA           
# munsell       NA           
# openssl       NA           
# pillar        NA           
# pkgbuild      NA           
# pkgconfig     NA           
# pkgload       NA           
# praise        NA           
# prettyunits   NA           
# processx      NA           
# progress      NA           
# ps            NA           
# purrr         NA           
# R6            NA           
# rappdirs      NA           
# rcmdcheck     NA           
# RColorBrewer  NA           
# Rcpp          NA           
# readr         NA           
# readxl        NA           
# rematch       NA           
# rematch2      NA           
# remotes       NA           
# reprex        NA           
# rlang         NA           
# rmarkdown     NA           
# roxygen2      NA           
# rprojroot     NA           
# rstudioapi    NA           
# rversions     NA           
# rvest         NA           
# scales        NA           
# selectr       NA           
# sessioninfo   NA           
# stringi       NA           
# stringr       NA           
# svglite       NA           
# sys           NA           
# systemfonts   NA           
# testthat      NA           
# tibble        NA           
# tidyr         NA           
# tidyselect    NA           
# tidyverse     NA           
# tinytex       NA           
# tzdb          NA           
# usethis       NA           
# utf8          NA           
# uuid          NA           
# vctrs         NA           
# viridisLite   NA           
# vroom         NA           
# waldo         NA           
# whisker       NA           
# withr         NA           
# xfun          NA           
# xml2          NA           
# xopen         NA           
# yaml          NA           
# zip           NA           
# base          "base"       
# boot          "recommended"
# class         "recommended"
# cluster       "recommended"
# codetools     "recommended"
# compiler      "base"       
# datasets      "base"       
# foreign       "recommended"
# graphics      "base"       
# grDevices     "base"       
# grid          "base"       
# KernSmooth    "recommended"
# lattice       "recommended"
# MASS          "recommended"
# Matrix        "recommended"
# methods       "base"       
# mgcv          "recommended"
# nlme          "recommended"
# nnet          "recommended"
# parallel      "base"       
# rpart         "recommended"
# spatial       "recommended"
# splines       "base"       
# stats         "base"       
# stats4        "base"       
# survival      "recommended"
# tcltk         "base"       
# tools         "base"       
# utils         "base"       
#               Depends                                              
# IRdisplay     "R (>= 3.0.1)"                                       
# IRkernel      "R (>= 3.2.0)"                                       
# pbdZMQ        "R (>= 3.5.0)"                                       
# repr          "R (>= 3.0.1)"                                       
# askpass       NA                                                   
# assertthat    NA                                                   
# backports     "R (>= 3.0.0)"                                       
# base64enc     "R (>= 2.9.0)"                                       
# bit           "R (>= 2.9.2)"                                       
# bit64         "R (>= 3.0.1), bit (>= 4.0.0), utils, methods, stats"
# blob          NA                                                   
# brew          NA                                                   
# brio          NA                                                   
# broom         "R (>= 3.1)"                                         
# cachem        NA                                                   
# callr         NA                                                   
# cellranger    "R (>= 3.0.0)"                                       
# cli           "R (>= 2.10)"                                        
# clipr         NA                                                   
# colorspace    "R (>= 3.0.0), methods"                              
# commonmark    NA                                                   
# cpp11         NA                                                   
# crayon        NA                                                   
# credentials   NA                                                   
# curl          "R (>= 3.0.0)"                                       
# data.table    "R (>= 3.1.0)"                                       
# DBI           "methods, R (>= 3.0.0)"                              
# dbplyr        "R (>= 3.1)"                                         
# desc          "R (>= 3.1.0)"                                       
# devtools      "R (>= 3.0.2), usethis (>= 2.0.1)"                   
# diffobj       "R (>= 3.1.0)"                                       
# digest        "R (>= 3.3.0)"                                       
# dplyr         "R (>= 3.3.0)"                                       
# dtplyr        "R (>= 3.3)"                                         
# ellipsis      "R (>= 3.2)"                                         
# evaluate      "R (>= 3.0.2)"                                       
# fansi         "R (>= 3.1.0)"                                       
# farver        NA                                                   
# fastmap       NA                                                   
# forcats       "R (>= 3.2)"                                         
# fs            "R (>= 3.1)"                                         
# gargle        "R (>= 3.3)"                                         
# generics      "R (>= 3.1)"                                         
# gert          NA                                                   
# ggplot2       "R (>= 3.3)"                                         
# gh            NA                                                   
# gitcreds      NA                                                   
# glue          "R (>= 3.2)"                                         
# googledrive   "R (>= 3.3)"                                         
# googlesheets4 "R (>= 3.3)"                                         
# gtable        "R (>= 3.0)"                                         
# haven         "R (>= 3.2)"                                         
# highr         "R (>= 3.2.3)"                                       
# hms           NA                                                   
# htmltools     "R (>= 2.14.1)"                                      
# httr          "R (>= 3.2)"                                         
# ids           NA                                                   
# ini           NA                                                   
# isoband       NA                                                   
# jsonlite      "methods"                                            
# knitr         "R (>= 3.2.3)"                                       
# labeling      NA                                                   
# lifecycle     "R (>= 3.3)"                                         
# lubridate     "methods, R (>= 3.2)"                                
# magrittr      NA                                                   
# markdown      "R (>= 2.11.1)"                                      
# memoise       NA                                                   
# mime          NA                                                   
# modelr        "R (>= 3.2)"                                         
# munsell       NA                                                   
# openssl       NA                                                   
# pillar        NA                                                   
# pkgbuild      "R (>= 3.1)"                                         
# pkgconfig     NA                                                   
# pkgload       NA                                                   
# praise        NA                                                   
# prettyunits   NA                                                   
# processx      NA                                                   
# progress      NA                                                   
# ps            "R (>= 3.1)"                                         
# purrr         "R (>= 3.2)"                                         
# R6            "R (>= 3.0)"                                         
# rappdirs      "R (>= 3.2)"                                         
# rcmdcheck     NA                                                   
# RColorBrewer  "R (>= 2.0.0)"                                       
# Rcpp          NA                                                   
# readr         "R (>= 3.1)"                                         
# readxl        NA                                                   
# rematch       NA                                                   
# rematch2      NA                                                   
# remotes       "R (>= 3.0.0)"                                       
# reprex        "R (>= 3.3)"                                         
# rlang         "R (>= 3.3.0)"                                       
# rmarkdown     "R (>= 3.0)"                                         
# roxygen2      "R (>= 3.2)"                                         
# rprojroot     "R (>= 3.0.0)"                                       
# rstudioapi    NA                                                   
# rversions     NA                                                   
# rvest         "R (>= 3.2)"                                         
# scales        "R (>= 3.2)"                                         
# selectr       "R (>= 3.0)"                                         
# sessioninfo   NA                                                   
# stringi       "R (>= 3.1)"                                         
# stringr       "R (>= 3.1)"                                         
# svglite       "R (>= 3.0.0)"                                       
# sys           NA                                                   
# systemfonts   "R (>= 3.2.0)"                                       
# testthat      "R (>= 3.1)"                                         
# tibble        "R (>= 3.1.0)"                                       
# tidyr         "R (>= 3.1)"                                         
# tidyselect    "R (>= 3.2)"                                         
# tidyverse     "R (>= 3.3)"                                         
# tinytex       NA                                                   
# tzdb          "R (>= 3.3)"                                         
# usethis       "R (>= 3.2)"                                         
# utf8          "R (>= 2.10)"                                        
# uuid          "R (>= 2.9.0)"                                       
# vctrs         "R (>= 3.3)"                                         
# viridisLite   "R (>= 2.10)"                                        
# vroom         "R (>= 3.1)"                                         
# waldo         NA                                                   
# whisker       NA                                                   
# withr         "R (>= 3.2.0)"                                       
# xfun          NA                                                   
# xml2          "R (>= 3.1.0)"                                       
# xopen         "R (>= 3.1)"                                         
# yaml          NA                                                   
# zip           NA                                                   
# base          NA                                                   
# boot          "R (>= 3.0.0), graphics, stats"                      
# class         "R (>= 3.0.0), stats, utils"                         
# cluster       "R (>= 3.4.0)"                                       
# codetools     "R (>= 2.1)"                                         
# compiler      NA                                                   
# datasets      NA                                                   
# foreign       "R (>= 4.0.0)"                                       
# graphics      NA                                                   
# grDevices     NA                                                   
# grid          NA                                                   
# KernSmooth    "R (>= 2.5.0), stats"                                
# lattice       "R (>= 3.0.0)"                                       
# MASS          "R (>= 3.3.0), grDevices, graphics, stats, utils"    
# Matrix        "R (>= 3.5.0)"                                       
# methods       NA                                                   
# mgcv          "R (>= 3.6.0), nlme (>= 3.1-64)"                     
# nlme          "R (>= 3.4.0)"                                       
# nnet          "R (>= 3.0.0), stats, utils"                         
# parallel      NA                                                   
# rpart         "R (>= 2.15.0), graphics, stats, grDevices"          
# spatial       "R (>= 3.0.0), graphics, stats, utils"               
# splines       NA                                                   
# stats         NA                                                   
# stats4        NA                                                   
# survival      "R (>= 3.5.0)"                                       
# tcltk         NA                                                   
# tools         NA                                                   
# utils         NA                                                   
#               Imports                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# IRdisplay     "methods, repr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# IRkernel      "repr (>= 0.4.99), methods, evaluate (>= 0.10), IRdisplay (>=\n0.3.0.9999), pbdZMQ (>= 0.2-1), crayon, jsonlite (>= 0.9.6),\nuuid, digest"                                                                                                                                                                                                                                                                                                                                                                                                                                               
# pbdZMQ        NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# repr          "utils, grDevices, htmltools, jsonlite, pillar (>= 1.4.0),\nbase64enc"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# askpass       "sys (>= 2.1)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# assertthat    "tools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# backports     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# base64enc     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# bit           NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# bit64         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# blob          "methods, rlang, vctrs (>= 0.2.1)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# brew          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# brio          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# broom         "backports, dplyr (>= 1.0.0), ellipsis, generics (>= 0.0.2),\nglue, methods, purrr, rlang, stringr, tibble (>= 3.0.0), tidyr\n(>= 1.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                
# cachem        "rlang, fastmap"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# callr         "processx (>= 3.5.0), R6, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# cellranger    "rematch, tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# cli           "glue, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# clipr         "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# colorspace    "graphics, grDevices, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# commonmark    NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# cpp11         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# crayon        "grDevices, methods, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# credentials   "openssl (>= 1.3), sys (>= 2.1), curl, jsonlite, askpass"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# curl          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# data.table    "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# DBI           NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# dbplyr        "assertthat (>= 0.2.0), blob (>= 1.2.0), DBI (>= 1.0.0), dplyr\n(>= 1.0.4), ellipsis, glue (>= 1.2.0), lifecycle (>= 1.0.0),\nmagrittr, methods, purrr (>= 0.2.5), R6 (>= 2.2.2), rlang (>=\n0.2.0), tibble (>= 1.4.2), tidyselect (>= 0.2.4), utils, vctrs,\nwithr"                                                                                                                                                                                                                                                                                                                     
# desc          "utils, R6, crayon, rprojroot"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# devtools      "callr (>= 3.6.0), cli (>= 2.4.0), desc (>= 1.3.0), ellipsis\n(>= 0.3.1), fs (>= 1.5.0), httr (>= 1.4.2), lifecycle (>=\n1.0.0), memoise (>= 2.0.0), pkgbuild (>= 1.2.0), pkgload (>=\n1.2.1), rcmdcheck (>= 1.3.3), remotes (>= 2.3.0), rlang (>=\n0.4.10), roxygen2 (>= 7.1.1), rstudioapi (>= 0.13), rversions\n(>= 2.0.2), sessioninfo (>= 1.1.1), stats, testthat (>= 3.0.2),\ntools, utils, withr (>= 2.4.1)"                                                                                                                                                                      
# diffobj       "crayon (>= 1.3.2), tools, methods, utils, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# digest        "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# dplyr         "ellipsis, generics, glue (>= 1.3.2), lifecycle (>= 1.0.0),\nmagrittr (>= 1.5), methods, R6, rlang (>= 0.4.10), tibble (>=\n2.1.3), tidyselect (>= 1.1.0), utils, vctrs (>= 0.3.5), pillar\n(>= 1.5.1)"                                                                                                                                                                                                                                                                                                                                                                                  
# dtplyr        "crayon, data.table (>= 1.12.4), dplyr (>= 1.0.3), ellipsis,\nglue, lifecycle, rlang, tibble, tidyselect, vctrs"                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# ellipsis      "rlang (>= 0.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# evaluate      "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# fansi         "grDevices, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# farver        NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# fastmap       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# forcats       "ellipsis, magrittr, rlang, tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# fs            "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# gargle        "cli (>= 3.0.0), fs (>= 1.3.1), glue (>= 1.3.0), httr (>=\n1.4.0), jsonlite, rappdirs, rlang (>= 0.4.9), rstudioapi,\nstats, utils, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                               
# generics      "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# gert          "askpass, credentials (>= 1.2.1), openssl (>= 1.4.1),\nrstudioapi (>= 0.11), sys, zip (>= 2.1.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# ggplot2       "digest, glue, grDevices, grid, gtable (>= 0.1.1), isoband,\nMASS, mgcv, rlang (>= 0.4.10), scales (>= 0.5.0), stats,\ntibble, withr (>= 2.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                         
# gh            "cli (>= 2.0.1), gitcreds, httr (>= 1.2), ini, jsonlite"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# gitcreds      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# glue          "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# googledrive   "cli (>= 3.0.0), gargle (>= 1.2.0), glue (>= 1.4.2), httr,\njsonlite, lifecycle, magrittr, pillar, purrr (>= 0.2.3), rlang\n(>= 0.4.9), tibble (>= 2.0.0), utils, uuid, vctrs (>= 0.3.0),\nwithr"                                                                                                                                                                                                                                                                                                                                                                                        
# googlesheets4 "cellranger, cli (>= 3.0.0), curl, gargle (>= 1.2.0), glue (>=\n1.3.0), googledrive (>= 2.0.0), httr, ids, magrittr, methods,\npurrr, rematch2, rlang (>= 0.4.11), tibble (>= 2.1.1), utils,\nvctrs (>= 0.2.3)"                                                                                                                                                                                                                                                                                                                                                                          
# gtable        "grid"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# haven         "forcats (>= 0.2.0), hms, methods, readr (>= 0.1.0), rlang (>=\n0.4.0), tibble, tidyselect, vctrs (>= 0.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# highr         "xfun (>= 0.18)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# hms           "ellipsis (>= 0.3.2), lifecycle, methods, pkgconfig, rlang,\nvctrs (>= 0.3.8)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# htmltools     "utils, digest, grDevices, base64enc, rlang"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# httr          "curl (>= 3.0.0), jsonlite, mime, openssl (>= 0.8), R6"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# ids           "openssl, uuid"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# ini           NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# isoband       "grid, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# jsonlite      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# knitr         "evaluate (>= 0.10), highr, methods, markdown, stringr (>=\n0.6), yaml (>= 2.1.19), xfun (>= 0.21), tools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# labeling      "stats, graphics"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# lifecycle     "glue, rlang (>= 0.4.10)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# lubridate     "generics, Rcpp (>= 0.12.13)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# magrittr      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# markdown      "utils, xfun, mime (>= 0.3)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# memoise       "rlang (>= 0.4.10), cachem"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# mime          "tools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# modelr        "broom, magrittr, purrr (>= 0.2.2), rlang (>= 0.2.0), tibble,\ntidyr (>= 0.8.0), tidyselect, vctrs"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# munsell       "colorspace, methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# openssl       "askpass"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# pillar        "cli, crayon (>= 1.3.4), ellipsis (>= 0.3.2), fansi, lifecycle,\nrlang (>= 0.3.0), utf8 (>= 1.1.0), utils, vctrs (>= 0.3.8)"                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# pkgbuild      "callr (>= 3.2.0), cli, crayon, desc, prettyunits, R6,\nrprojroot, withr (>= 2.1.2)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# pkgconfig     "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# pkgload       "cli, crayon, desc, methods, rlang, rprojroot, rstudioapi,\nutils, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# praise        NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# prettyunits   NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# processx      "ps (>= 1.2.0), R6, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# progress      "hms, prettyunits, R6, crayon"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# ps            "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# purrr         "magrittr (>= 1.5), rlang (>= 0.3.1)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# R6            NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# rappdirs      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# rcmdcheck     "callr (>= 3.1.1.9000), cli (>= 1.1.0), crayon, desc (>=\n1.2.0), digest, pkgbuild, prettyunits, R6, rprojroot,\nsessioninfo (>= 1.1.1), utils, withr, xopen"                                                                                                                                                                                                                                                                                                                                                                                                                            
# RColorBrewer  NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# Rcpp          "methods, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# readr         "cli, clipr, crayon, hms (>= 0.4.1), methods, rlang, R6,\ntibble, vroom (>= 1.5.4), utils, lifecycle (>= 0.2.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# readxl        "cellranger, Rcpp (>= 0.12.18), tibble (>= 1.3.1), utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# rematch       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# rematch2      "tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# remotes       "methods, stats, tools, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# reprex        "callr (>= 3.6.0), cli (>= 2.3.1), clipr (>= 0.4.0), fs, glue,\nknitr (>= 1.23), rlang (>= 0.4.0), rmarkdown, rstudioapi,\nutils, withr (>= 2.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                      
# rlang         "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# rmarkdown     "tools, utils, knitr (>= 1.22), yaml (>= 2.1.19), htmltools (>=\n0.3.5), evaluate (>= 0.13), jsonlite, tinytex (>= 0.31), xfun\n(>= 0.21), methods, stringr (>= 1.2.0)"                                                                                                                                                                                                                                                                                                                                                                                                                  
# roxygen2      "brew, commonmark, desc (>= 1.2.0), digest, knitr, methods,\npkgload (>= 1.0.2), purrr (>= 0.3.3), R6 (>= 2.1.2), Rcpp (>=\n0.11.0), rlang, stringi, stringr (>= 1.0.0), utils, xml2"                                                                                                                                                                                                                                                                                                                                                                                                    
# rprojroot     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# rstudioapi    NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# rversions     "curl, utils, xml2 (>= 1.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# rvest         "httr (>= 0.5), lifecycle (>= 1.0.0), magrittr, rlang (>=\n0.4.10), selectr, tibble, xml2 (>= 1.3)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# scales        "farver (>= 2.0.3), labeling, lifecycle, munsell (>= 0.5), R6,\nRColorBrewer, viridisLite"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# selectr       "methods, stringr, R6"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# sessioninfo   "cli, tools, utils, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# stringi       "tools, utils, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# stringr       "glue (>= 1.2.0), magrittr, stringi (>= 1.1.7)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# svglite       "systemfonts (>= 1.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# sys           NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# systemfonts   NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# testthat      "brio, callr (>= 3.5.1), cli (>= 2.2.0), crayon (>= 1.3.4),\ndesc, digest, ellipsis (>= 0.2.0), evaluate, jsonlite,\nlifecycle, magrittr, methods, pkgload, praise, processx, ps (>=\n1.3.4), R6 (>= 2.2.0), rlang (>= 0.4.9), utils, waldo (>=\n0.2.4), withr (>= 2.3.0)"                                                                                                                                                                                                                                                                                                               
# tibble        "ellipsis (>= 0.3.2), fansi (>= 0.4.0), lifecycle (>= 0.2.0),\nmagrittr, methods, pillar (>= 1.6.0), pkgconfig, rlang (>=\n0.4.3), utils, vctrs (>= 0.3.8)"                                                                                                                                                                                                                                                                                                                                                                                                                              
# tidyr         "dplyr (>= 0.8.2), ellipsis (>= 0.1.0), glue, lifecycle,\nmagrittr, purrr, rlang, tibble (>= 2.1.1), tidyselect (>=\n1.1.0), utils, vctrs (>= 0.3.6)"                                                                                                                                                                                                                                                                                                                                                                                                                                    
# tidyselect    "ellipsis, glue (>= 1.3.0), purrr (>= 0.3.2), rlang (>= 0.4.6),\nvctrs (>= 0.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# tidyverse     "broom (>= 0.7.6), cli (>= 2.4.0), crayon (>= 1.4.1), dbplyr\n(>= 2.1.1), dplyr (>= 1.0.5), dtplyr (>= 1.1.0), forcats (>=\n0.5.1), googledrive (>= 1.0.1), googlesheets4 (>= 0.3.0),\nggplot2 (>= 3.3.3), haven (>= 2.3.1), hms (>= 1.0.0), httr (>=\n1.4.2), jsonlite (>= 1.7.2), lubridate (>= 1.7.10), magrittr\n(>= 2.0.1), modelr (>= 0.1.8), pillar (>= 1.6.0), purrr (>=\n0.3.4), readr (>= 1.4.0), readxl (>= 1.3.1), reprex (>= 2.0.0),\nrlang (>= 0.4.10), rstudioapi (>= 0.13), rvest (>= 1.0.0),\nstringr (>= 1.4.0), tibble (>= 3.1.0), tidyr (>= 1.1.3), xml2\n(>= 1.3.2)"
# tinytex       "xfun (>= 0.23)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# tzdb          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# usethis       "cli, clipr (>= 0.3.0), crayon, curl (>= 2.7), desc, fs (>=\n1.3.0), gert (>= 1.0.2), gh (>= 1.2.0), glue (>= 1.3.0),\njsonlite, lifecycle, purrr, rappdirs, rlang (>= 0.4.10),\nrprojroot (>= 1.2), rstudioapi, stats, utils, whisker, withr\n(>= 2.3.0), yaml"                                                                                                                                                                                                                                                                                                                         
# utf8          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# uuid          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# vctrs         "ellipsis (>= 0.2.0), glue, rlang (>= 0.4.10)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# viridisLite   NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# vroom         "bit64, crayon, cli, glue, hms, lifecycle, methods, rlang (>=\n0.4.2), stats, tibble (>= 2.0.0), tzdb (>= 0.1.1), vctrs (>=\n0.2.0), tidyselect, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                  
# waldo         "cli, diffobj (>= 0.3.4), fansi, glue, methods, rematch2, rlang\n(>= 0.4.10), tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# whisker       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# withr         "graphics, grDevices, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# xfun          "stats, tools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# xml2          "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# xopen         "processx"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# yaml          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# zip           NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# base          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# boot          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# class         "MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# cluster       "graphics, grDevices, stats, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# codetools     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# compiler      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# datasets      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# foreign       "methods, utils, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# graphics      "grDevices"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# grDevices     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# grid          "grDevices, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# KernSmooth    NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# lattice       "grid, grDevices, graphics, stats, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# MASS          "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# Matrix        "methods, graphics, grid, stats, utils, lattice"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# methods       "utils, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# mgcv          "methods, stats, graphics, Matrix, splines, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# nlme          "graphics, stats, utils, lattice"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# nnet          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# parallel      "tools, compiler"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# rpart         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# spatial       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# splines       "graphics, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# stats         "utils, grDevices, graphics"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# stats4        "graphics, methods, stats"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# survival      "graphics, Matrix, methods, splines, stats, utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# tcltk         "utils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# tools         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# utils         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
#               LinkingTo                                               
# IRdisplay     NA                                                      
# IRkernel      NA                                                      
# pbdZMQ        NA                                                      
# repr          NA                                                      
# askpass       NA                                                      
# assertthat    NA                                                      
# backports     NA                                                      
# base64enc     NA                                                      
# bit           NA                                                      
# bit64         NA                                                      
# blob          NA                                                      
# brew          NA                                                      
# brio          NA                                                      
# broom         NA                                                      
# cachem        NA                                                      
# callr         NA                                                      
# cellranger    NA                                                      
# cli           NA                                                      
# clipr         NA                                                      
# colorspace    NA                                                      
# commonmark    NA                                                      
# cpp11         NA                                                      
# crayon        NA                                                      
# credentials   NA                                                      
# curl          NA                                                      
# data.table    NA                                                      
# DBI           NA                                                      
# dbplyr        NA                                                      
# desc          NA                                                      
# devtools      NA                                                      
# diffobj       NA                                                      
# digest        NA                                                      
# dplyr         NA                                                      
# dtplyr        NA                                                      
# ellipsis      NA                                                      
# evaluate      NA                                                      
# fansi         NA                                                      
# farver        NA                                                      
# fastmap       NA                                                      
# forcats       NA                                                      
# fs            NA                                                      
# gargle        NA                                                      
# generics      NA                                                      
# gert          NA                                                      
# ggplot2       NA                                                      
# gh            NA                                                      
# gitcreds      NA                                                      
# glue          NA                                                      
# googledrive   NA                                                      
# googlesheets4 NA                                                      
# gtable        NA                                                      
# haven         "cpp11"                                                 
# highr         NA                                                      
# hms           NA                                                      
# htmltools     NA                                                      
# httr          NA                                                      
# ids           NA                                                      
# ini           NA                                                      
# isoband       NA                                                      
# jsonlite      NA                                                      
# knitr         NA                                                      
# labeling      NA                                                      
# lifecycle     NA                                                      
# lubridate     "Rcpp"                                                  
# magrittr      NA                                                      
# markdown      NA                                                      
# memoise       NA                                                      
# mime          NA                                                      
# modelr        NA                                                      
# munsell       NA                                                      
# openssl       NA                                                      
# pillar        NA                                                      
# pkgbuild      NA                                                      
# pkgconfig     NA                                                      
# pkgload       NA                                                      
# praise        NA                                                      
# prettyunits   NA                                                      
# processx      NA                                                      
# progress      NA                                                      
# ps            NA                                                      
# purrr         NA                                                      
# R6            NA                                                      
# rappdirs      NA                                                      
# rcmdcheck     NA                                                      
# RColorBrewer  NA                                                      
# Rcpp          NA                                                      
# readr         "cpp11, tzdb (>= 0.1.1)"                                
# readxl        "progress, Rcpp"                                        
# rematch       NA                                                      
# rematch2      NA                                                      
# remotes       NA                                                      
# reprex        NA                                                      
# rlang         NA                                                      
# rmarkdown     NA                                                      
# roxygen2      "Rcpp"                                                  
# rprojroot     NA                                                      
# rstudioapi    NA                                                      
# rversions     NA                                                      
# rvest         NA                                                      
# scales        NA                                                      
# selectr       NA                                                      
# sessioninfo   NA                                                      
# stringi       NA                                                      
# stringr       NA                                                      
# svglite       "cpp11, systemfonts"                                    
# sys           NA                                                      
# systemfonts   "cpp11 (>= 0.2.1)"                                      
# testthat      NA                                                      
# tibble        NA                                                      
# tidyr         "cpp11 (>= 0.2.6)"                                      
# tidyselect    NA                                                      
# tidyverse     NA                                                      
# tinytex       NA                                                      
# tzdb          "cpp11 (>= 0.2.7)"                                      
# usethis       NA                                                      
# utf8          NA                                                      
# uuid          NA                                                      
# vctrs         NA                                                      
# viridisLite   NA                                                      
# vroom         "progress (>= 1.2.1), cpp11 (>= 0.2.0), tzdb (>= 0.1.1)"
# waldo         NA                                                      
# whisker       NA                                                      
# withr         NA                                                      
# xfun          NA                                                      
# xml2          NA                                                      
# xopen         NA                                                      
# yaml          NA                                                      
# zip           NA                                                      
# base          NA                                                      
# boot          NA                                                      
# class         NA                                                      
# cluster       NA                                                      
# codetools     NA                                                      
# compiler      NA                                                      
# datasets      NA                                                      
# foreign       NA                                                      
# graphics      NA                                                      
# grDevices     NA                                                      
# grid          NA                                                      
# KernSmooth    NA                                                      
# lattice       NA                                                      
# MASS          NA                                                      
# Matrix        NA                                                      
# methods       NA                                                      
# mgcv          NA                                                      
# nlme          NA                                                      
# nnet          NA                                                      
# parallel      NA                                                      
# rpart         NA                                                      
# spatial       NA                                                      
# splines       NA                                                      
# stats         NA                                                      
# stats4        NA                                                      
# survival      NA                                                      
# tcltk         NA                                                      
# tools         NA                                                      
# utils         NA                                                      
#               Suggests                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# IRdisplay     "testthat, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# IRkernel      "testthat, roxygen2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# pbdZMQ        NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# repr          "methods, highr, Cairo, stringr, testthat (>= 3.0.0), leaflet"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# askpass       "testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# assertthat    "testthat, covr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# backports     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# base64enc     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# bit           "testthat (>= 0.11.0), roxygen2, knitr, rmarkdown,\nmicrobenchmark, bit64 (>= 4.0.0), ff (>= 4.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# bit64         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# blob          "covr, crayon, pillar (>= 1.2.1), testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# brew          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# brio          "testthat (>= 2.1.0), covr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# broom         "AER, akima, AUC, bbmle, betareg, biglm, binGroup, boot,\nbtergm, car, caret, cluster, cmprsk, coda, covr, drc, e1071,\nemmeans, epiR, ergm (>= 3.10.4), fixest (>= 0.9.0), gam (>=\n1.15), gee, geepack, ggplot2, glmnet, glmnetUtils, gmm, Hmisc,\nirlba, joineRML, Kendall, knitr, ks, Lahman, lavaan, leaps,\nlfe, lm.beta, lme4, lmodel2, lmtest (>= 0.9.38), lsmeans, maps,\nmaptools, margins, MASS, Matrix, mclust, mediation, metafor,\nmfx, mgcv, mlogit, modeldata, modeltests, muhaz, multcomp,\nnetwork, nnet, orcutt (>= 2.2), ordinal, plm, poLCA, psych,\nquantreg, rgeos, rmarkdown, robust, robustbase, rsample,\nsandwich, sp, spdep (>= 1.1), spatialreg, speedglm, spelling,\nsurvey, survival, systemfit, testthat (>= 2.1.0), tseries,\nvars, zoo"
# cachem        "testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# callr         "cli, covr, ps, rprojroot, spelling, testthat, withr (>=\n2.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# cellranger    "covr, testthat (>= 1.0.0), knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# cli           "callr, covr, grDevices, htmlwidgets, knitr, methods, mockery,\nprettycode (>= 1.1.0), processx, ps (>= 1.3.4.9000), rlang,\nrmarkdown, rstudioapi, shiny, testthat, tibble, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# clipr         "covr, knitr, rmarkdown, rstudioapi (>= 0.5), testthat (>=\n2.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# colorspace    "datasets, utils, KernSmooth, MASS, kernlab, mvtnorm, vcd,\ntcltk, shiny, shinyjs, ggplot2, dplyr, scales, grid, png, jpeg,\nknitr, rmarkdown, RColorBrewer, rcartocolor, scico, viridis,\nwesanderson"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# commonmark    "curl, testthat, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# cpp11         "bench, brio, callr, cli, covr, decor, desc, ggplot2, glue,\nknitr, lobstr, mockery, progress, rmarkdown, scales, Rcpp,\ntestthat, tibble, utils, vctrs, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# crayon        "mockery, rstudioapi, testthat, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# credentials   "testthat, knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# curl          "spelling, testthat (>= 1.0.0), knitr, jsonlite, rmarkdown,\nmagrittr, httpuv (>= 1.4.4), webutils"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# data.table    "bit64 (>= 4.0.0), bit (>= 4.0.4), curl, R.utils, xts,\nnanotime, zoo (>= 1.8-1), yaml, knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# DBI           "blob, covr, dbplyr, dplyr, glue, hms, knitr, magrittr,\nrmarkdown, rprojroot, RMariaDB, RSQLite (>= 1.1-2), testthat,\nxml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# dbplyr        "bit64, covr, knitr, Lahman, nycflights13, odbc, RMariaDB (>=\n1.0.2), rmarkdown, RPostgres (>= 1.1.3), RPostgreSQL, RSQLite\n(>= 2.1.0), testthat (>= 3.0.2), tidyr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# desc          "covr, testthat, whoami, withr, spelling, gh"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# devtools      "BiocManager (>= 1.30.12), covr (>= 3.5.1), curl (>= 4.3),\ndigest (>= 0.6.27), DT (>= 0.17), foghorn (>= 1.3.2), gh (>=\n1.2.1), gmailr (>= 1.0.0), knitr (>= 1.31), lintr (>= 2.0.1),\nMASS, mockery (>= 0.4.2), pingr (>= 2.0.1), pkgdown (>= 1.6.1),\nrhub (>= 1.1.1), rmarkdown (>= 2.7), spelling (>= 2.2)"                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# diffobj       "knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# digest        "tinytest, knitr, rmarkdown, minidown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# dplyr         "bench, broom, callr, covr, DBI, dbplyr (>= 1.4.3), knitr,\nLahman, lobstr, microbenchmark, nycflights13, purrr, rmarkdown,\nRMySQL, RPostgreSQL, RSQLite, testthat (>= 3.0.2), tidyr, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# dtplyr        "bench, covr, knitr, rmarkdown, testthat (>= 3.0.0), tidyr (>=\n1.1.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# ellipsis      "covr, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# evaluate      "testthat, lattice, ggplot2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# fansi         "unitizer, knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# farver        "testthat (>= 2.1.0), covr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# fastmap       "testthat (>= 2.1.1)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# forcats       "covr, dplyr, ggplot2, knitr, readr, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# fs            "testthat, covr, pillar (>= 1.0.0), tibble (>= 1.1.0), crayon,\nrmarkdown, knitr, withr, spelling, vctrs (>= 0.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# gargle        "aws.ec2metadata, aws.signature, covr, httpuv, knitr, mockr,\nrmarkdown, sodium, spelling, testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# generics      "covr, pkgload, testthat, tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# gert          "spelling, knitr, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# ggplot2       "covr, ragg, dplyr, ggplot2movies, hexbin, Hmisc, interp,\nknitr, lattice, mapproj, maps, maptools, multcomp, munsell,\nnlme, profvis, quantreg, RColorBrewer, rgeos, rmarkdown, rpart,\nsf (>= 0.7-3), svglite (>= 1.2.0.9001), testthat (>= 2.1.0),\nvdiffr (>= 1.0.0), xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# gh            "covr, knitr, mockery, rmarkdown, rprojroot, spelling,\ntestthat (>= 2.3.2), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# gitcreds      "codetools, testthat, knitr, mockery, oskeyring, rmarkdown,\nwithr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# glue          "testthat, covr, magrittr, crayon, knitr, rmarkdown, DBI,\nRSQLite, R.utils, forcats, microbenchmark, rprintf, stringr,\nggplot2, dplyr, withr, vctrs (>= 0.3.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# googledrive   "covr, curl, downlit, dplyr (>= 1.0.0), knitr, mockr,\nrmarkdown, roxygen2, sodium, spelling, testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# googlesheets4 "covr, readr, rmarkdown, sodium, spelling, testthat (>=\n3.0.0), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# gtable        "covr, testthat, knitr, rmarkdown, ggplot2, profvis"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# haven         "cli, covr, crayon, fs, knitr, pillar (>= 1.4.0), rmarkdown,\ntestthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# highr         "knitr, markdown, testit"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# hms           "crayon, lubridate, pillar (>= 1.1.0), testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# htmltools     "markdown, testthat, withr, Cairo, ragg, shiny"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# httr          "covr, httpuv, jpeg, knitr, png, readr, rmarkdown, testthat\n(>= 0.8.0), xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# ids           "knitr, rcorpora, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# ini           "testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# isoband       "covr, ggplot2, knitr, magick, microbenchmark, rmarkdown, sf,\ntestthat, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# jsonlite      "httr, curl, plyr, testthat, knitr, rmarkdown, R.rsp, sf"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# knitr         "formatR, testit, digest, rgl (>= 0.95.1201), codetools,\nrmarkdown, htmlwidgets (>= 0.7), webshot, tikzDevice (>= 0.10),\ntinytex, reticulate (>= 1.4), JuliaCall (>= 0.11.1), magick,\npng, jpeg, gifski, xml2 (>= 1.2.0), httr, DBI (>= 0.4-1),\nshowtext, tibble, sass, bslib, ragg, styler (>= 1.2.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# labeling      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# lifecycle     "covr, crayon, knitr, rmarkdown, testthat (>= 3.0.1), tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# lubridate     "covr, knitr, testthat (>= 2.1.0), vctrs (>= 0.3.0), rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# magrittr      "covr, knitr, rlang, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# markdown      "knitr, RCurl"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# memoise       "digest, aws.s3, covr, googleAuthR, googleCloudStorageR, httr,\ntestthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# mime          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# modelr        "compiler, covr, ggplot2, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# munsell       "ggplot2, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# openssl       "testthat (>= 2.1.0), digest, knitr, rmarkdown, jsonlite,\njose, sodium"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# pillar        "bit64, debugme, DiagrammeR, dplyr, formattable, ggplot2,\nknitr, lubridate, nycflights13, palmerpenguins, rmarkdown,\nscales, stringi, survival, testthat (>= 3.0.2), tibble, units\n(>= 0.7.2), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# pkgbuild      "Rcpp, cpp11, testthat, covr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# pkgconfig     "covr, testthat, disposables (>= 1.0.3)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# pkgload       "bitops, covr, pkgbuild, Rcpp, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# praise        "testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# prettyunits   "codetools, covr, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# processx      "callr (>= 3.2.0), cli, codetools, covr, curl, debugme,\nparallel, testthat, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# progress      "Rcpp, testthat, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# ps            "callr, covr, curl, pingr, processx (>= 3.1.0), R6, rlang,\ntestthat, tibble"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# purrr         "covr, crayon, dplyr (>= 0.7.8), knitr, rmarkdown, testthat,\ntibble, tidyselect"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# R6            "testthat, pryr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# rappdirs      "roxygen2, testthat (>= 3.0.0), covr, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# rcmdcheck     "covr, knitr, mockery, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# RColorBrewer  NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# Rcpp          "tinytest, inline, rbenchmark, pkgKitten (>= 0.1.2)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# readr         "covr, curl, dplyr, knitr, rmarkdown, spelling, stringi,\ntestthat, tzdb (>= 0.1.1), waldo, withr, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# readxl        "covr, knitr, rmarkdown, rprojroot (>= 1.1), testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# rematch       "covr, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# rematch2      "covr, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# remotes       "brew, callr, codetools, curl, covr, git2r (>= 0.23.0), knitr,\nmockery, pkgbuild (>= 1.0.1), pingr, rmarkdown, rprojroot,\ntestthat, webfakes, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# reprex        "covr, fortunes, miniUI, mockr, rprojroot, sessioninfo, shiny,\nspelling, styler (>= 1.2.0), testthat (>= 3.0.2)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# rlang         "cli, covr, crayon, glue, magrittr, methods, pak, pillar,\nrmarkdown, testthat (>= 3.0.0), vctrs (>= 0.2.3), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# rmarkdown     "shiny (>= 1.6.0), tufte, testthat (>= 3.0.0), digest,\ndygraphs, vctrs, tibble, fs, rsconnect, withr (>= 2.4.2), bslib\n(>= 0.2.5.1), sass (>= 0.4.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# roxygen2      "covr, devtools, rmarkdown, testthat (>= 2.1.0), R.methodsS3,\nR.oo"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# rprojroot     "covr, knitr, lifecycle, mockr, rmarkdown, testthat (>=\n3.0.0), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# rstudioapi    "testthat, knitr, rmarkdown, clipr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# rversions     "mockery, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# rvest         "covr, glue, knitr, readr, rmarkdown, repurrrsive, spelling,\nstringi (>= 0.3.1), testthat (>= 3.0.2), webfakes"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# scales        "bit64, covr, dichromat, ggplot2, hms (>= 0.5.0), testthat (>=\n2.1.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# selectr       "testthat, XML, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# sessioninfo   "callr, covr, mockery, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# stringi       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# stringr       "covr, htmltools, htmlwidgets, knitr, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# svglite       "htmltools, testthat, xml2 (>= 1.0.0), covr, fontquiver (>=\n0.2.0), knitr, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# sys           "unix (>= 1.4), spelling, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# systemfonts   "testthat (>= 2.1.0), covr, knitr, rmarkdown, tools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# testthat      "covr, curl (>= 0.9.5), diffviewer (>= 0.1.0), knitr, mockery,\nrmarkdown, rstudioapi, shiny, usethis, vctrs (>= 0.1.0), xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# tibble        "bench, bit64, blob, cli, covr, crayon (>= 1.3.4), DiagrammeR,\ndplyr, evaluate, formattable, ggplot2, hms, htmltools, knitr,\nlubridate, mockr, nycflights13, purrr, rmarkdown, testthat (>=\n3.0.2), tidyr, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# tidyr         "covr, data.table, jsonlite, knitr, readr, repurrrsive (>=\n1.0.0), rmarkdown, testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
# tidyselect    "covr, crayon, dplyr, knitr, magrittr, rmarkdown, testthat (>=\n2.3.0), tibble (>= 2.1.3), withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# tidyverse     "covr, feather, glue, knitr, rmarkdown, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# tinytex       "testit, rstudioapi"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
# tzdb          "covr, testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# usethis       "covr, knitr, magick, mockr, rmarkdown, roxygen2, spelling (>=\n1.2), styler (>= 1.2.0), testthat (>= 3.0.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# utf8          "cli, covr, knitr, rlang, rmarkdown, testthat (>= 3.0.0),\nwithr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# uuid          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# vctrs         "bit64, covr, crayon, dplyr (>= 0.8.5), generics, knitr,\npillar (>= 1.4.4), pkgdown, rmarkdown, testthat (>= 2.3.0),\ntibble, withr, xml2, waldo (>= 0.2.0), zeallot"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# viridisLite   "hexbin (>= 1.27.0), ggplot2 (>= 1.0.1), testthat, covr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
# vroom         "archive, bench (>= 1.1.0), covr, curl, dplyr, forcats, fs,\nggplot2, knitr, patchwork, prettyunits, purrr, rmarkdown,\nrstudioapi, scales, spelling, testthat (>= 2.1.0), tidyr,\nwaldo, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# waldo         "testthat (>= 3.0.0), covr, R6, xml2"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# whisker       "markdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# withr         "covr, DBI, knitr, lattice, methods, rmarkdown, RSQLite,\ntestthat (>= 2.1.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# xfun          "testit, parallel, codetools, rstudioapi, tinytex (>= 0.30),\nmime, markdown, knitr, htmltools, remotes, pak, renv, curl,\njsonlite, rmarkdown"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# xml2          "covr, curl, httr, knitr, magrittr, mockery, rmarkdown,\ntestthat (>= 2.1.0)"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# xopen         "ps, testthat"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# yaml          "RUnit"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
# zip           "covr, processx, R6, testthat, withr"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
# base          "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# boot          "MASS, survival"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# class         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# cluster       "MASS, Matrix"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
# codetools     NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# compiler      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# datasets      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# foreign       NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# graphics      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# grDevices     "KernSmooth"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# grid          NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# KernSmooth    "MASS, carData"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# lattice       "KernSmooth, MASS, latticeExtra"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
# MASS          "lattice, nlme, nnet, survival"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
# Matrix        "expm, MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# methods       "codetools"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
# mgcv          "parallel, survival, MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# nlme          "Hmisc, MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
# nnet          "MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# parallel      "methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
# rpart         "survival"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# spatial       "MASS"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
# splines       "Matrix, methods"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# stats         "MASS, Matrix, SuppDists, methods, stats4"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
# stats4        NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# survival      NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# tcltk         NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
# tools         "codetools, methods, xml2, curl, commonmark"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
# utils         "methods, xml2, commonmark"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
#               Enhances                                                             
# IRdisplay     NA                                                                   
# IRkernel      NA                                                                   
# pbdZMQ        NA                                                                   
# repr          "data.table, tibble, htmlwidgets, vegalite, plotly, geojsonio"       
# askpass       NA                                                                   
# assertthat    NA                                                                   
# backports     NA                                                                   
# base64enc     "png"                                                                
# bit           NA                                                                   
# bit64         NA                                                                   
# blob          NA                                                                   
# brew          NA                                                                   
# brio          NA                                                                   
# broom         NA                                                                   
# cachem        NA                                                                   
# callr         NA                                                                   
# cellranger    NA                                                                   
# cli           NA                                                                   
# clipr         NA                                                                   
# colorspace    NA                                                                   
# commonmark    NA                                                                   
# cpp11         NA                                                                   
# crayon        NA                                                                   
# credentials   NA                                                                   
# curl          NA                                                                   
# data.table    NA                                                                   
# DBI           NA                                                                   
# dbplyr        NA                                                                   
# desc          NA                                                                   
# devtools      NA                                                                   
# diffobj       NA                                                                   
# digest        NA                                                                   
# dplyr         NA                                                                   
# dtplyr        NA                                                                   
# ellipsis      NA                                                                   
# evaluate      NA                                                                   
# fansi         NA                                                                   
# farver        NA                                                                   
# fastmap       NA                                                                   
# forcats       NA                                                                   
# fs            NA                                                                   
# gargle        NA                                                                   
# generics      NA                                                                   
# gert          NA                                                                   
# ggplot2       "sp"                                                                 
# gh            NA                                                                   
# gitcreds      NA                                                                   
# glue          NA                                                                   
# googledrive   NA                                                                   
# googlesheets4 NA                                                                   
# gtable        NA                                                                   
# haven         NA                                                                   
# highr         NA                                                                   
# hms           NA                                                                   
# htmltools     "knitr"                                                              
# httr          NA                                                                   
# ids           NA                                                                   
# ini           NA                                                                   
# isoband       NA                                                                   
# jsonlite      NA                                                                   
# knitr         NA                                                                   
# labeling      NA                                                                   
# lifecycle     NA                                                                   
# lubridate     "chron, timeDate, tis, zoo"                                          
# magrittr      NA                                                                   
# markdown      NA                                                                   
# memoise       NA                                                                   
# mime          NA                                                                   
# modelr        NA                                                                   
# munsell       NA                                                                   
# openssl       NA                                                                   
# pillar        NA                                                                   
# pkgbuild      NA                                                                   
# pkgconfig     NA                                                                   
# pkgload       NA                                                                   
# praise        NA                                                                   
# prettyunits   NA                                                                   
# processx      NA                                                                   
# progress      NA                                                                   
# ps            NA                                                                   
# purrr         NA                                                                   
# R6            NA                                                                   
# rappdirs      NA                                                                   
# rcmdcheck     NA                                                                   
# RColorBrewer  NA                                                                   
# Rcpp          NA                                                                   
# readr         NA                                                                   
# readxl        NA                                                                   
# rematch       NA                                                                   
# rematch2      NA                                                                   
# remotes       NA                                                                   
# reprex        NA                                                                   
# rlang         "winch"                                                              
# rmarkdown     NA                                                                   
# roxygen2      NA                                                                   
# rprojroot     NA                                                                   
# rstudioapi    NA                                                                   
# rversions     NA                                                                   
# rvest         NA                                                                   
# scales        NA                                                                   
# selectr       NA                                                                   
# sessioninfo   NA                                                                   
# stringi       NA                                                                   
# stringr       NA                                                                   
# svglite       NA                                                                   
# sys           NA                                                                   
# systemfonts   NA                                                                   
# testthat      NA                                                                   
# tibble        NA                                                                   
# tidyr         NA                                                                   
# tidyselect    NA                                                                   
# tidyverse     NA                                                                   
# tinytex       NA                                                                   
# tzdb          NA                                                                   
# usethis       NA                                                                   
# utf8          NA                                                                   
# uuid          NA                                                                   
# vctrs         NA                                                                   
# viridisLite   NA                                                                   
# vroom         NA                                                                   
# waldo         NA                                                                   
# whisker       NA                                                                   
# withr         NA                                                                   
# xfun          NA                                                                   
# xml2          NA                                                                   
# xopen         NA                                                                   
# yaml          NA                                                                   
# zip           NA                                                                   
# base          NA                                                                   
# boot          NA                                                                   
# class         NA                                                                   
# cluster       NA                                                                   
# codetools     NA                                                                   
# compiler      NA                                                                   
# datasets      NA                                                                   
# foreign       NA                                                                   
# graphics      NA                                                                   
# grDevices     NA                                                                   
# grid          NA                                                                   
# KernSmooth    NA                                                                   
# lattice       "chron"                                                              
# MASS          NA                                                                   
# Matrix        "MatrixModels, graph, SparseM, sfsmisc, igraph, maptools, sp,\nspdep"
# methods       NA                                                                   
# mgcv          NA                                                                   
# nlme          NA                                                                   
# nnet          NA                                                                   
# parallel      "snow, nws, Rmpi"                                                    
# rpart         NA                                                                   
# spatial       NA                                                                   
# splines       NA                                                                   
# stats         NA                                                                   
# stats4        NA                                                                   
# survival      NA                                                                   
# tcltk         NA                                                                   
# tools         NA                                                                   
# utils         NA                                                                   
#               License                                  License_is_FOSS
# IRdisplay     "MIT + file LICENSE"                     NA             
# IRkernel      "MIT + file LICENSE"                     NA             
# pbdZMQ        "GPL-3"                                  NA             
# repr          "GPL-3"                                  NA             
# askpass       "MIT + file LICENSE"                     NA             
# assertthat    "GPL-3"                                  NA             
# backports     "GPL-2 | GPL-3"                          NA             
# base64enc     "GPL-2 | GPL-3"                          NA             
# bit           "GPL-2 | GPL-3"                          NA             
# bit64         "GPL-2 | GPL-3"                          NA             
# blob          "MIT + file LICENSE"                     NA             
# brew          "GPL-2"                                  NA             
# brio          "MIT + file LICENSE"                     NA             
# broom         "MIT + file LICENSE"                     NA             
# cachem        "MIT + file LICENSE"                     NA             
# callr         "MIT + file LICENSE"                     NA             
# cellranger    "MIT + file LICENSE"                     NA             
# cli           "MIT + file LICENSE"                     NA             
# clipr         "GPL-3"                                  NA             
# colorspace    "BSD_3_clause + file LICENSE"            NA             
# commonmark    "BSD_2_clause + file LICENSE"            NA             
# cpp11         "MIT + file LICENSE"                     NA             
# crayon        "MIT + file LICENSE"                     NA             
# credentials   "MIT + file LICENSE"                     NA             
# curl          "MIT + file LICENSE"                     NA             
# data.table    "MPL-2.0 | file LICENSE"                 NA             
# DBI           "LGPL (>= 2.1)"                          NA             
# dbplyr        "MIT + file LICENSE"                     NA             
# desc          "MIT + file LICENSE"                     NA             
# devtools      "MIT + file LICENSE"                     NA             
# diffobj       "GPL (>= 2)"                             NA             
# digest        "GPL (>= 2)"                             NA             
# dplyr         "MIT + file LICENSE"                     NA             
# dtplyr        "MIT + file LICENSE"                     NA             
# ellipsis      "MIT + file LICENSE"                     NA             
# evaluate      "MIT + file LICENSE"                     NA             
# fansi         "GPL (>= 2)"                             NA             
# farver        "MIT + file LICENSE"                     NA             
# fastmap       "MIT + file LICENSE"                     NA             
# forcats       "MIT + file LICENSE"                     NA             
# fs            "GPL-3"                                  NA             
# gargle        "MIT + file LICENSE"                     NA             
# generics      "MIT + file LICENSE"                     NA             
# gert          "MIT + file LICENSE"                     NA             
# ggplot2       "MIT + file LICENSE"                     NA             
# gh            "MIT + file LICENSE"                     NA             
# gitcreds      "MIT + file LICENSE"                     NA             
# glue          "MIT + file LICENSE"                     NA             
# googledrive   "MIT + file LICENSE"                     NA             
# googlesheets4 "MIT + file LICENSE"                     NA             
# gtable        "GPL-2"                                  NA             
# haven         "MIT + file LICENSE"                     NA             
# highr         "GPL"                                    NA             
# hms           "MIT + file LICENSE"                     NA             
# htmltools     "GPL (>= 2)"                             NA             
# httr          "MIT + file LICENSE"                     NA             
# ids           "MIT + file LICENSE"                     NA             
# ini           "GPL-3"                                  NA             
# isoband       "MIT + file LICENSE"                     NA             
# jsonlite      "MIT + file LICENSE"                     NA             
# knitr         "GPL"                                    NA             
# labeling      "MIT + file LICENSE | Unlimited"         NA             
# lifecycle     "MIT + file LICENSE"                     NA             
# lubridate     "GPL (>= 2)"                             NA             
# magrittr      "MIT + file LICENSE"                     NA             
# markdown      "GPL-2"                                  NA             
# memoise       "MIT + file LICENSE"                     NA             
# mime          "GPL"                                    NA             
# modelr        "GPL-3"                                  NA             
# munsell       "MIT + file LICENSE"                     NA             
# openssl       "MIT + file LICENSE"                     NA             
# pillar        "MIT + file LICENSE"                     NA             
# pkgbuild      "MIT + file LICENSE"                     NA             
# pkgconfig     "MIT + file LICENSE"                     NA             
# pkgload       "GPL-3"                                  NA             
# praise        "MIT + file LICENSE"                     NA             
# prettyunits   "MIT + file LICENSE"                     NA             
# processx      "MIT + file LICENSE"                     NA             
# progress      "MIT + file LICENSE"                     NA             
# ps            "MIT + file LICENSE"                     NA             
# purrr         "GPL-3 | file LICENSE"                   NA             
# R6            "MIT + file LICENSE"                     NA             
# rappdirs      "MIT + file LICENSE"                     NA             
# rcmdcheck     "MIT + file LICENSE"                     NA             
# RColorBrewer  "Apache License 2.0"                     NA             
# Rcpp          "GPL (>= 2)"                             NA             
# readr         "MIT + file LICENSE"                     NA             
# readxl        "GPL-3"                                  NA             
# rematch       "MIT + file LICENSE"                     NA             
# rematch2      "MIT + file LICENSE"                     NA             
# remotes       "MIT + file LICENSE"                     NA             
# reprex        "MIT + file LICENSE"                     NA             
# rlang         "MIT + file LICENSE"                     NA             
# rmarkdown     "GPL-3"                                  NA             
# roxygen2      "GPL (>= 2)"                             NA             
# rprojroot     "MIT + file LICENSE"                     NA             
# rstudioapi    "MIT + file LICENSE"                     NA             
# rversions     "MIT + file LICENSE"                     NA             
# rvest         "MIT + file LICENSE"                     NA             
# scales        "MIT + file LICENSE"                     NA             
# selectr       "BSD_3_clause + file LICENCE"            NA             
# sessioninfo   "GPL-2"                                  NA             
# stringi       "file LICENSE"                           "yes"          
# stringr       "GPL-2 | file LICENSE"                   NA             
# svglite       "GPL (>= 2)"                             NA             
# sys           "MIT + file LICENSE"                     NA             
# systemfonts   "MIT + file LICENSE"                     NA             
# testthat      "MIT + file LICENSE"                     NA             
# tibble        "MIT + file LICENSE"                     NA             
# tidyr         "MIT + file LICENSE"                     NA             
# tidyselect    "MIT + file LICENSE"                     NA             
# tidyverse     "MIT + file LICENSE"                     NA             
# tinytex       "MIT + file LICENSE"                     NA             
# tzdb          "MIT + file LICENSE"                     NA             
# usethis       "MIT + file LICENSE"                     NA             
# utf8          "Apache License (== 2.0) | file LICENSE" NA             
# uuid          "MIT + file LICENSE"                     NA             
# vctrs         "MIT + file LICENSE"                     NA             
# viridisLite   "MIT + file LICENSE"                     NA             
# vroom         "MIT + file LICENSE"                     NA             
# waldo         "MIT + file LICENSE"                     NA             
# whisker       "GPL-3"                                  NA             
# withr         "MIT + file LICENSE"                     NA             
# xfun          "MIT + file LICENSE"                     NA             
# xml2          "GPL (>= 2)"                             NA             
# xopen         "MIT + file LICENSE"                     NA             
# yaml          "BSD_3_clause + file LICENSE"            NA             
# zip           "MIT + file LICENSE"                     NA             
# base          "Part of R 4.1.1"                        NA             
# boot          "Unlimited"                              NA             
# class         "GPL-2 | GPL-3"                          NA             
# cluster       "GPL (>= 2)"                             NA             
# codetools     "GPL"                                    NA             
# compiler      "Part of R 4.1.1"                        NA             
# datasets      "Part of R 4.1.1"                        NA             
# foreign       "GPL (>= 2)"                             NA             
# graphics      "Part of R 4.1.1"                        NA             
# grDevices     "Part of R 4.1.1"                        NA             
# grid          "Part of R 4.1.1"                        NA             
# KernSmooth    "Unlimited"                              NA             
# lattice       "GPL (>= 2)"                             NA             
# MASS          "GPL-2 | GPL-3"                          NA             
# Matrix        "GPL (>= 2) | file LICENCE"              NA             
# methods       "Part of R 4.1.1"                        NA             
# mgcv          "GPL (>= 2)"                             NA             
# nlme          "GPL (>= 2) | file LICENCE"              NA             
# nnet          "GPL-2 | GPL-3"                          NA             
# parallel      "Part of R 4.1.1"                        NA             
# rpart         "GPL-2 | GPL-3"                          NA             
# spatial       "GPL-2 | GPL-3"                          NA             
# splines       "Part of R 4.1.1"                        NA             
# stats         "Part of R 4.1.1"                        NA             
# stats4        "Part of R 4.1.1"                        NA             
# survival      "LGPL (>= 2)"                            NA             
# tcltk         "Part of R 4.1.1"                        NA             
# tools         "Part of R 4.1.1"                        NA             
# utils         "Part of R 4.1.1"                        NA             
#               License_restricts_use OS_type MD5sum NeedsCompilation Built  
# IRdisplay     NA                    NA      NA     "no"             "4.1.1"
# IRkernel      NA                    NA      NA     "no"             "4.1.1"
# pbdZMQ        NA                    NA      NA     "yes"            "4.1.1"
# repr          NA                    NA      NA     "no"             "4.1.1"
# askpass       NA                    NA      NA     "yes"            "4.0.0"
# assertthat    NA                    NA      NA     "no"             "4.0.0"
# backports     NA                    NA      NA     "yes"            "4.0.3"
# base64enc     NA                    NA      NA     "yes"            "4.0.0"
# bit           NA                    NA      NA     "yes"            "4.0.2"
# bit64         NA                    NA      NA     "yes"            "4.0.2"
# blob          NA                    NA      NA     "no"             "4.1.0"
# brew          NA                    NA      NA     NA               "4.0.0"
# brio          NA                    NA      NA     "yes"            "4.0.5"
# broom         NA                    NA      NA     "no"             "4.1.0"
# cachem        NA                    NA      NA     "yes"            "4.1.1"
# callr         NA                    NA      NA     "no"             "4.0.5"
# cellranger    NA                    NA      NA     "no"             "4.0.0"
# cli           NA                    NA      NA     "yes"            "4.1.0"
# clipr         NA                    NA      NA     "no"             "4.0.2"
# colorspace    NA                    NA      NA     "yes"            "4.1.0"
# commonmark    NA                    NA      NA     "yes"            "4.0.0"
# cpp11         NA                    NA      NA     "no"             "4.1.0"
# crayon        NA                    NA      NA     "no"             "4.0.3"
# credentials   NA                    NA      NA     "no"             "4.1.0"
# curl          NA                    NA      NA     "yes"            "4.1.0"
# data.table    NA                    NA      NA     "yes"            "4.0.4"
# DBI           NA                    NA      NA     "no"             "4.0.3"
# dbplyr        NA                    NA      NA     "no"             "4.0.5"
# desc          NA                    NA      NA     "no"             "4.0.4"
# devtools      NA                    NA      NA     "no"             "4.1.0"
# diffobj       NA                    NA      NA     "yes"            "4.0.4"
# digest        NA                    NA      NA     "yes"            "4.0.3"
# dplyr         NA                    NA      NA     "yes"            "4.1.0"
# dtplyr        NA                    NA      NA     "no"             "4.0.4"
# ellipsis      NA                    NA      NA     "yes"            "4.0.5"
# evaluate      NA                    NA      NA     "no"             "4.0.0"
# fansi         NA                    NA      NA     "yes"            "4.1.0"
# farver        NA                    NA      NA     "yes"            "4.0.4"
# fastmap       NA                    NA      NA     "yes"            "4.0.3"
# forcats       NA                    NA      NA     "no"             "4.0.3"
# fs            NA                    NA      NA     "yes"            "4.0.2"
# gargle        NA                    NA      NA     "no"             "4.1.0"
# generics      NA                    NA      NA     "no"             "4.0.3"
# gert          NA                    NA      NA     "yes"            "4.1.1"
# ggplot2       NA                    NA      NA     "no"             "4.1.0"
# gh            NA                    NA      NA     "no"             "4.0.5"
# gitcreds      NA                    NA      NA     "no"             "4.0.3"
# glue          NA                    NA      NA     "yes"            "4.0.2"
# googledrive   NA                    NA      NA     "no"             "4.1.0"
# googlesheets4 NA                    NA      NA     "no"             "4.1.0"
# gtable        NA                    NA      NA     "no"             "4.0.0"
# haven         NA                    NA      NA     "yes"            "4.1.0"
# highr         NA                    NA      NA     "no"             "4.0.5"
# hms           NA                    NA      NA     "no"             "4.0.5"
# htmltools     NA                    NA      NA     "yes"            "4.0.3"
# httr          NA                    NA      NA     "no"             "4.0.2"
# ids           NA                    NA      NA     "no"             "4.0.0"
# ini           NA                    NA      NA     "no"             "4.0.0"
# isoband       NA                    NA      NA     "yes"            "4.1.0"
# jsonlite      NA                    NA      NA     "yes"            "4.0.3"
# knitr         NA                    NA      NA     "no"             "4.0.5"
# labeling      NA                    NA      NA     "no"             "4.0.3"
# lifecycle     NA                    NA      NA     "no"             "4.0.4"
# lubridate     NA                    NA      NA     "yes"            "4.0.4"
# magrittr      NA                    NA      NA     "yes"            "4.0.3"
# markdown      NA                    NA      NA     "yes"            "4.0.0"
# memoise       NA                    NA      NA     "no"             "4.0.3"
# mime          NA                    NA      NA     "yes"            "4.1.0"
# modelr        NA                    NA      NA     "no"             "4.0.0"
# munsell       NA                    NA      NA     "no"             "4.0.0"
# openssl       NA                    NA      NA     "yes"            "4.0.2"
# pillar        NA                    NA      NA     "no"             "4.1.0"
# pkgbuild      NA                    NA      NA     "no"             "4.0.3"
# pkgconfig     NA                    NA      NA     "no"             "4.0.0"
# pkgload       NA                    NA      NA     "yes"            "4.0.5"
# praise        NA                    NA      NA     "no"             "4.0.0"
# prettyunits   NA                    NA      NA     "no"             "4.0.0"
# processx      NA                    NA      NA     "yes"            "4.0.5"
# progress      NA                    NA      NA     "no"             "4.0.0"
# ps            NA                    NA      NA     "yes"            "4.0.4"
# purrr         NA                    NA      NA     "yes"            "4.0.0"
# R6            NA                    NA      NA     "no"             "4.1.1"
# rappdirs      NA                    NA      NA     "yes"            "4.0.3"
# rcmdcheck     NA                    NA      NA     "no"             "4.0.0"
# RColorBrewer  NA                    NA      NA     "no"             "4.0.0"
# Rcpp          NA                    NA      NA     "yes"            "4.1.0"
# readr         NA                    NA      NA     "yes"            "4.1.1"
# readxl        NA                    NA      NA     "yes"            "4.0.0"
# rematch       NA                    NA      NA     "no"             "4.0.0"
# rematch2      NA                    NA      NA     "no"             "4.0.0"
# remotes       NA                    NA      NA     "no"             "4.1.0"
# reprex        NA                    NA      NA     "no"             "4.1.0"
# rlang         NA                    NA      NA     "yes"            "4.0.5"
# rmarkdown     NA                    NA      NA     "no"             "4.1.0"
# roxygen2      NA                    NA      NA     "yes"            "4.0.2"
# rprojroot     NA                    NA      NA     "no"             "4.0.3"
# rstudioapi    NA                    NA      NA     "no"             "4.0.3"
# rversions     NA                    NA      NA     "no"             "4.1.0"
# rvest         NA                    NA      NA     "no"             "4.1.0"
# scales        NA                    NA      NA     "no"             "4.0.0"
# selectr       NA                    NA      NA     "no"             "4.0.0"
# sessioninfo   NA                    NA      NA     "no"             "4.0.0"
# stringi       NA                    NA      NA     "yes"            "4.1.0"
# stringr       NA                    NA      NA     "no"             "4.0.0"
# svglite       NA                    NA      NA     "yes"            "4.0.4"
# sys           NA                    NA      NA     "yes"            "4.0.2"
# systemfonts   NA                    NA      NA     "yes"            "4.0.5"
# testthat      NA                    NA      NA     "yes"            "4.1.0"
# tibble        NA                    NA      NA     "yes"            "4.1.0"
# tidyr         NA                    NA      NA     "yes"            "4.0.4"
# tidyselect    NA                    NA      NA     "yes"            "4.0.5"
# tidyverse     NA                    NA      NA     "no"             "4.0.5"
# tinytex       NA                    NA      NA     "no"             "4.1.0"
# tzdb          NA                    NA      NA     "yes"            "4.1.0"
# usethis       NA                    NA      NA     "no"             "4.0.3"
# utf8          NA                    NA      NA     "yes"            "4.1.0"
# uuid          NA                    NA      NA     "yes"            "4.0.0"
# vctrs         NA                    NA      NA     "yes"            "4.0.5"
# viridisLite   NA                    NA      NA     "no"             "4.0.5"
# vroom         NA                    NA      NA     "yes"            "4.1.0"
# waldo         NA                    NA      NA     "no"             "4.1.1"
# whisker       NA                    NA      NA     "no"             "4.0.0"
# withr         NA                    NA      NA     "no"             "4.0.5"
# xfun          NA                    NA      NA     "yes"            "4.1.0"
# xml2          NA                    NA      NA     "yes"            "4.0.0"
# xopen         NA                    NA      NA     "no"             "4.0.0"
# yaml          NA                    NA      NA     "yes"            "4.0.0"
# zip           NA                    NA      NA     "yes"            "4.1.0"
# base          NA                    NA      NA     NA               "4.1.1"
# boot          NA                    NA      NA     "no"             "4.0.5"
# class         NA                    NA      NA     "yes"            "4.0.5"
# cluster       NA                    NA      NA     "yes"            "4.0.5"
# codetools     NA                    NA      NA     "no"             "4.0.3"
# compiler      NA                    NA      NA     NA               "4.1.1"
# datasets      NA                    NA      NA     NA               "4.1.1"
# foreign       NA                    NA      NA     "yes"            "4.0.3"
# graphics      NA                    NA      NA     "yes"            "4.1.1"
# grDevices     NA                    NA      NA     "yes"            "4.1.1"
# grid          NA                    NA      NA     "yes"            "4.1.1"
# KernSmooth    NA                    NA      NA     "yes"            "4.0.5"
# lattice       NA                    NA      NA     "yes"            "4.1.0"
# MASS          NA                    NA      NA     "yes"            "4.0.5"
# Matrix        NA                    NA      NA     "yes"            "4.1.0"
# methods       NA                    NA      NA     "yes"            "4.1.1"
# mgcv          NA                    NA      NA     "yes"            "4.1.0"
# nlme          NA                    NA      NA     "yes"            "4.0.3"
# nnet          NA                    NA      NA     "yes"            "4.0.5"
# parallel      NA                    NA      NA     "yes"            "4.1.1"
# rpart         NA                    NA      NA     "yes"            "4.0.0"
# spatial       NA                    NA      NA     "yes"            "4.0.5"
# splines       NA                    NA      NA     "yes"            "4.1.1"
# stats         NA                    NA      NA     "yes"            "4.1.1"
# stats4        NA                    NA      NA     NA               "4.1.1"
# survival      NA                    NA      NA     "yes"            "4.1.1"
# tcltk         NA                    NA      NA     "yes"            "4.1.1"
# tools         NA                    NA      NA     "yes"            "4.1.1"
# utils         NA                    NA      NA     "yes"            "4.1.1"